In [1]:
import pandas as pd

data_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'
df_ohlcv = pd.read_parquet(data_path, engine='pyarrow')
print(f'df_ohlcv.info():\n{df_ohlcv.info()}')
df_ohlcv

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4423522 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('ZWS', Timestamp('2025-10-09 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 186.3+ MB
df_ohlcv.info():
None


Adj Open  Adj High  Adj Low  Adj Close    Volume
Ticker Date                                                        
A      1999-11-18   27.2452   29.9398  23.9518    26.3470  74716417
       1999-11-19   25.7108   25.7482  23.8396    24.1764  18198348
       1999-11-22   24.7378   26.3470  23.9893    26.3470   7857766
       1999-11-23   25.4488   26.1225  23.9518    23.9518   7138322
       1999-11-24   24.0267   25.1120  23.9518    24.5881   5785607
...                     ...       ...      ...        ...       ...
ZWS    2025-10-03   46.8700   47.3700  46.6200    46.8400    780500
       2025-10-06   47.1600   47.3700  46.6700    47.2900    661000
       2025-10-07   47.1800   47.7000  46.7300    47.2200    862300
       2025-10-08   47.4600   47.7600  47.0900    47.3300    601600
       2025-10-09   47.3900   47.4200  46.6400    46.7600    454200

[4423522 rows x 5 columns]

In [2]:
# ==============================================================================
# GOLDEN COPY - COMPLETE PROJECT CODE (All Fixes Included)
# Version: Verified Portfolio and Benchmark Sharpe (ATR)  
# Date: 2025-10-15
# ==============================================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import time
import pprint
import os # Make sure os is imported for the export function later
import re

from datetime import datetime, date
from IPython.display import display, Markdown
from tqdm.auto import tqdm
from pathlib import Path
from itertools import product


pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 3000)


# --- REFACTORING PHASE 1 CODE: Feature Generation Engine ---

def generate_features(df_ohlcv: pd.DataFrame, 
                      atr_period: int = 14, 
                      quality_window: int = 252, 
                      quality_min_periods: int = 126) -> pd.DataFrame:
    """
    Generates a comprehensive DataFrame of derived features from raw OHLCV data.

    This function performs all heavy, window-based calculations upfront to be used
    by downstream analysis functions. It calculates:
    1. Technical Indicators: True Range (TR), ATR, and ATRP.
    2. Data Quality Metrics: Rolling stale percentage, median dollar volume, etc.

    Args:
        df_ohlcv: The primary DataFrame with a (Ticker, Date) MultiIndex and
                  columns for 'Adj High', 'Adj Low', 'Adj Close', 'Volume'.
        atr_period: The lookback period for the ATR's Exponential Moving Average.
        quality_window: The rolling window size for data quality metrics.
        quality_min_periods: The minimum number of observations required to have
                             a valid quality metric.

    Returns:
        A new DataFrame with the same (Ticker, Date) MultiIndex containing all
        calculated feature columns.
    """
    print("--- Starting Feature Generation ---")
    
    # Ensure the DataFrame is sorted for correct window and shift operations
    # FIX: Replaced is_lexsorted() with the current pandas attribute
    if not df_ohlcv.index.is_monotonic_increasing:
        print("Sorting index for calculation accuracy...")
        df_ohlcv = df_ohlcv.sort_index()

    # --- 1. Technical Indicator Calculation (TR, ATR, ATRP) ---
    print(f"Calculating technical indicators (ATR Period: {atr_period})...")
    
    # Group by ticker to handle each security independently
    grouped = df_ohlcv.groupby(level='Ticker')
    
    # Get the previous day's close required for True Range
    prev_close = grouped['Adj Close'].shift(1)
    
    # Calculate the three components of True Range
    high_low = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
    high_prev_close = abs(df_ohlcv['Adj High'] - prev_close)
    low_prev_close = abs(df_ohlcv['Adj Low'] - prev_close)
    
    # Combine the components to get the final TR
    tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1, skipna=False)
    
    # # Calculate the ATR using an Exponential Moving Average
    # --- FIX IS HERE ---
    # Use .transform() to apply the EWM function. 
    # This guarantees the resulting Series has the exact same index as 'tr',
    # preventing the index alignment error during the subsequent division.
    atr = tr.groupby(level='Ticker').transform(
        lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean()
    )

    # --- CHANGE 1: Removed .fillna(0) ---
    # ATRP will now be NaN on the first day, consistent with TR and ATR.
    atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)

    indicator_df = pd.DataFrame({
        'TR': tr,
        'ATR': atr,
        'ATRP': atrp
    })
    
    # --- 2. Data Quality Metric Calculation ---
    print(f"Calculating data quality metrics (Window: {quality_window} days)...")
    
    # Create intermediate flags needed for quality calculations
    is_stale = np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0)
    dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']
    has_same_volume = (grouped['Volume'].diff() == 0).astype(int)
    
    # Combine flags into a temporary DataFrame for rolling calculations
    quality_temp_df = pd.DataFrame({
        'IsStale': is_stale,
        'DollarVolume': dollar_volume,
        'HasSameVolume': has_same_volume
    }, index=df_ohlcv.index) # Explicitly set index to be safe
    
    # Perform the rolling calculations on the grouped data
    # --- FIX IS HERE ---
    # We switch to the older, more compatible dictionary-based aggregation method.
    # This syntax is understood by nearly all versions of pandas.
    rolling_result = quality_temp_df.groupby(level='Ticker').rolling(
        window=quality_window,
        min_periods=quality_min_periods
    ).agg({
        'IsStale': 'mean',
        'DollarVolume': 'median',
        'HasSameVolume': 'sum'
    })
    
    # The dictionary syntax produces columns with the original names ('IsStale', etc.).
    # We now explicitly rename them to our desired final names.
    rolling_result = rolling_result.rename(columns={
        'IsStale': 'RollingStalePct',
        'DollarVolume': 'RollMedDollarVol', # <-- RENAMED HERE
        'HasSameVolume': 'RollingSameVolCount'
    })

    # The index after a grouped rolling operation is hierarchical.
    # We remove the outermost 'Ticker' level to restore the original index structure.
    rolling_quality = rolling_result.reset_index(level=0, drop=True)

    # --- 3. Combine All Features ---
    print("Combining all feature sets...")
    features_df = pd.concat([indicator_df, rolling_quality], axis=1)
    
    print("✅ Feature generation complete.")
    return features_df

def test_features_df(features_df: pd.DataFrame, 
                     df_ohlcv: pd.DataFrame, 
                     test_ticker: str = 'AAPL',
                     spot_check_date: str = '2020-03-20'):
    """
    Runs a suite of tests to verify the correctness of the generated features_df.

    Args:
        features_df: The generated DataFrame from the generate_features function.
        df_ohlcv: The original source OHLCV DataFrame.
        test_ticker: A common, liquid ticker to use for specific value checks.
    """
    print(f"\n--- Running Verification Suite for features_df (Test Ticker: {test_ticker}) ---")
    
    # --- Test 1: Structural Integrity ---
    print("\n[Test 1: Structural Integrity]")
    assert features_df.index.equals(df_ohlcv.index), "FAIL: Index does not match original df_ohlcv."
    print("  ✅ PASS: Index matches original df_ohlcv.")
    
    expected_cols = ['TR', 'ATR', 'ATRP', 'RollingStalePct', 'RollMedDollarVol', 'RollingSameVolCount']
    assert all(col in features_df.columns for col in expected_cols), "FAIL: Missing one or more expected columns."
    print("  ✅ PASS: All expected feature columns are present.")
    print(f"  - DataFrame Info:")
    features_df.info(verbose=False, memory_usage='deep')


    # --- Test 2: ATR Calculation Logic ---
    print("\n[Test 2: ATR Logic Verification]")
    ticker_features = features_df.loc[test_ticker]
    
    # Test 2a: First TR value should be NaN (since prev_close is NaN)
    first_tr = ticker_features['TR'].iloc[0]
    assert pd.isna(first_tr), f"FAIL: First TR value for {test_ticker} should be NaN, but got {first_tr}."
    print(f"  ✅ PASS: First TR value for {test_ticker} is NaN as expected.")

    # Test 2b: The first valid ATR should equal the first valid TR (EWM cold start behavior)
    first_valid_tr_val = ticker_features['TR'].dropna().iloc[0]
    first_valid_atr_val = ticker_features['ATR'].dropna().iloc[0]
    assert np.isclose(first_valid_tr_val, first_valid_atr_val), \
        f"FAIL: First valid ATR ({first_valid_atr_val}) should equal first valid TR ({first_valid_tr_val})."
    print("  ✅ PASS: First valid ATR correctly seeded with first valid TR.")


    # --- Test 3: Rolling Quality Metrics Logic ---
    print("\n[Test 3: Rolling Quality Metrics Logic Verification]")
    quality_min_periods = 126 # Should match the parameter used in generation
    
    # Test 3a: Check for leading NaNs
    first_valid_quality_idx = ticker_features['RollingStalePct'].first_valid_index()
    if first_valid_quality_idx is None:
        print(f"  - INFO: No valid quality metrics found for {test_ticker} (likely too little data). Skipping test.")
    else:
        position_of_first_valid = ticker_features.index.get_loc(first_valid_quality_idx)
        assert position_of_first_valid == quality_min_periods - 1, \
            f"FAIL: First valid quality metric should appear at index {quality_min_periods - 1}, but appeared at {position_of_first_valid}."
        print(f"  ✅ PASS: Leading NaNs are present for the first {quality_min_periods - 1} periods as expected.")


    # --- Test 4: Spot Check Against Manual Calculation ---
    print("\n[Test 4: Spot Check vs. Manual Calculation]")
    # # Choose a specific date for a manual calculation
    # spot_check_date = '2020-03-20' # A volatile day for a good test
    
    # Manual TR Calculation
    today_data = df_ohlcv.loc[(test_ticker, spot_check_date)]
    yesterday_data = df_ohlcv.loc[(test_ticker, pd.to_datetime(spot_check_date) - pd.Timedelta(days=1))] # simple lookback for test
    
    manual_h_l = today_data['Adj High'] - today_data['Adj Low']
    manual_h_pc = abs(today_data['Adj High'] - yesterday_data['Adj Close'])
    manual_l_pc = abs(today_data['Adj Low'] - yesterday_data['Adj Close'])
    manual_tr = max(manual_h_l, manual_h_pc, manual_l_pc)
    
    code_tr = ticker_features.loc[spot_check_date]['TR']
    
    assert np.isclose(manual_tr, code_tr), f"FAIL: Manual TR ({manual_tr:.4f}) does not match code TR ({code_tr:.4f}) on {spot_check_date}."
    print(f"  ✅ PASS: Manually calculated TR on {spot_check_date} matches code's TR.")
    
    print("\n--- ✅ All Verification Tests Passed ---")

def export_ticker_data(ticker_to_export: str, 
                         df_ohlcv: pd.DataFrame, 
                         features_df: pd.DataFrame, 
                         output_dir: str = 'export_csv'):
    """
    Exports the raw OHLCV data and the corresponding calculated features for a 
    single ticker to two separate CSV files.

    This function is designed for easy manual verification of data and calculations.
    It will create the output directory if it does not exist.

    Args:
        ticker_to_export: The ticker symbol to export (e.g., 'AAPL').
        df_ohlcv: The main DataFrame containing the raw OHLCV data with a 
                  (Ticker, Date) MultiIndex.
        features_df: The DataFrame containing the calculated features with a 
                     (Ticker, Date) MultiIndex.
        output_dir: The directory where the CSV files will be saved. 
                    Defaults to 'export_csv'.
    """
    print(f"--- Attempting to export data for ticker: {ticker_to_export} ---")
    
    # --- 1. Ensure the output directory exists ---
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Output directory '{output_dir}' is ready.")
    except OSError as e:
        print(f"Error: Could not create directory '{output_dir}'. Reason: {e}")
        return

    # --- 2. Isolate the data for the specified ticker ---
    try:
        # Use .loc to select all rows for the given ticker from the MultiIndex
        ticker_ohlcv = df_ohlcv.loc[ticker_to_export]
        ticker_features = features_df.loc[ticker_to_export]
        
        if ticker_ohlcv.empty:
            print(f"Warning: No OHLCV data found for ticker '{ticker_to_export}'. Cannot export.")
            return
            
        print(f"Found {len(ticker_ohlcv)} rows of data for '{ticker_to_export}'.")
        
    except KeyError:
        print(f"Error: Ticker '{ticker_to_export}' not found in one or both of the DataFrames. Please check the symbol.")
        return
    except Exception as e:
        print(f"An unexpected error occurred while accessing data: {e}")
        return

    # --- 3. Construct file paths and export to CSV ---
    try:
        # Define the full path for each output file
        ohlcv_filename = f"{ticker_to_export}_ohlcv.csv"
        features_filename = f"{ticker_to_export}_features.csv"
        
        ohlcv_filepath = os.path.join(output_dir, ohlcv_filename)
        features_filepath = os.path.join(output_dir, features_filename)
        
        # Export the DataFrames to CSV. The index (Date) will be included.
        ticker_ohlcv.to_csv(ohlcv_filepath)
        ticker_features.to_csv(features_filepath)
        
        print("\n✅ Export successful!")
        print(f"   - Raw OHLCV data saved to: {ohlcv_filepath}")
        print(f"   - Calculated features saved to: {features_filepath}")

    except Exception as e:
        print(f"Error: Failed to write data to CSV files. Reason: {e}")

def create_synthetic_ticker_data(
    ticker_name: str = 'SYNTH', 
    num_days: int = 50,
    num_zero_volume_days: int = 5,
    num_flat_price_days: int = 3
) -> pd.DataFrame:
    """
    Creates a synthetic OHLCV DataFrame with predictable patterns and randomly injected
    stale data conditions for robust testing.

    Args:
        ticker_name: The name for the synthetic ticker.
        num_days: The total number of days for the ticker's history.
        num_zero_volume_days: The number of random days to set Volume to 0.
        num_flat_price_days: The number of random days to set High == Low.

    Returns:
        A pandas DataFrame with a (Ticker, Date) MultiIndex.
    """
    print(f"--- Creating synthetic data for '{ticker_name}' with {num_days} days ---")
    
    # 1. Create a base DataFrame with "normal" data
    dates = pd.to_datetime(pd.date_range(start='2023-01-01', periods=num_days, freq='B'))
    data = {
        'Adj Open': 100.0, 'Adj High': 102.0, 'Adj Low': 98.0,
        'Adj Close': 100.0, 'Volume': 1_000_000
    }
    df = pd.DataFrame(data, index=dates)
    df['Adj Close'] = df['Adj Close'] + np.random.randn(num_days) * 0.5 # Add some noise

    # 2. Define a "protected" window for specific verification tests.
    # The `verify_synthetic_ticker_features` function depends on this exact window.
    # We will not inject random stale days here.
    protected_start_idx, protected_end_idx = 10, 20
    
    # 3. Inject random "stale" days OUTSIDE the protected window
    available_indices = df.index.drop(df.index[protected_start_idx:protected_end_idx])
    
    # Inject zero-volume days
    if num_zero_volume_days > 0:
        if len(available_indices) < num_zero_volume_days:
            raise ValueError("Not enough available days to inject zero-volume days.")
        zero_vol_dates = np.random.choice(available_indices, num_zero_volume_days, replace=False)
        df.loc[zero_vol_dates, 'Volume'] = 0
        print(f"  - Injected {num_zero_volume_days} random zero-volume 'stale' days.")
        # Update available indices to avoid overlap
        available_indices = available_indices.drop(zero_vol_dates)

    # Inject flat-price days (High == Low)
    if num_flat_price_days > 0:
        if len(available_indices) < num_flat_price_days:
            raise ValueError("Not enough available days to inject flat-price days.")
        flat_price_dates = np.random.choice(available_indices, num_flat_price_days, replace=False)
        # Set High and Low to be the same as the Close price for that day
        df.loc[flat_price_dates, 'Adj High'] = df.loc[flat_price_dates, 'Adj Close']
        df.loc[flat_price_dates, 'Adj Low'] = df.loc[flat_price_dates, 'Adj Close']
        print(f"  - Injected {num_flat_price_days} random flat-price 'stale' days.")

    # 4. Inject the specific, hand-crafted patterns inside the protected window for verification
    print("  - Injecting specific patterns for programmatic verification...")
    # Pattern for RollingStalePct: 2 stale days in 10 (20%)
    df.iloc[10, df.columns.get_loc('Volume')] = 0  # Stale day (zero volume)
    df.iloc[11, df.columns.get_loc('Adj High')] = 99.0 # Stale day (High == Low)
    df.iloc[11, df.columns.get_loc('Adj Low')] = 99.0
    
    # Pattern for RollingMedianVolume
    for i in range(10):
        df.iloc[10 + i, df.columns.get_loc('Adj Close')] = 100.0 # Standardize price for easy median calc
        df.iloc[10 + i, df.columns.get_loc('Volume')] = (i + 1) * 10000

    # Pattern for RollingSameVolCount
    df.iloc[15, df.columns.get_loc('Volume')] = 77777
    df.iloc[16, df.columns.get_loc('Volume')] = 77777
    df.iloc[17, df.columns.get_loc('Volume')] = 77777
    
    # 5. Set the MultiIndex
    df['Ticker'] = ticker_name
    df = df.set_index(['Ticker', df.index])
    df.index.names = ['Ticker', 'Date']
    
    print("✅ Synthetic data created successfully.")
    return df

def verify_synthetic_ticker_features(features_df: pd.DataFrame, 
                                       ticker_name: str = 'SYNTH',
                                       quality_window: int = 10):
    """
    Verifies the quality metric calculations on the features_df generated from
    the synthetic ticker data.

    Args:
        features_df: The DataFrame of calculated features.
        ticker_name: The name of the synthetic ticker.
        quality_window: The rolling window used, which must match the window
                        of the synthetic data pattern.
    """
    print(f"\n--- Running Verification on Synthetic Ticker '{ticker_name}' ---")
    
    # --- Expected values based on our synthetic data design ---
    EXPECTED_STALE_PCT = 0.20  # 2 stale days out of 10
    EXPECTED_MEDIAN_DOLLAR_VOL = 5_500_000.0 # median of (10k..100k) * price of 100
    EXPECTED_SAME_VOL_COUNT = 2.0 # Three consecutive days gives two 'diff() == 0' events

    try:
        # Isolate the features for our synthetic ticker
        ticker_features = features_df.loc[ticker_name]
        
        # The first valid calculation will be on the last day of our 10-day window.
        # The window starts at index 10 and has a length of 10, so it ends at index 19.
        verification_date = ticker_features.index[19]
        
        print(f"Verifying calculations on date: {verification_date.date()}")
        
        # Get the calculated values from the DataFrame
        calculated_values = ticker_features.loc[verification_date]
        stale_pct = calculated_values['RollingStalePct']
        # --- CHANGE 2 (continued): Accessing the renamed column ---
        median_vol = calculated_values['RollMedDollarVol'] 
        same_vol_count = calculated_values['RollingSameVolCount']
        
        # --- Perform Assertions ---
        print("\n[Test 1: RollingStalePct]")
        assert np.isclose(stale_pct, EXPECTED_STALE_PCT), f"FAIL: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}"
        print(f"  ✅ PASS: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}")

        print("\n[Test 2: RollMedDollarVol]")
        assert np.isclose(median_vol, EXPECTED_MEDIAN_DOLLAR_VOL), f"FAIL: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}"
        print(f"  ✅ PASS: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}")

        print("\n[Test 3: RollingSameVolCount]")
        assert np.isclose(same_vol_count, EXPECTED_SAME_VOL_COUNT), f"FAIL: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}"
        print(f"  ✅ PASS: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}")

        print("\n--- ✅ All Synthetic Data Verification Tests Passed ---")

    except KeyError:
        print(f"FAIL: Ticker '{ticker_name}' not found in features_df.")
    except IndexError:
        print("FAIL: Not enough data in features_df to run verification. Check num_days.")
    except Exception as e:
        print(f"An unexpected error occurred during verification: {e}")

# --- A. HELPER FUNCTIONS ---

def calculate_gain(price_series: pd.Series):
    """Calculates the total gain over a series of prices."""
    # Ensure there are at least two data points to calculate a gain
    if price_series.dropna().shape[0] < 2: return np.nan
    # Use forward-fill for the end price and back-fill for the start price
    # to handle potential NaNs at the beginning or end of the series.
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series: pd.Series):
    """Calculates the annualized Sharpe ratio from a series of daily returns."""
    # Ensure there are at least two returns to calculate a standard deviation
    if return_series.dropna().shape[0] < 2: return np.nan
    std_dev = return_series.std()
    # Avoid division by zero if returns are constant
    if std_dev > 0 and std_dev != np.inf:
        return (return_series.mean() / std_dev) * np.sqrt(252)
    return np.nan

def calculate_sharpe_atr(return_series: pd.Series, atrp_series: pd.Series):
    """Calculates a Sharpe-like ratio using mean ATRP as the denominator."""
    # Ensure there are returns and that ATRP data is valid
    if return_series.dropna().shape[0] < 2 or atrp_series.dropna().empty:
        return np.nan
        
    mean_return = return_series.mean()
    mean_atrp = atrp_series.mean()
    
    # Avoid division by zero
    if mean_atrp > 0 and mean_atrp != np.inf:
        return mean_return / mean_atrp
        
    return np.nan

def print_nested(d, indent=0, width=4):
    """Pretty-print any nested dict/list/tuple combination."""
    spacing = ' ' * indent
    if isinstance(d, dict):
        for k, v in d.items():
            print(f'{spacing}{k}:')
            print_nested(v, indent + width, width)
    elif isinstance(d, (list, tuple)):
        for item in d:
            print_nested(item, indent, width)
    else:
        print(f'{spacing}{d}')

# --- B. MODULAR METRIC CALCULATION ENGINE ---

def calculate_price_metric(metric_data: dict) -> pd.Series:
    """
    Calculates the 'Price' metric (total gain) over the calculation period.

    Args:
        metric_data: A dictionary containing pre-calculated data Series.
                     Requires 'calc_close' (pd.DataFrame): The close prices for the calc period.

    Returns:
        A pandas Series of the calculated metric values, indexed by Ticker.
    """
    calc_close = metric_data['calc_close']
    
    # Ensure there are at least two data points to calculate a gain
    if len(calc_close) < 2:
        return pd.Series(dtype='float64', index=calc_close.columns)

    first_prices = calc_close.bfill().iloc[0]
    last_prices = calc_close.ffill().iloc[-1]
    
    # The division of two Series aligns by index (Ticker), which is what we want.
    price_metric = last_prices / first_prices
    
    return price_metric.dropna()

def calculate_sharpe_metric(metric_data: dict) -> pd.Series:
    """
    Calculates the annualized 'Sharpe' ratio metric over the calculation period.

    Args:
        metric_data: A dictionary containing pre-calculated data Series.
                     Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.

    Returns:
        A pandas Series of the calculated metric values, indexed by Ticker.
    """
    daily_returns = metric_data['daily_returns']
    
    # Ensure there's enough data to calculate standard deviation
    if len(daily_returns.dropna()) < 2:
        return pd.Series(dtype='float64', index=daily_returns.columns)
    
    mean_returns = daily_returns.mean()
    std_returns = daily_returns.std()
    
    # Standard annualized Sharpe Ratio calculation. Avoid division by zero.
    # We replace resulting NaNs/infs with 0 to handle cases of zero volatility.
    sharpe_ratio = (mean_returns / std_returns * np.sqrt(252))
    
    return sharpe_ratio.replace([np.inf, -np.inf], np.nan).fillna(0)
    
def calculate_sharpe_atr_metric(metric_data: dict) -> pd.Series:
    """
    Calculates the 'Sharpe (ATR)' metric over the calculation period.

    Args:
        metric_data: A dictionary containing pre-calculated data Series.
                     Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.
                     Requires 'atrp' (pd.Series): Mean ATRP for each ticker over the period.

    Returns:
        A pandas Series of the calculated metric values, indexed by Ticker.
    """
    daily_returns = metric_data['daily_returns']
    atrp = metric_data['atrp']
    
    mean_returns = daily_returns.mean()
    
    # ATRP-based Sharpe. Avoid division by zero.
    # We replace resulting NaNs/infs with 0.
    sharpe_atr = mean_returns / atrp
    
    return sharpe_atr.replace([np.inf, -np.inf], np.nan).fillna(0)


def calculate_buy_and_hold_performance(df_close: pd.DataFrame,
                                       features_df: pd.DataFrame,
                                       tickers: list,
                                       start_date: pd.Timestamp,
                                       end_date: pd.Timestamp):
    """
    Calculates the performance series for an equal-weighted, buy-and-hold portfolio.

    [CORRECTED to calculate a value-weighted ATRP consistent with a buy-and-hold strategy]

    Args:
        df_close (pd.DataFrame): DataFrame of adjusted close prices with dates as index and tickers as columns.
        features_df (pd.DataFrame): DataFrame with a MultiIndex ('Ticker', 'Date') containing features like 'ATRP'.
        tickers (list): A list of ticker symbols to include in the portfolio.
        start_date (pd.Timestamp): The starting date for the performance calculation.
        end_date (pd.Timestamp): The ending date for the performance calculation.

    Returns:
        tuple: A tuple containing:
            - pd.Series: The normalized portfolio value over time (buy-and-hold).
            - pd.Series: The daily portfolio returns (buy-and-hold).
            - pd.Series: The daily VALUE-WEIGHTED portfolio ATRP (buy-and-hold).
    """
    if not tickers or tickers is None:
        empty_series = pd.Series(dtype='float64')
        return empty_series, empty_series, empty_series

    # 1. Calculate Portfolio Return Series (This part was already correct)
    prices_raw = df_close[tickers].loc[start_date:end_date]
    if prices_raw.dropna(how='all').empty:
        empty_series = pd.Series(dtype='float64')
        return empty_series, empty_series, empty_series

    prices_norm = prices_raw.div(prices_raw.bfill().iloc[0])
    value_series = prices_norm.mean(axis=1)
    return_series = value_series.pct_change()

    # 2. Calculate Portfolio ATRP Series (This part is NOW CORRECTED)
    # Get the individual ATRP values for each stock in the portfolio
    full_period_index = pd.MultiIndex.from_product(
        [tickers, return_series.index],
        names=['Ticker', 'Date']
    )
    portfolio_atrp_features = features_df.loc[features_df.index.intersection(full_period_index)]
    portfolio_atrp_daily_unstacked = portfolio_atrp_features['ATRP'].unstack(level='Ticker')

    # +++ NEW LOGIC: Calculate daily weights based on market value drift +++
    # The sum of normalized prices each day represents the total portfolio value
    # relative to the start.
    daily_portfolio_total_value = prices_norm.sum(axis=1)

    # The weight of each asset is its normalized price divided by the daily total.
    # .div(..., axis=0) ensures row-wise division.
    daily_weights = prices_norm.div(daily_portfolio_total_value, axis=0)
    
    # Align the ATRP DataFrame columns with the weights DataFrame columns to ensure correct multiplication
    # This handles cases where a ticker might have a price but no ATRP value on a given day or vice-versa
    aligned_weights, aligned_atrp = daily_weights.align(portfolio_atrp_daily_unstacked, join='inner', axis=1)

    # The portfolio ATRP is the weighted sum of the individual ATRPs.
    # This is calculated by element-wise multiplication of weights and ATRPs,
    # then summing across the tickers for each day.
    atrp_series = (aligned_weights * aligned_atrp).sum(axis=1)
    # --- END OF CORRECTED LOGIC ---

    return value_series, return_series, atrp_series



# The single source of truth for all available ranking metrics.
# Maps the user-facing name to the calculation function.
METRIC_REGISTRY = {
    'Price': calculate_price_metric,
    'Sharpe': calculate_sharpe_metric,
    'Sharpe (ATR)': calculate_sharpe_atr_metric,
}

print("✅ Metric Registry Initialized with:", list(METRIC_REGISTRY.keys()))

# --- B. THE CORE CALCULATION ENGINE ---

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           features_df,
#                           debug=False):
#     """
#     Runs a single step of the walk-forward analysis with a strict, pre-emptive
#     check to ensure the full period is available.
#     """
#     debug_data = {} if debug else None

#     # 1. Determine exact date ranges with a NEW pre-emptive check
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)

#     # +++ THIS IS THE NEW PRE-EMPTIVE CHECK LOGIC +++
#     # Calculate the desired end index without clamping first.
#     desired_viz_end_idx = start_idx + calc_period + fwd_period
    
#     # Check if the desired end index is out of bounds.
#     if desired_viz_end_idx >= len(master_trading_days):
#         last_available_date = master_trading_days[-1].date()
#         required_days = calc_period + fwd_period
#         available_days = len(master_trading_days) - start_idx
#         error_msg = (f"Not enough data for the full requested period. "
#                      f"Required: {required_days} days, Available: {available_days} days until {last_available_date}.")
#         return ({'error': error_msg}, None)
#     # --- END OF NEW CHECK ---

#     # If the check passes, we know the full period is available.
#     # The 'min' calls are now just a redundant safety measure.
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     # (The rest of the function remains completely unchanged...)
#     # 2. Slice data for the calculation period
#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     # 3. Calculate INTERMEDIATE data required for the ranking metrics
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     valid_tickers = calc_close.columns
#     calc_period_index = pd.MultiIndex.from_product([valid_tickers, calc_close.index], names=['Ticker', 'Date'])
#     features_in_period = features_df.loc[features_df.index.intersection(calc_period_index)]
#     atrp = features_in_period.groupby(level='Ticker')['ATRP'].mean()

#     # 4. Calculate all ranking metrics by iterating through the METRIC_REGISTRY
#     metric_ingredients = { 'calc_close': calc_close, 'daily_returns': daily_returns, 'atrp': atrp, }
#     metric_values = {}
#     for name, func in METRIC_REGISTRY.items():
#         metric_values[name] = func(metric_ingredients)
#     if metric not in metric_values or metric_values[metric].empty:
#         return ({'error': f"Metric '{metric}' could not be calculated or resulted in no valid tickers."}, None)

#     # 5. Rank tickers and select the portfolio
#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     # 6. Calculate Portfolio & Benchmark Performance
#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')
#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
#         if benchmark_price_series is not None:
#             bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#             calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#             fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         if benchmark_price_series is not None:
#             calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#             fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     full_period_index = pd.MultiIndex.from_product([tickers_to_display, portfolio_return_series.index], names=['Ticker', 'Date'])
#     portfolio_atrp_features = features_df.loc[features_df.index.intersection(full_period_index)]
#     portfolio_atrp_daily_unstacked = portfolio_atrp_features['ATRP'].unstack(level='Ticker')
#     portfolio_atrp_series = portfolio_atrp_daily_unstacked.mean(axis=1)
#     if benchmark_ticker in df_close_full.columns:
#         benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[safe_start_date:safe_viz_end_date]
#     else:
#         benchmark_atrp_series = pd.Series(dtype='float64')
#     calc_portfolio_atrp = portfolio_atrp_series.loc[:actual_calc_end_ts]
#     fwd_portfolio_atrp = portfolio_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     calc_benchmark_atrp = benchmark_atrp_series.loc[:actual_calc_end_ts]
#     fwd_benchmark_atrp = benchmark_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)
#     perf_data['calc_p_sharpe_atr'] = calculate_sharpe_atr(calc_portfolio_returns, calc_portfolio_atrp)
#     perf_data['fwd_p_sharpe_atr'] = calculate_sharpe_atr(fwd_portfolio_returns, fwd_portfolio_atrp)
#     perf_data['full_p_sharpe_atr'] = calculate_sharpe_atr(portfolio_return_series, portfolio_atrp_series)
#     perf_data['calc_b_sharpe_atr'] = calculate_sharpe_atr(calc_benchmark_returns, calc_benchmark_atrp)
#     perf_data['fwd_b_sharpe_atr'] = calculate_sharpe_atr(fwd_benchmark_returns, fwd_benchmark_atrp)
#     perf_data['full_b_sharpe_atr'] = calculate_sharpe_atr(benchmark_return_series, benchmark_atrp_series)
#     if debug:
#         df_ranking_base = pd.DataFrame({'MeanDailyReturn': daily_returns.mean(),'StdDevDailyReturn': daily_returns.std(),'MeanATRP': atrp})
#         df_metrics = pd.DataFrame(metric_values)
#         df_metrics.columns = [f'Metric_{col}' for col in df_metrics.columns]
#         df_ranking = df_ranking_base.join(df_metrics, how='left')
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)
#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace
#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)



def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
                          master_trading_days,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker,
                          features_df,
                          debug=False):
    """
    Runs a single step of the walk-forward analysis with a strict, pre-emptive
    check to ensure the full period is available.
    """
    debug_data = {} if debug else None

    # 1. Determine exact date ranges with a NEW pre-emptive check
    try:
        start_idx = master_trading_days.get_loc(start_date)
    except KeyError:
        return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)

    # +++ THIS IS THE NEW PRE-EMPTIVE CHECK LOGIC +++
    # Calculate the desired end index without clamping first.
    desired_viz_end_idx = start_idx + calc_period + fwd_period
    
    # Check if the desired end index is out of bounds.
    if desired_viz_end_idx >= len(master_trading_days):
        last_available_date = master_trading_days[-1].date()
        required_days = calc_period + fwd_period
        available_days = len(master_trading_days) - start_idx
        error_msg = (f"Not enough data for the full requested period. "
                     f"Required: {required_days} days, Available: {available_days} days until {last_available_date}.")
        return ({'error': error_msg}, None)
    # --- END OF NEW CHECK ---

    # If the check passes, we know the full period is available.
    # The 'min' calls are now just a redundant safety measure.
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

    safe_start_date = master_trading_days[start_idx]
    safe_calc_end_date = master_trading_days[calc_end_idx]
    safe_viz_end_date = master_trading_days[viz_end_idx]
    
    if safe_start_date >= safe_calc_end_date:
        return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

    # (The rest of the function remains completely unchanged...)
    # 2. Slice data for the calculation period
    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all')
    if calc_close.shape[1] == 0 or len(calc_close) < 2:
        return ({'error': "Not enough data in calc period."}, None)

    # 3. Calculate INTERMEDIATE data required for the ranking metrics
    daily_returns = calc_close.bfill().ffill().pct_change()
    valid_tickers = calc_close.columns
    calc_period_index = pd.MultiIndex.from_product([valid_tickers, calc_close.index], names=['Ticker', 'Date'])
    features_in_period = features_df.loc[features_df.index.intersection(calc_period_index)]
    atrp = features_in_period.groupby(level='Ticker')['ATRP'].mean()

    # 4. Calculate all ranking metrics by iterating through the METRIC_REGISTRY
    metric_ingredients = { 'calc_close': calc_close, 'daily_returns': daily_returns, 'atrp': atrp, }
    metric_values = {}
    for name, func in METRIC_REGISTRY.items():
        metric_values[name] = func(metric_ingredients)
    if metric not in metric_values or metric_values[metric].empty:
        return ({'error': f"Metric '{metric}' could not be calculated or resulted in no valid tickers."}, None)

    # 5. Rank tickers and select the portfolio
    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display:
        return ({'error': "No tickers found for the selected rank."}, None)

#################################
    # 6. Calculate Portfolio & Benchmark Performance
    # +++ THIS ENTIRE SECTION IS REFACTORED +++
    actual_calc_end_ts = calc_close.index.max()

    # +++ FIX: RE-INTRODUCE THE `normalized_plot_data` CALCULATION HERE +++
    # This is needed for the debug trace and the final plot output.
    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    # --- END OF FIX ---

    # Use the new central function for the portfolio
    portfolio_series, portfolio_return_series, portfolio_atrp_series = \
        calculate_buy_and_hold_performance(df_close_full, features_df, tickers_to_display,
                                           safe_start_date, safe_viz_end_date)

    # Use the new central function for the benchmark (a portfolio of one)
    if benchmark_ticker in df_close_full.columns:
        benchmark_price_series, benchmark_return_series, benchmark_atrp_series = \
            calculate_buy_and_hold_performance(df_close_full, features_df, [benchmark_ticker],
                                               safe_start_date, safe_viz_end_date)
    else:
        benchmark_price_series = pd.Series(dtype='float64')
        benchmark_return_series = pd.Series(dtype='float64')
        benchmark_atrp_series = pd.Series(dtype='float64')

    # Now, split the generated series into calc and fwd periods
    # This logic remains, but it's much cleaner as it operates on the final series
    try:
        boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
        calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
        fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
        calc_portfolio_atrp = portfolio_atrp_series.iloc[:boundary_loc + 1]
        fwd_portfolio_atrp = portfolio_atrp_series.iloc[boundary_loc + 1:]
        
        if not benchmark_return_series.empty:
            bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
            calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
            fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
            calc_benchmark_atrp = benchmark_atrp_series.iloc[:bm_boundary_loc + 1]
            fwd_benchmark_atrp = benchmark_atrp_series.iloc[bm_boundary_loc + 1:]
        else:
            calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
            calc_benchmark_atrp, fwd_benchmark_atrp = pd.Series(dtype='float64'), pd.Series(dtype='float64')

    except (KeyError, IndexError): # Fallback for edge cases
        calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
        fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
        calc_portfolio_atrp = portfolio_atrp_series.loc[:actual_calc_end_ts]
        fwd_portfolio_atrp = portfolio_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
        
        if not benchmark_return_series.empty:
            calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
            fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]
            calc_benchmark_atrp = benchmark_atrp_series.loc[:actual_calc_end_ts]
            fwd_benchmark_atrp = benchmark_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
        else:
            calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
            calc_benchmark_atrp, fwd_benchmark_atrp = pd.Series(dtype='float64'), pd.Series(dtype='float64')


###################

    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
    perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
    perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)
    perf_data['calc_p_sharpe_atr'] = calculate_sharpe_atr(calc_portfolio_returns, calc_portfolio_atrp)
    perf_data['fwd_p_sharpe_atr'] = calculate_sharpe_atr(fwd_portfolio_returns, fwd_portfolio_atrp)
    perf_data['full_p_sharpe_atr'] = calculate_sharpe_atr(portfolio_return_series, portfolio_atrp_series)
    perf_data['calc_b_sharpe_atr'] = calculate_sharpe_atr(calc_benchmark_returns, calc_benchmark_atrp)
    perf_data['fwd_b_sharpe_atr'] = calculate_sharpe_atr(fwd_benchmark_returns, fwd_benchmark_atrp)
    perf_data['full_b_sharpe_atr'] = calculate_sharpe_atr(benchmark_return_series, benchmark_atrp_series)
    if debug:
        df_ranking_base = pd.DataFrame({'MeanDailyReturn': daily_returns.mean(),'StdDevDailyReturn': daily_returns.std(),'MeanATRP': atrp})
        df_metrics = pd.DataFrame(metric_values)
        df_metrics.columns = [f'Metric_{col}' for col in df_metrics.columns]
        df_ranking = df_ranking_base.join(df_metrics, how='left')
        df_ranking.index.name = 'Ticker'
        debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)
    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])
    if debug:
        df_trace = normalized_plot_data.copy()
        df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
        df_trace['Norm_Price_Portfolio'] = portfolio_series
        if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
            norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
            df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
        for col in df_trace.columns:
            if 'Norm_Price' in col:
                df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
        debug_data['portfolio_trace'] = df_trace
    final_results = {
        'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
        'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
        'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
        'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
        'error': None
    }
    return (final_results, debug_data)




# --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

def get_eligible_universe(features_df, filter_date, thresholds):
    """Filters the universe of tickers based on quality metrics for a given date."""
    filter_date_ts = pd.to_datetime(filter_date)
    # The index is now the comprehensive features_df
    date_index = features_df.index.get_level_values('Date').unique().sort_values()
    
    if filter_date_ts < date_index[0]:
        print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
        return []
        
    # Find the most recent date with quality data on or before the filter date
    valid_prior_dates = date_index[date_index <= filter_date_ts]
    if valid_prior_dates.empty:
        print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
        return []
        
    actual_date_to_use = valid_prior_dates[-1]
    if actual_date_to_use.date() != filter_date_ts.date():
        print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")

    metrics_on_date = features_df.xs(actual_date_to_use, level='Date')
    
    # Apply filters using the new column names from features_df
    mask = ((metrics_on_date['RollMedDollarVol'] >= thresholds['min_median_dollar_volume']) & # <-- RENAMED
            (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
            (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
            
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers

# --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

def plot_walk_forward_analyzer(df_ohlcv,
                               default_start_date=None, default_calc_period=126, default_fwd_period=63,
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
                               quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
                               debug=False):
    # (No changes to the initial setup part of this function...)
    print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")
    print("--- Generating all features upfront... ---")
    features_df = generate_features(df_ohlcv)
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
    fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
    if default_metric not in METRIC_REGISTRY:
        fallback_metric = list(METRIC_REGISTRY.keys())[0]
        print(f"⚠️ Warning: Default metric '{default_metric}' not in registry. Using '{fallback_metric}'.")
        default_metric = fallback_metric
    metric_dropdown = widgets.Dropdown(options=list(METRIC_REGISTRY.keys()), value=default_metric, description='Metric:')
    rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
    rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container, debug_data_container = [None], [None]
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date_raw = pd.to_datetime(start_date_picker.value)
        start_date_idx = master_trading_days.searchsorted(start_date_raw)
        if start_date_idx >= len(master_trading_days):
            with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
        actual_start_date = master_trading_days[start_date_idx]
        with ticker_list_output:
            if start_date_raw.date() != actual_start_date.date():
                print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")
        calc_period = calc_period_input.value; fwd_period = fwd_period_input.value; metric = metric_dropdown.value
        rank_start = rank_start_input.value; rank_end = rank_end_input.value; benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
        if rank_start < 1 or calc_period < 2 or fwd_period < 1:
            with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return
        required_days = calc_period + fwd_period
        if start_date_idx + required_days > len(master_trading_days):
            available_days = len(master_trading_days) - start_date_idx; last_available_date = master_trading_days[-1].date()
            with ticker_list_output:
                print(f"Error: Not enough data for the requested period.\n  Start Date: {actual_start_date.date()}\n  Required Days: {calc_period} (calc) + {fwd_period} (fwd) = {required_days}\n  Available Days from Start: {available_days} (until {last_available_date})\n  Please shorten the 'Calc Period' / 'Fwd Period' or choose an earlier 'Start Date'.")
            return
        eligible_tickers = get_eligible_universe(features_df, actual_start_date, quality_thresholds)
        if not eligible_tickers:
            with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        results, debug_output = run_walk_forward_step(df_close_full, df_high_full, df_low_full, master_trading_days, actual_start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker, features_df=features_df, debug=debug)
        if results.get('error'):
            with ticker_list_output: print(f"Error: {results['error']}"); return
        period_dates = {'calc_period_start': results['safe_start_date'], 'calc_period_end': results['actual_calc_end_ts'], 'forward_period_start': results['actual_calc_end_ts'], 'forward_period_end': results['safe_viz_end_date']}
        run_parameters = {'calc_period': calc_period, 'fwd_period': fwd_period, 'rank_metric': metric, 'rank_start': rank_start, 'rank_end': rank_end, 'benchmark_ticker': benchmark_ticker}
        results.update(period_dates); results.update(run_parameters)
        if debug_output is not None and isinstance(debug_output, dict):
            debug_output.update(period_dates); debug_output.update(run_parameters)
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
                    trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark.values, f"Benchmark ({benchmark_ticker})", True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
        results_container[0] = results; debug_data_container[0] = debug_output
        with ticker_list_output:
            print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            pprint.pprint(results['tickers_to_display'])
            p = results['performance_data']
            
            # --- START OF MODIFIED BLOCK ---
            rows = []
            # Gain Metrics
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p.get('full_b_gain')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': '== Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            
            # Standard Sharpe Metrics
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p.get('full_b_sharpe')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': '== Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})

            # Sharpe (ATR) Metrics
            rows.append({'Metric': 'Group Portfolio Sharpe (ATR)', 'Full': p['full_p_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr']})
            if not np.isnan(p.get('full_b_sharpe_atr')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe (ATR)', 'Full': p['full_b_sharpe_atr'], 'Calc': p['calc_b_sharpe_atr'], 'Fwd': p['fwd_b_sharpe_atr']})
                rows.append({'Metric': '== Sharpe (ATR) Delta (vs Bm)', 'Full': p['full_p_sharpe_atr'] - p['full_b_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'] - p['calc_b_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr'] - p['fwd_b_sharpe_atr']})

            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row] # This now correctly includes both types of Sharpe
            # --- END OF MODIFIED BLOCK ---
            
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.4f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return (results_container, debug_data_container)

def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
    """Runs a full backtest of a strategy over a specified date range."""
    print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
    
    # (No changes to the initial setup part...)
    start_date, end_date = pd.to_datetime(strategy_params['start_date']), pd.to_datetime(strategy_params['end_date'])
    calc_period, fwd_period = strategy_params['calc_period'], strategy_params['fwd_period']
    metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
    benchmark_ticker = strategy_params['benchmark_ticker']
    master_calendar_ticker = strategy_params.get('master_calendar_ticker', 'VOO')
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    start_idx = master_trading_days.searchsorted(start_date)
    end_idx = master_trading_days.searchsorted(end_date, side='right')
    print("--- Generating all features upfront... ---")
    features_df = generate_features(df_ohlcv)
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    # Loop through all periods in the backtest range
    step_indices = range(start_idx, end_idx, fwd_period)
    all_fwd_gains, period_by_period_debug = [], {}

    print(f"Simulating {len(step_indices)} periods from {master_trading_days[step_indices[0]].date()} to {master_trading_days[step_indices[-1]].date()}...")
    for current_idx in tqdm(step_indices, desc="Backtest Progress"):
        step_date = master_trading_days[current_idx]
        eligible_tickers = get_eligible_universe(features_df, step_date, quality_thresholds)
        if not eligible_tickers: continue
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        
        # +++ UPDATE THE CALL HERE +++
        results, debug_output = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step, master_trading_days,
            step_date, calc_period, fwd_period,
            metric, rank_start, rank_end, benchmark_ticker,
            features_df=features_df,  # Pass the features_df
            debug=True
        )
        
        # (The rest of the function remains unchanged...)
        if results['error'] is None:
            fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
            all_fwd_gains.append(fwd_series.pct_change().dropna())
            period_by_period_debug[step_date.date().isoformat()] = debug_output
            
    if not all_fwd_gains:
        print("Error: No valid periods were simulated."); return None

    strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
    benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
    cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
    fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
    fig.show()
    final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
    print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
    return final_backtest_results

# --- E. VERIFICATION TOOLS (User Requested) ---

def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
                                                  start_date, calc_period, fwd_period,
                                                  master_calendar_ticker='VOO', export_csv=False):
    """Verifies portfolio and benchmark performance and optionally exports the data."""
    display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
    # 1. Setup trading day calendar and determine exact period dates
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    if start_idx >= len(master_trading_days):
        print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
    actual_start_date = master_trading_days[start_idx]
    
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
    actual_calc_end_date = master_trading_days[calc_end_idx]
    actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
    display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
                    f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
                    f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

    # 2. Recreate the portfolio and benchmark series from scratch
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
    portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    
    # 3. Optionally export the underlying daily data to a CSV for external checking
    if export_csv:
        export_df = pd.DataFrame({
            'Portfolio_Normalized_Price': portfolio_value_series,
            'Portfolio_Daily_Return': portfolio_value_series.pct_change()
        })
        if benchmark_price_series is not None:
            norm_bm = benchmark_price_series.loc[actual_start_date:actual_fwd_end_date]
            norm_bm = norm_bm / norm_bm.bfill().iloc[0]
            export_df['Benchmark_Normalized_Price'] = norm_bm
            export_df['Benchmark_Daily_Return'] = norm_bm.pct_change()

        output_dir = 'export_csv'
        os.makedirs(output_dir, exist_ok=True)
        tickers_str = '_'.join(tickers_to_verify)
        filename = f"verify_group_{actual_start_date.date()}_{tickers_str}.csv"
        filepath = os.path.join(output_dir, filename)
        export_df.to_csv(filepath)
        print(f"\n✅ Data exported to: {filepath}")

    # 4. Define a helper to print detailed calculation steps
    def print_verification_steps(title, price_series):
        display(Markdown(f"#### Verification for: `{title}`"))
        if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
        start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
        gain = (end_price / start_price) - 1
        print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
        returns = price_series.pct_change()
        sharpe = calculate_sharpe(returns)
        print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
        return {'gain': gain, 'sharpe': sharpe}

    # 5. Run verification for each period
    display(Markdown("### A. Calculation Period"))
    perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
    if benchmark_price_series is not None:
        perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
    
    display(Markdown("### B. Forward Period"))
    perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
    if benchmark_price_series is not None:
        perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])

def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
                                  master_calendar_ticker='VOO', export_csv=False):
    """Verifies ranking metrics for a single ticker and optionally exports the data."""
    display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
    # 1. Setup trading day calendar and determine exact period dates
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    if start_idx >= len(master_trading_days):
        print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
    actual_start_date = master_trading_days[start_idx]
    
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    actual_calc_end_date = master_trading_days[calc_end_idx]

    # 2. Extract and prepare the raw data for the specific ticker and period
    df_ticker = df_ohlcv.loc[ticker].sort_index()
    calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
    if calc_df.empty or len(calc_df) < 2: 
        print("No data or not enough data in calc period."); return

    display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
    display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
                    f"**Period End:** `{actual_calc_end_date.date()}`\n"
                    f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
    display(Markdown("#### Detailed Metric Calculation Data"))
    
    # 3. Calculate all intermediate metrics as new columns for full transparency
    vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
    vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
    
    # Corrected True Range (TR) calculation for a single ticker (Series)
    tr_df = pd.DataFrame({
        'h_l': vdf['Adj High'] - vdf['Adj Low'],
        'h_cp': abs(vdf['Adj High'] - vdf['Adj Close'].shift(1)),
        'l_cp': abs(vdf['Adj Low'] - vdf['Adj Close'].shift(1))
    })
    vdf['TR'] = tr_df.max(axis=1)
    
    vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
    vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
    print("--- Start of Calculation Period ---")
    display(vdf.head())
    print("\n--- End of Calculation Period ---")
    display(vdf.tail())

    # 4. Optionally export this detailed breakdown to CSV
    if export_csv:
        output_dir = 'export_csv'
        os.makedirs(output_dir, exist_ok=True)
        filename = f"verify_ticker_{actual_start_date.date()}_{ticker}.csv"
        filepath = os.path.join(output_dir, filename)
        vdf.to_csv(filepath)
        print(f"\n✅ Data exported to: {filepath}")
    
    # 5. Print final metric calculations with formulas
    display(Markdown("#### `MetricValue` Verification Summary:"))
    
    calc_start_price = vdf['Adj Close'].bfill().iloc[0]
    calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
    price_metric = (calc_end_price / calc_start_price)
    print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
    daily_returns = vdf['Daily_Return'].dropna()
    sharpe_ratio = calculate_sharpe(daily_returns)
    print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

    atrp_mean = vdf['ATRP'].mean()
    mean_daily_return = vdf['Daily_Return'].mean()
    sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
    print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")


# def verify_sharpe_atr_calculation_checked(df_ohlcv, features_df, tickers_to_verify, benchmark_ticker,
#                                     start_date, calc_period, fwd_period,
#                                     master_calendar_ticker='VOO', debug=False):
#     """
#     Verifies the Sharpe (ATR) calculations for a portfolio and benchmark.

#     This function transparently recalculates the key components for Sharpe (ATR)
#     and can optionally export the underlying source data for manual inspection.
#     """
#     display(Markdown(f"## Verification Report for Sharpe (ATR) Calculation"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))

#     # --- 1. Determine Exact Period Dates (No changes here) ---
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     actual_start_date = master_trading_days[start_idx]
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
#     actual_calc_end_date = master_trading_days[calc_end_idx]
#     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
#     display(Markdown(f"**Full Period:** `{actual_start_date.date()}` to `{actual_fwd_end_date.date()}`\n"
#                     f"**Calc End Date:** `{actual_calc_end_date.date()}`"))

#     # Original debug export block (can be kept or removed)
#     if debug:
#         # ... (original export code remains here) ...
#         pass # Assuming original block is here

#     # --- 2. Recreate Portfolio & Benchmark Series from Scratch ---
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     portfolio_prices_raw = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
#     portfolio_prices_norm = portfolio_prices_raw.div(portfolio_prices_raw.bfill().iloc[0])
#     portfolio_value_series = portfolio_prices_norm.mean(axis=1)
#     portfolio_return_series = portfolio_value_series.pct_change()
#     p_idx = pd.MultiIndex.from_product([tickers_to_verify, portfolio_return_series.index])
#     p_atrp_df = features_df.loc[features_df.index.intersection(p_idx)]['ATRP'].unstack(level=0)
#     portfolio_atrp_series = p_atrp_df.mean(axis=1)

# ###############################    
#     # benchmark_return_series = df_close_full[benchmark_ticker].pct_change().loc[actual_start_date:actual_fwd_end_date]

#     # 1. First, slice the raw prices for the desired date range.
#     benchmark_prices_raw = df_close_full[benchmark_ticker].loc[actual_start_date:actual_fwd_end_date]
#     # 2. Then, calculate the percentage change on the sliced data.
#     benchmark_return_series = benchmark_prices_raw.pct_change()

# ###############################    
#     benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[actual_start_date:actual_fwd_end_date]


#     # +++ NEW: DETAILED RETURN CALCULATION TRACE (PORTFOLIO) +++
#     if debug:
#         display(Markdown("---"))
#         display(Markdown("### 🐛 Detailed Portfolio Return Calculation Trace"))

#         # Step 1: Show the raw prices being used
#         print("\n[STEP 1] Raw Adjusted Close prices used for calculation.")
#         print("Compare these values with your 'Adj Close' columns.")
#         display(portfolio_prices_raw)

#         # Step 2: Show the normalization base and the result
#         normalization_base = portfolio_prices_raw.bfill().iloc[0]
#         print("\n[STEP 2] Normalization base (first row of prices).")
#         print("Each column in Step 1 is divided by this corresponding value.")
#         display(pd.DataFrame(normalization_base).T)

#         print("\n[STEP 2a] Normalized prices (Result of division).")
#         print("Compare these values with your 'N Close' columns.")
#         display(portfolio_prices_norm)

#         # Step 3: Show the averaged portfolio value series
#         print("\n[STEP 3] Averaged normalized portfolio value series.")
#         print("This is the row-by-row average of the table in Step 2a.")
#         print("Compare this series with your 'N_portf' column.")
#         display(pd.DataFrame(portfolio_value_series, columns=['N_portf']))

#         # Step 4: Show the final portfolio return series
#         print("\n[STEP 4] Final portfolio daily return series (pct_change).")
#         print("This is the percentage change of the series in Step 3.")
#         print("Compare this series with your 'N_portf_rtn' column.")
#         display(pd.DataFrame(portfolio_return_series, columns=['N_portf_rtn']))
#         display(Markdown("---"))
#     # +++ END OF NEW DEBUG BLOCK +++


#     # --- 3. Define a Helper to Print Detailed Calculation Steps ---
#     # MODIFIED to include more debug details inside
#     def _calculate_and_print_metrics(period_name, returns, atrps):
#         display(Markdown(f"#### {period_name}"))
#         if returns.dropna().empty or atrps.dropna().empty:
#             print("  - Not enough data to calculate.")
#             return np.nan

#         # Standard calculations
#         mean_return = returns.mean()
#         mean_atrp = atrps.mean()
#         sharpe_atr = mean_return / mean_atrp if mean_atrp > 0 else np.nan

#         # +++ ADDED: Detailed Mean Calculation Breakdown +++
#         if debug:
#             valid_returns = returns.dropna()
#             num_returns = valid_returns.count()
#             sum_returns = valid_returns.sum()
#             manual_mean = sum_returns / num_returns if num_returns > 0 else 0
#             print(f"  - (Debug) Number of valid daily returns: {num_returns}")
#             print(f"  - (Debug) Sum of all daily returns:      {sum_returns:,.8f}")
#             print(f"  - (Debug) Manually calculated mean:      {manual_mean:,.8f} (Sum / Count)")
#         # +++ END OF ADDED DETAIL +++

#         print(f"  - Mean Daily Return: {mean_return:,.6f}")
#         print(f"  - Mean Daily ATRP:  {mean_atrp:,.6f}")
#         print(f"  - Sharpe (ATR) = (Mean Return / Mean ATRP) = {sharpe_atr:,.4f}")
#         return sharpe_atr

#     # --- 4. Run Verification for Portfolio (No changes here) ---
#     display(Markdown("### A. Group Portfolio Verification"))
#     _calculate_and_print_metrics("Full Period", portfolio_return_series, portfolio_atrp_series)
#     _calculate_and_print_metrics("Calculation Period", portfolio_return_series.loc[:actual_calc_end_date], portfolio_atrp_series.loc[:actual_calc_end_date])
#     _calculate_and_print_metrics("Forward Period", portfolio_return_series.loc[actual_calc_end_date:].iloc[1:], portfolio_atrp_series.loc[actual_calc_end_date:].iloc[1:])

#     # --- 5. Run Verification for Benchmark (No changes here) ---
#     display(Markdown(f"### B. Benchmark ({benchmark_ticker}) Verification"))
#     _calculate_and_print_metrics("Full Period", benchmark_return_series, benchmark_atrp_series)
#     _calculate_and_print_metrics("Calculation Period", benchmark_return_series.loc[:actual_calc_end_date], benchmark_atrp_series.loc[:actual_calc_end_date])
#     _calculate_and_print_metrics("Forward Period", benchmark_return_series.loc[actual_calc_end_date:].iloc[1:], benchmark_atrp_series.loc[actual_calc_end_date:].iloc[1:])

def verify_sharpe_atr_calculation_checked(df_ohlcv, features_df, tickers_to_verify, benchmark_ticker,
                                    start_date, calc_period, fwd_period,
                                    master_calendar_ticker='VOO', debug=False):
    """
    Verifies the Sharpe (ATR) calculations for a portfolio and benchmark.

    [MODIFIED to use the centralized performance function and trace the new value-weighted ATRP logic]
    """
    # Assuming display and Markdown are imported from IPython.display
    from IPython.display import display, Markdown
    
    display(Markdown(f"## Verification Report for Sharpe (ATR) Calculation"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))

    # --- 1. Determine Exact Period Dates (No changes) ---
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    actual_start_date = master_trading_days[start_idx]
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    actual_calc_end_date = master_trading_days[calc_end_idx]
    actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
    display(Markdown(f"**Full Period:** `{actual_start_date.date()}` to `{actual_fwd_end_date.date()}`\n"
                    f"**Calc End Date:** `{actual_calc_end_date.date()}`"))

    # --- 2. Recreate Portfolio & Benchmark Series using the Centralized Function ---
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)

    # Use the central function for both portfolio and benchmark
    portfolio_value_series, portfolio_return_series, portfolio_atrp_series = \
        calculate_buy_and_hold_performance(df_close_full, features_df, tickers_to_verify,
                                           actual_start_date, actual_fwd_end_date)
    
    _, benchmark_return_series, benchmark_atrp_series = \
        calculate_buy_and_hold_performance(df_close_full, features_df, [benchmark_ticker],
                                           actual_start_date, actual_fwd_end_date)

    # --- 3. DETAILED DEBUG TRACE ---
    # We re-calculate intermediate steps here ONLY for the purpose of detailed printing.
    # The final series used in the analysis below come from the trusted central function.
    if debug:
        display(Markdown("---"))
        # --- RETURN TRACE (largely unchanged) ---
        display(Markdown("### 🐛 Detailed Portfolio Return Calculation Trace"))
        portfolio_prices_raw = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
        normalization_base = portfolio_prices_raw.bfill().iloc[0]
        portfolio_prices_norm = portfolio_prices_raw.div(normalization_base)
        
        print("\n[STEP 1] Raw Adjusted Close prices.")
        display(portfolio_prices_raw)
        print("\n[STEP 2] Normalization base (first valid row of prices).")
        display(pd.DataFrame(normalization_base).T)
        print("\n[STEP 2a] Normalized prices (each stock's value from an initial $1 investment).")
        display(portfolio_prices_norm)
        print("\n[STEP 3] Averaged normalized portfolio value series.")
        display(pd.DataFrame(portfolio_value_series, columns=['N_portf_value']))
        print("\n[STEP 4] Final portfolio daily return series (pct_change of Step 3).")
        display(pd.DataFrame(portfolio_return_series, columns=['N_portf_rtn']))
        
        # +++ NEW: DETAILED ATRP CALCULATION TRACE +++
        display(Markdown("### 🐛 Detailed Portfolio ATRP Calculation Trace (Value-Weighted)"))
        p_idx = pd.MultiIndex.from_product([tickers_to_verify, portfolio_return_series.index])
        p_atrp_df = features_df.loc[features_df.index.intersection(p_idx)]['ATRP'].unstack(level=0)
        
        daily_total_value = portfolio_prices_norm.sum(axis=1)
        daily_weights = portfolio_prices_norm.div(daily_total_value, axis=0)

        print("\n[STEP 5] Individual component ATRP values for each stock.")
        print("This is the raw risk input for each component.")
        display(p_atrp_df)

        print("\n[STEP 6] Drifting daily portfolio weights (based on market value).")
        print("Note how weights start near equal and drift over time. This is the core of the buy-and-hold logic.")
        display(daily_weights)
        
        print("\n[STEP 7] Final value-weighted portfolio ATRP series.")
        print("Result of (Weights * Component_ATRPs) summed each day.")
        display(pd.DataFrame(portfolio_atrp_series, columns=['value_weighted_atrp']))
        display(Markdown("---"))
    # +++ END OF DEBUG BLOCK +++

    # --- 4. Define a Helper to Print Detailed Calculation Steps (Unchanged) ---
    def _calculate_and_print_metrics(period_name, returns, atrps):
        display(Markdown(f"#### {period_name}"))
        if returns.dropna().empty or atrps.dropna().empty:
            print("  - Not enough data to calculate.")
            return np.nan
        mean_return = returns.mean()
        mean_atrp = atrps.mean()
        sharpe_atr = mean_return / mean_atrp if mean_atrp > 0 else np.nan
        if debug:
            # ... (debug printouts for mean calculation) ...
            pass
        print(f"  - Mean Daily Return: {mean_return:,.6f}")
        print(f"  - Mean Daily ATRP:  {mean_atrp:,.6f}")
        print(f"  - Sharpe (ATR) = (Mean Return / Mean ATRP) = {sharpe_atr:,.4f}")
        return sharpe_atr

    # --- 5. Run Verification for Portfolio (Unchanged) ---
    display(Markdown("### A. Group Portfolio Verification"))
    _calculate_and_print_metrics("Full Period", portfolio_return_series, portfolio_atrp_series)
    _calculate_and_print_metrics("Calculation Period", portfolio_return_series.loc[:actual_calc_end_date], portfolio_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", portfolio_return_series.loc[actual_calc_end_date:].iloc[1:], portfolio_atrp_series.loc[actual_calc_end_date:].iloc[1:])

    # --- 6. Run Verification for Benchmark (Unchanged) ---
    display(Markdown(f"### B. Benchmark ({benchmark_ticker}) Verification"))
    _calculate_and_print_metrics("Full Period", benchmark_return_series, benchmark_atrp_series)
    _calculate_and_print_metrics("Calculation Period", benchmark_return_series.loc[:actual_calc_end_date], benchmark_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", benchmark_return_series.loc[actual_calc_end_date:].iloc[1:], benchmark_atrp_series.loc[actual_calc_end_date:].iloc[1:])



# --- F. AUTOMATION SCRIPT - STRATEGY SEARCH ---

def run_strategy_search(df_ohlcv, config):
    """
    Runs the main backtesting loop with checkpointing to be resumable.
    """
    start_time = time.time()
    
    # --- 1. SETUP & LOAD PROGRESS ---
    print("--- Phase 1: Pre-processing and Loading Progress ---")

    # +++ ADD THIS BLOCK +++
    # Pre-calculate all features for the entire dataset ONCE.
    print("--- Generating all features upfront... ---")
    features_df = generate_features(df_ohlcv)

    # --- DELETE THIS LINE ---
    # quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    
    print("Unstacking data for performance...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    master_calendar_ticker = config['master_calendar_ticker']
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

    results_path = config['results_output_path']
    completed_params = set()
    
    if os.path.exists(results_path):
        print(f"Found existing results file. Loading progress from: {results_path}")
        df_progress = pd.read_csv(results_path)
        for _, row in df_progress.iterrows():
            param_key = (
                row['calc_period'], row['fwd_period'], row['metric'],
                (row['rank_start'], row['rank_end'])
            )
            completed_params.add(param_key)
        print(f"Found {len(completed_params)} completed parameter sets to skip.")
    else:
        print("No existing results file found. Starting a new run.")

    print("✅ Pre-processing complete.\n")

    # --- 2. SETUP THE MAIN LOOP (No changes here) ---
    print("--- Phase 2: Setting up Simulation Loops ---")
    param_combinations = list(product(
        config['calc_periods'], config['fwd_periods'],
        config['metrics'], config['rank_slices']
    ))
    search_start_date = pd.to_datetime(config['search_start_date'])
    search_end_date = pd.to_datetime(config['search_end_date'])
    start_idx = master_trading_days.searchsorted(search_start_date, side='left')
    end_idx = master_trading_days.searchsorted(search_end_date, side='right')
    step_dates_map = {}
    print("Pre-calculating rebalancing schedules for each holding period...")
    for fwd_period in sorted(config['fwd_periods']):
        step_indices = range(start_idx, end_idx, fwd_period)
        step_dates_map[fwd_period] = master_trading_days[step_indices]
        print(f"  - Holding Period {fwd_period} days: {len(step_dates_map[fwd_period])} rebalances")
    print(f"Found {len(param_combinations)} total parameter sets to simulate.")
    print("✅ Setup complete. Starting main loop...\n")

    # --- 3. RUN THE MAIN LOOP ---
    print("--- Phase 3: Running Simulations ---")
    pbar = tqdm(param_combinations, desc="Parameter Sets")
    
    for params in pbar:
        calc_period, fwd_period, metric, rank_slice = params
        rank_start, rank_end = rank_slice
        
        param_key = (calc_period, fwd_period, metric, rank_slice)
        if param_key in completed_params:
            pbar.set_description(f"Skipping {param_key}")
            continue

        pbar.set_description(f"Running {param_key}")
        
        current_params_results = []
        
        current_step_dates = step_dates_map[fwd_period]
        for step_date in current_step_dates:
            # +++ UPDATE THIS CALL +++
            eligible_tickers = get_eligible_universe(
                features_df, filter_date=step_date, thresholds=config['quality_thresholds']
            )
            if not eligible_tickers: continue
            
            df_close_step = df_close_full[eligible_tickers]
            df_high_step = df_high_full[eligible_tickers]
            df_low_step = df_low_full[eligible_tickers]

            # +++ UPDATE THIS CALL +++
            step_result, _ = run_walk_forward_step(
                df_close_full=df_close_step, df_high_full=df_high_step, df_low_full=df_low_step,
                master_trading_days=master_trading_days, start_date=step_date,
                calc_period=calc_period, fwd_period=fwd_period,
                metric=metric, rank_start=rank_start, rank_end=rank_end,
                benchmark_ticker=config['benchmark_ticker'],
                features_df=features_df, # Pass the features_df
                debug=False
            )
            
            if step_result['error'] is None:
                p = step_result['performance_data']
                log_entry = {
                    'step_date': step_date.date(), 'calc_period': calc_period,
                    'fwd_period': fwd_period, 'metric': metric,
                    'rank_start': rank_start, 'rank_end': rank_end,
                    'num_universe': len(eligible_tickers),
                    'num_portfolio': len(step_result['tickers_to_display']),
                    'fwd_p_gain': p['fwd_p_gain'], 'fwd_b_gain': p['fwd_b_gain'],
                    'fwd_gain_delta': p['fwd_p_gain'] - p['fwd_b_gain'] if not np.isnan(p['fwd_b_gain']) else np.nan,
                    'fwd_p_sharpe': p['fwd_p_sharpe'],
                }
                current_params_results.append(log_entry)
        
        # --- CHECKPOINTING (No changes here) ---
        if current_params_results:
            df_to_append = pd.DataFrame(current_params_results)
            df_to_append.to_csv(
                results_path,
                mode='a',
                header=not os.path.exists(results_path),
                index=False
            )
            completed_params.add(param_key)

    print("✅ Main loop finished.\n")
    
    # --- 4. RETURN FINAL DATAFRAME (No changes here) ---
    print("--- Phase 4: Loading Final Results ---")
    if os.path.exists(results_path):
        final_df = pd.read_csv(results_path)
        end_time = time.time()
        print(f"✅ Process complete. Total execution time: {time.time() - start_time:.2f} seconds.")
        return final_df
    else:
        print("Warning: No results were generated.")
        return None    



✅ Metric Registry Initialized with: ['Price', 'Sharpe', 'Sharpe (ATR)']


In [3]:
results_container, debug_container = plot_walk_forward_analyzer(
    df_ohlcv=df_ohlcv,
    default_start_date='2018-10-03',
    default_calc_period=252,
    default_fwd_period=63,
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=5,
    default_benchmark_ticker='VOO',
    quality_thresholds={ 'min_median_dollar_volume': 10_000_000, 
                         'max_stale_pct': 0.05, 
                         'max_same_vol_count': 1 },
    debug=True  # <-- Activate the new mode!
)

Initializing Walk-Forward Analyzer (using Trading Day Logic)...
Master trading day calendar created from 'VOO' (3795 days).
--- Generating all features upfront... ---
--- Starting Feature Generation ---
Calculating technical indicators (ATR Period: 14)...
Calculating data quality metrics (Window: 252 days)...
Combining all feature sets...
✅ Feature generation complete.
Pre-processing data (unstacking)...


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'placeholder_0',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'a9b49555-5eb4-47e4-b535-bc43ebafff5f',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_1',
              'showlegend': False,
              'type': 'scatter',
              'uid': '1e87a0c5-e381-4cb0-8325-bff7d0363f42',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_2',
              'showlegend': False,
              'type': 'scatter',
              'uid': '2d66d579-ad4b-4d83-bb53-2678d6b24140',
              'visible': False,
              'x': [None],
              'y': [None]},
             {'mode': 'lines',
              'name': 'placeholder_3',
              'showlegend': False,
              'type': 

Dynamic Filter (2018-10-03): Kept 618 of 722 tickers.


In [4]:
# Assume 'df_ohlcv' is your loaded dataset
# You might need to regenerate features_df if it's not in your notebook's memory
print("--- Regenerating features for verification ---")
features_df = generate_features(df_ohlcv)

--- Regenerating features for verification ---
--- Starting Feature Generation ---
Calculating technical indicators (ATR Period: 14)...
Calculating data quality metrics (Window: 252 days)...
Combining all feature sets...
✅ Feature generation complete.


In [5]:
tickers_to_verify = ['STIP', 'RCL']
start_date = '2025-08-13'
end_date = '2025-09-04'
benchmark_ticker = 'VOO'

In [6]:
# Now call the verification function with the exact parameters from the UI
verify_sharpe_atr_calculation_checked(
    df_ohlcv=df_ohlcv,
    features_df=features_df,
    tickers_to_verify=tickers_to_verify, # <-- From the UI output
    benchmark_ticker=benchmark_ticker,
    start_date=start_date,
    calc_period=10,
    fwd_period=5,
    debug=True,
)

## Verification Report for Sharpe (ATR) Calculation

**Portfolio Tickers:** `['STIP', 'RCL']`
**Benchmark Ticker:** `VOO`

**Full Period:** `2025-08-13` to `2025-09-04`
**Calc End Date:** `2025-08-27`

---

### 🐛 Detailed Portfolio Return Calculation Trace

...

  - Mean Daily Return: 0.000438
  - Mean Daily ATRP:  0.008277
  - Sharpe (ATR) = (Mean Return / Mean ATRP) = 0.0529


#### Calculation Period

  - Mean Daily Return: 0.000283
  - Mean Daily ATRP:  0.008317
  - Sharpe (ATR) = (Mean Return / Mean ATRP) = 0.0340


#### Forward Period

  - Mean Daily Return: 0.000748
  - Mean Daily ATRP:  0.008191
  - Sharpe (ATR) = (Mean Return / Mean ATRP) = 0.0913


In [ ]:
# # ==============================================================================
# # GOLDEN COPY - COMPLETE PROJECT CODE (All Fixes Included)
# # Version: Verified Portfolio and Benchmark Sharpe (ATR)  
# # Date: 2025-10-15
# # ==============================================================================

# import pandas as pd
# import numpy as np
# import plotly.graph_objects as go
# import ipywidgets as widgets
# import time
# import pprint
# import os # Make sure os is imported for the export function later
# import re

# from datetime import datetime, date
# from IPython.display import display, Markdown
# from tqdm.auto import tqdm
# from pathlib import Path
# from itertools import product


# pd.set_option('display.max_rows', 30)
# pd.set_option('display.max_columns', 50)
# pd.set_option('display.width', 3000)


# # --- REFACTORING PHASE 1 CODE: Feature Generation Engine ---

# def generate_features(df_ohlcv: pd.DataFrame, 
#                       atr_period: int = 14, 
#                       quality_window: int = 252, 
#                       quality_min_periods: int = 126) -> pd.DataFrame:
#     """
#     Generates a comprehensive DataFrame of derived features from raw OHLCV data.

#     This function performs all heavy, window-based calculations upfront to be used
#     by downstream analysis functions. It calculates:
#     1. Technical Indicators: True Range (TR), ATR, and ATRP.
#     2. Data Quality Metrics: Rolling stale percentage, median dollar volume, etc.

#     Args:
#         df_ohlcv: The primary DataFrame with a (Ticker, Date) MultiIndex and
#                   columns for 'Adj High', 'Adj Low', 'Adj Close', 'Volume'.
#         atr_period: The lookback period for the ATR's Exponential Moving Average.
#         quality_window: The rolling window size for data quality metrics.
#         quality_min_periods: The minimum number of observations required to have
#                              a valid quality metric.

#     Returns:
#         A new DataFrame with the same (Ticker, Date) MultiIndex containing all
#         calculated feature columns.
#     """
#     print("--- Starting Feature Generation ---")
    
#     # Ensure the DataFrame is sorted for correct window and shift operations
#     # FIX: Replaced is_lexsorted() with the current pandas attribute
#     if not df_ohlcv.index.is_monotonic_increasing:
#         print("Sorting index for calculation accuracy...")
#         df_ohlcv = df_ohlcv.sort_index()

#     # --- 1. Technical Indicator Calculation (TR, ATR, ATRP) ---
#     print(f"Calculating technical indicators (ATR Period: {atr_period})...")
    
#     # Group by ticker to handle each security independently
#     grouped = df_ohlcv.groupby(level='Ticker')
    
#     # Get the previous day's close required for True Range
#     prev_close = grouped['Adj Close'].shift(1)
    
#     # Calculate the three components of True Range
#     high_low = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
#     high_prev_close = abs(df_ohlcv['Adj High'] - prev_close)
#     low_prev_close = abs(df_ohlcv['Adj Low'] - prev_close)
    
#     # Combine the components to get the final TR
#     tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1, skipna=False)
    
#     # # Calculate the ATR using an Exponential Moving Average
#     # --- FIX IS HERE ---
#     # Use .transform() to apply the EWM function. 
#     # This guarantees the resulting Series has the exact same index as 'tr',
#     # preventing the index alignment error during the subsequent division.
#     atr = tr.groupby(level='Ticker').transform(
#         lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean()
#     )

#     # --- CHANGE 1: Removed .fillna(0) ---
#     # ATRP will now be NaN on the first day, consistent with TR and ATR.
#     atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)

#     indicator_df = pd.DataFrame({
#         'TR': tr,
#         'ATR': atr,
#         'ATRP': atrp
#     })
    
#     # --- 2. Data Quality Metric Calculation ---
#     print(f"Calculating data quality metrics (Window: {quality_window} days)...")
    
#     # Create intermediate flags needed for quality calculations
#     is_stale = np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0)
#     dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']
#     has_same_volume = (grouped['Volume'].diff() == 0).astype(int)
    
#     # Combine flags into a temporary DataFrame for rolling calculations
#     quality_temp_df = pd.DataFrame({
#         'IsStale': is_stale,
#         'DollarVolume': dollar_volume,
#         'HasSameVolume': has_same_volume
#     }, index=df_ohlcv.index) # Explicitly set index to be safe
    
#     # Perform the rolling calculations on the grouped data
#     # --- FIX IS HERE ---
#     # We switch to the older, more compatible dictionary-based aggregation method.
#     # This syntax is understood by nearly all versions of pandas.
#     rolling_result = quality_temp_df.groupby(level='Ticker').rolling(
#         window=quality_window,
#         min_periods=quality_min_periods
#     ).agg({
#         'IsStale': 'mean',
#         'DollarVolume': 'median',
#         'HasSameVolume': 'sum'
#     })
    
#     # The dictionary syntax produces columns with the original names ('IsStale', etc.).
#     # We now explicitly rename them to our desired final names.
#     rolling_result = rolling_result.rename(columns={
#         'IsStale': 'RollingStalePct',
#         'DollarVolume': 'RollMedDollarVol', # <-- RENAMED HERE
#         'HasSameVolume': 'RollingSameVolCount'
#     })

#     # The index after a grouped rolling operation is hierarchical.
#     # We remove the outermost 'Ticker' level to restore the original index structure.
#     rolling_quality = rolling_result.reset_index(level=0, drop=True)

#     # --- 3. Combine All Features ---
#     print("Combining all feature sets...")
#     features_df = pd.concat([indicator_df, rolling_quality], axis=1)
    
#     print("✅ Feature generation complete.")
#     return features_df

# def test_features_df(features_df: pd.DataFrame, 
#                      df_ohlcv: pd.DataFrame, 
#                      test_ticker: str = 'AAPL',
#                      spot_check_date: str = '2020-03-20'):
#     """
#     Runs a suite of tests to verify the correctness of the generated features_df.

#     Args:
#         features_df: The generated DataFrame from the generate_features function.
#         df_ohlcv: The original source OHLCV DataFrame.
#         test_ticker: A common, liquid ticker to use for specific value checks.
#     """
#     print(f"\n--- Running Verification Suite for features_df (Test Ticker: {test_ticker}) ---")
    
#     # --- Test 1: Structural Integrity ---
#     print("\n[Test 1: Structural Integrity]")
#     assert features_df.index.equals(df_ohlcv.index), "FAIL: Index does not match original df_ohlcv."
#     print("  ✅ PASS: Index matches original df_ohlcv.")
    
#     expected_cols = ['TR', 'ATR', 'ATRP', 'RollingStalePct', 'RollMedDollarVol', 'RollingSameVolCount']
#     assert all(col in features_df.columns for col in expected_cols), "FAIL: Missing one or more expected columns."
#     print("  ✅ PASS: All expected feature columns are present.")
#     print(f"  - DataFrame Info:")
#     features_df.info(verbose=False, memory_usage='deep')


#     # --- Test 2: ATR Calculation Logic ---
#     print("\n[Test 2: ATR Logic Verification]")
#     ticker_features = features_df.loc[test_ticker]
    
#     # Test 2a: First TR value should be NaN (since prev_close is NaN)
#     first_tr = ticker_features['TR'].iloc[0]
#     assert pd.isna(first_tr), f"FAIL: First TR value for {test_ticker} should be NaN, but got {first_tr}."
#     print(f"  ✅ PASS: First TR value for {test_ticker} is NaN as expected.")

#     # Test 2b: The first valid ATR should equal the first valid TR (EWM cold start behavior)
#     first_valid_tr_val = ticker_features['TR'].dropna().iloc[0]
#     first_valid_atr_val = ticker_features['ATR'].dropna().iloc[0]
#     assert np.isclose(first_valid_tr_val, first_valid_atr_val), \
#         f"FAIL: First valid ATR ({first_valid_atr_val}) should equal first valid TR ({first_valid_tr_val})."
#     print("  ✅ PASS: First valid ATR correctly seeded with first valid TR.")


#     # --- Test 3: Rolling Quality Metrics Logic ---
#     print("\n[Test 3: Rolling Quality Metrics Logic Verification]")
#     quality_min_periods = 126 # Should match the parameter used in generation
    
#     # Test 3a: Check for leading NaNs
#     first_valid_quality_idx = ticker_features['RollingStalePct'].first_valid_index()
#     if first_valid_quality_idx is None:
#         print(f"  - INFO: No valid quality metrics found for {test_ticker} (likely too little data). Skipping test.")
#     else:
#         position_of_first_valid = ticker_features.index.get_loc(first_valid_quality_idx)
#         assert position_of_first_valid == quality_min_periods - 1, \
#             f"FAIL: First valid quality metric should appear at index {quality_min_periods - 1}, but appeared at {position_of_first_valid}."
#         print(f"  ✅ PASS: Leading NaNs are present for the first {quality_min_periods - 1} periods as expected.")


#     # --- Test 4: Spot Check Against Manual Calculation ---
#     print("\n[Test 4: Spot Check vs. Manual Calculation]")
#     # # Choose a specific date for a manual calculation
#     # spot_check_date = '2020-03-20' # A volatile day for a good test
    
#     # Manual TR Calculation
#     today_data = df_ohlcv.loc[(test_ticker, spot_check_date)]
#     yesterday_data = df_ohlcv.loc[(test_ticker, pd.to_datetime(spot_check_date) - pd.Timedelta(days=1))] # simple lookback for test
    
#     manual_h_l = today_data['Adj High'] - today_data['Adj Low']
#     manual_h_pc = abs(today_data['Adj High'] - yesterday_data['Adj Close'])
#     manual_l_pc = abs(today_data['Adj Low'] - yesterday_data['Adj Close'])
#     manual_tr = max(manual_h_l, manual_h_pc, manual_l_pc)
    
#     code_tr = ticker_features.loc[spot_check_date]['TR']
    
#     assert np.isclose(manual_tr, code_tr), f"FAIL: Manual TR ({manual_tr:.4f}) does not match code TR ({code_tr:.4f}) on {spot_check_date}."
#     print(f"  ✅ PASS: Manually calculated TR on {spot_check_date} matches code's TR.")
    
#     print("\n--- ✅ All Verification Tests Passed ---")

# def export_ticker_data(ticker_to_export: str, 
#                          df_ohlcv: pd.DataFrame, 
#                          features_df: pd.DataFrame, 
#                          output_dir: str = 'export_csv'):
#     """
#     Exports the raw OHLCV data and the corresponding calculated features for a 
#     single ticker to two separate CSV files.

#     This function is designed for easy manual verification of data and calculations.
#     It will create the output directory if it does not exist.

#     Args:
#         ticker_to_export: The ticker symbol to export (e.g., 'AAPL').
#         df_ohlcv: The main DataFrame containing the raw OHLCV data with a 
#                   (Ticker, Date) MultiIndex.
#         features_df: The DataFrame containing the calculated features with a 
#                      (Ticker, Date) MultiIndex.
#         output_dir: The directory where the CSV files will be saved. 
#                     Defaults to 'export_csv'.
#     """
#     print(f"--- Attempting to export data for ticker: {ticker_to_export} ---")
    
#     # --- 1. Ensure the output directory exists ---
#     try:
#         os.makedirs(output_dir, exist_ok=True)
#         print(f"Output directory '{output_dir}' is ready.")
#     except OSError as e:
#         print(f"Error: Could not create directory '{output_dir}'. Reason: {e}")
#         return

#     # --- 2. Isolate the data for the specified ticker ---
#     try:
#         # Use .loc to select all rows for the given ticker from the MultiIndex
#         ticker_ohlcv = df_ohlcv.loc[ticker_to_export]
#         ticker_features = features_df.loc[ticker_to_export]
        
#         if ticker_ohlcv.empty:
#             print(f"Warning: No OHLCV data found for ticker '{ticker_to_export}'. Cannot export.")
#             return
            
#         print(f"Found {len(ticker_ohlcv)} rows of data for '{ticker_to_export}'.")
        
#     except KeyError:
#         print(f"Error: Ticker '{ticker_to_export}' not found in one or both of the DataFrames. Please check the symbol.")
#         return
#     except Exception as e:
#         print(f"An unexpected error occurred while accessing data: {e}")
#         return

#     # --- 3. Construct file paths and export to CSV ---
#     try:
#         # Define the full path for each output file
#         ohlcv_filename = f"{ticker_to_export}_ohlcv.csv"
#         features_filename = f"{ticker_to_export}_features.csv"
        
#         ohlcv_filepath = os.path.join(output_dir, ohlcv_filename)
#         features_filepath = os.path.join(output_dir, features_filename)
        
#         # Export the DataFrames to CSV. The index (Date) will be included.
#         ticker_ohlcv.to_csv(ohlcv_filepath)
#         ticker_features.to_csv(features_filepath)
        
#         print("\n✅ Export successful!")
#         print(f"   - Raw OHLCV data saved to: {ohlcv_filepath}")
#         print(f"   - Calculated features saved to: {features_filepath}")

#     except Exception as e:
#         print(f"Error: Failed to write data to CSV files. Reason: {e}")

# def create_synthetic_ticker_data(
#     ticker_name: str = 'SYNTH', 
#     num_days: int = 50,
#     num_zero_volume_days: int = 5,
#     num_flat_price_days: int = 3
# ) -> pd.DataFrame:
#     """
#     Creates a synthetic OHLCV DataFrame with predictable patterns and randomly injected
#     stale data conditions for robust testing.

#     Args:
#         ticker_name: The name for the synthetic ticker.
#         num_days: The total number of days for the ticker's history.
#         num_zero_volume_days: The number of random days to set Volume to 0.
#         num_flat_price_days: The number of random days to set High == Low.

#     Returns:
#         A pandas DataFrame with a (Ticker, Date) MultiIndex.
#     """
#     print(f"--- Creating synthetic data for '{ticker_name}' with {num_days} days ---")
    
#     # 1. Create a base DataFrame with "normal" data
#     dates = pd.to_datetime(pd.date_range(start='2023-01-01', periods=num_days, freq='B'))
#     data = {
#         'Adj Open': 100.0, 'Adj High': 102.0, 'Adj Low': 98.0,
#         'Adj Close': 100.0, 'Volume': 1_000_000
#     }
#     df = pd.DataFrame(data, index=dates)
#     df['Adj Close'] = df['Adj Close'] + np.random.randn(num_days) * 0.5 # Add some noise

#     # 2. Define a "protected" window for specific verification tests.
#     # The `verify_synthetic_ticker_features` function depends on this exact window.
#     # We will not inject random stale days here.
#     protected_start_idx, protected_end_idx = 10, 20
    
#     # 3. Inject random "stale" days OUTSIDE the protected window
#     available_indices = df.index.drop(df.index[protected_start_idx:protected_end_idx])
    
#     # Inject zero-volume days
#     if num_zero_volume_days > 0:
#         if len(available_indices) < num_zero_volume_days:
#             raise ValueError("Not enough available days to inject zero-volume days.")
#         zero_vol_dates = np.random.choice(available_indices, num_zero_volume_days, replace=False)
#         df.loc[zero_vol_dates, 'Volume'] = 0
#         print(f"  - Injected {num_zero_volume_days} random zero-volume 'stale' days.")
#         # Update available indices to avoid overlap
#         available_indices = available_indices.drop(zero_vol_dates)

#     # Inject flat-price days (High == Low)
#     if num_flat_price_days > 0:
#         if len(available_indices) < num_flat_price_days:
#             raise ValueError("Not enough available days to inject flat-price days.")
#         flat_price_dates = np.random.choice(available_indices, num_flat_price_days, replace=False)
#         # Set High and Low to be the same as the Close price for that day
#         df.loc[flat_price_dates, 'Adj High'] = df.loc[flat_price_dates, 'Adj Close']
#         df.loc[flat_price_dates, 'Adj Low'] = df.loc[flat_price_dates, 'Adj Close']
#         print(f"  - Injected {num_flat_price_days} random flat-price 'stale' days.")

#     # 4. Inject the specific, hand-crafted patterns inside the protected window for verification
#     print("  - Injecting specific patterns for programmatic verification...")
#     # Pattern for RollingStalePct: 2 stale days in 10 (20%)
#     df.iloc[10, df.columns.get_loc('Volume')] = 0  # Stale day (zero volume)
#     df.iloc[11, df.columns.get_loc('Adj High')] = 99.0 # Stale day (High == Low)
#     df.iloc[11, df.columns.get_loc('Adj Low')] = 99.0
    
#     # Pattern for RollingMedianVolume
#     for i in range(10):
#         df.iloc[10 + i, df.columns.get_loc('Adj Close')] = 100.0 # Standardize price for easy median calc
#         df.iloc[10 + i, df.columns.get_loc('Volume')] = (i + 1) * 10000

#     # Pattern for RollingSameVolCount
#     df.iloc[15, df.columns.get_loc('Volume')] = 77777
#     df.iloc[16, df.columns.get_loc('Volume')] = 77777
#     df.iloc[17, df.columns.get_loc('Volume')] = 77777
    
#     # 5. Set the MultiIndex
#     df['Ticker'] = ticker_name
#     df = df.set_index(['Ticker', df.index])
#     df.index.names = ['Ticker', 'Date']
    
#     print("✅ Synthetic data created successfully.")
#     return df

# def verify_synthetic_ticker_features(features_df: pd.DataFrame, 
#                                        ticker_name: str = 'SYNTH',
#                                        quality_window: int = 10):
#     """
#     Verifies the quality metric calculations on the features_df generated from
#     the synthetic ticker data.

#     Args:
#         features_df: The DataFrame of calculated features.
#         ticker_name: The name of the synthetic ticker.
#         quality_window: The rolling window used, which must match the window
#                         of the synthetic data pattern.
#     """
#     print(f"\n--- Running Verification on Synthetic Ticker '{ticker_name}' ---")
    
#     # --- Expected values based on our synthetic data design ---
#     EXPECTED_STALE_PCT = 0.20  # 2 stale days out of 10
#     EXPECTED_MEDIAN_DOLLAR_VOL = 5_500_000.0 # median of (10k..100k) * price of 100
#     EXPECTED_SAME_VOL_COUNT = 2.0 # Three consecutive days gives two 'diff() == 0' events

#     try:
#         # Isolate the features for our synthetic ticker
#         ticker_features = features_df.loc[ticker_name]
        
#         # The first valid calculation will be on the last day of our 10-day window.
#         # The window starts at index 10 and has a length of 10, so it ends at index 19.
#         verification_date = ticker_features.index[19]
        
#         print(f"Verifying calculations on date: {verification_date.date()}")
        
#         # Get the calculated values from the DataFrame
#         calculated_values = ticker_features.loc[verification_date]
#         stale_pct = calculated_values['RollingStalePct']
#         # --- CHANGE 2 (continued): Accessing the renamed column ---
#         median_vol = calculated_values['RollMedDollarVol'] 
#         same_vol_count = calculated_values['RollingSameVolCount']
        
#         # --- Perform Assertions ---
#         print("\n[Test 1: RollingStalePct]")
#         assert np.isclose(stale_pct, EXPECTED_STALE_PCT), f"FAIL: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}"
#         print(f"  ✅ PASS: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}")

#         print("\n[Test 2: RollMedDollarVol]")
#         assert np.isclose(median_vol, EXPECTED_MEDIAN_DOLLAR_VOL), f"FAIL: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}"
#         print(f"  ✅ PASS: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}")

#         print("\n[Test 3: RollingSameVolCount]")
#         assert np.isclose(same_vol_count, EXPECTED_SAME_VOL_COUNT), f"FAIL: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}"
#         print(f"  ✅ PASS: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}")

#         print("\n--- ✅ All Synthetic Data Verification Tests Passed ---")

#     except KeyError:
#         print(f"FAIL: Ticker '{ticker_name}' not found in features_df.")
#     except IndexError:
#         print("FAIL: Not enough data in features_df to run verification. Check num_days.")
#     except Exception as e:
#         print(f"An unexpected error occurred during verification: {e}")

# # --- A. HELPER FUNCTIONS ---

# def calculate_gain(price_series: pd.Series):
#     """Calculates the total gain over a series of prices."""
#     # Ensure there are at least two data points to calculate a gain
#     if price_series.dropna().shape[0] < 2: return np.nan
#     # Use forward-fill for the end price and back-fill for the start price
#     # to handle potential NaNs at the beginning or end of the series.
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     """Calculates the annualized Sharpe ratio from a series of daily returns."""
#     # Ensure there are at least two returns to calculate a standard deviation
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     # Avoid division by zero if returns are constant
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# def calculate_sharpe_atr(return_series: pd.Series, atrp_series: pd.Series):
#     """Calculates a Sharpe-like ratio using mean ATRP as the denominator."""
#     # Ensure there are returns and that ATRP data is valid
#     if return_series.dropna().shape[0] < 2 or atrp_series.dropna().empty:
#         return np.nan
        
#     mean_return = return_series.mean()
#     mean_atrp = atrp_series.mean()
    
#     # Avoid division by zero
#     if mean_atrp > 0 and mean_atrp != np.inf:
#         return mean_return / mean_atrp
        
#     return np.nan

# def print_nested(d, indent=0, width=4):
#     """Pretty-print any nested dict/list/tuple combination."""
#     spacing = ' ' * indent
#     if isinstance(d, dict):
#         for k, v in d.items():
#             print(f'{spacing}{k}:')
#             print_nested(v, indent + width, width)
#     elif isinstance(d, (list, tuple)):
#         for item in d:
#             print_nested(item, indent, width)
#     else:
#         print(f'{spacing}{d}')

# # --- B. MODULAR METRIC CALCULATION ENGINE ---

# def calculate_price_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the 'Price' metric (total gain) over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'calc_close' (pd.DataFrame): The close prices for the calc period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     calc_close = metric_data['calc_close']
    
#     # Ensure there are at least two data points to calculate a gain
#     if len(calc_close) < 2:
#         return pd.Series(dtype='float64', index=calc_close.columns)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
    
#     # The division of two Series aligns by index (Ticker), which is what we want.
#     price_metric = last_prices / first_prices
    
#     return price_metric.dropna()

# def calculate_sharpe_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the annualized 'Sharpe' ratio metric over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     daily_returns = metric_data['daily_returns']
    
#     # Ensure there's enough data to calculate standard deviation
#     if len(daily_returns.dropna()) < 2:
#         return pd.Series(dtype='float64', index=daily_returns.columns)
    
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     # Standard annualized Sharpe Ratio calculation. Avoid division by zero.
#     # We replace resulting NaNs/infs with 0 to handle cases of zero volatility.
#     sharpe_ratio = (mean_returns / std_returns * np.sqrt(252))
    
#     return sharpe_ratio.replace([np.inf, -np.inf], np.nan).fillna(0)
    
# def calculate_sharpe_atr_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the 'Sharpe (ATR)' metric over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.
#                      Requires 'atrp' (pd.Series): Mean ATRP for each ticker over the period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     daily_returns = metric_data['daily_returns']
#     atrp = metric_data['atrp']
    
#     mean_returns = daily_returns.mean()
    
#     # ATRP-based Sharpe. Avoid division by zero.
#     # We replace resulting NaNs/infs with 0.
#     sharpe_atr = mean_returns / atrp
    
#     return sharpe_atr.replace([np.inf, -np.inf], np.nan).fillna(0)

# # The single source of truth for all available ranking metrics.
# # Maps the user-facing name to the calculation function.
# METRIC_REGISTRY = {
#     'Price': calculate_price_metric,
#     'Sharpe': calculate_sharpe_metric,
#     'Sharpe (ATR)': calculate_sharpe_atr_metric,
# }

# print("✅ Metric Registry Initialized with:", list(METRIC_REGISTRY.keys()))

# # --- B. THE CORE CALCULATION ENGINE ---

# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           features_df,
#                           debug=False):
#     """
#     Runs a single step of the walk-forward analysis with a strict, pre-emptive
#     check to ensure the full period is available.
#     """
#     debug_data = {} if debug else None

#     # 1. Determine exact date ranges with a NEW pre-emptive check
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)

#     # +++ THIS IS THE NEW PRE-EMPTIVE CHECK LOGIC +++
#     # Calculate the desired end index without clamping first.
#     desired_viz_end_idx = start_idx + calc_period + fwd_period
    
#     # Check if the desired end index is out of bounds.
#     if desired_viz_end_idx >= len(master_trading_days):
#         last_available_date = master_trading_days[-1].date()
#         required_days = calc_period + fwd_period
#         available_days = len(master_trading_days) - start_idx
#         error_msg = (f"Not enough data for the full requested period. "
#                      f"Required: {required_days} days, Available: {available_days} days until {last_available_date}.")
#         return ({'error': error_msg}, None)
#     # --- END OF NEW CHECK ---

#     # If the check passes, we know the full period is available.
#     # The 'min' calls are now just a redundant safety measure.
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     # (The rest of the function remains completely unchanged...)
#     # 2. Slice data for the calculation period
#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     # 3. Calculate INTERMEDIATE data required for the ranking metrics
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     valid_tickers = calc_close.columns
#     calc_period_index = pd.MultiIndex.from_product([valid_tickers, calc_close.index], names=['Ticker', 'Date'])
#     features_in_period = features_df.loc[features_df.index.intersection(calc_period_index)]
#     atrp = features_in_period.groupby(level='Ticker')['ATRP'].mean()

#     # 4. Calculate all ranking metrics by iterating through the METRIC_REGISTRY
#     metric_ingredients = { 'calc_close': calc_close, 'daily_returns': daily_returns, 'atrp': atrp, }
#     metric_values = {}
#     for name, func in METRIC_REGISTRY.items():
#         metric_values[name] = func(metric_ingredients)
#     if metric not in metric_values or metric_values[metric].empty:
#         return ({'error': f"Metric '{metric}' could not be calculated or resulted in no valid tickers."}, None)

#     # 5. Rank tickers and select the portfolio
#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     # 6. Calculate Portfolio & Benchmark Performance
#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')
#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
#         if benchmark_price_series is not None:
#             bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#             calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#             fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         if benchmark_price_series is not None:
#             calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#             fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     full_period_index = pd.MultiIndex.from_product([tickers_to_display, portfolio_return_series.index], names=['Ticker', 'Date'])
#     portfolio_atrp_features = features_df.loc[features_df.index.intersection(full_period_index)]
#     portfolio_atrp_daily_unstacked = portfolio_atrp_features['ATRP'].unstack(level='Ticker')
#     portfolio_atrp_series = portfolio_atrp_daily_unstacked.mean(axis=1)
#     if benchmark_ticker in df_close_full.columns:
#         benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[safe_start_date:safe_viz_end_date]
#     else:
#         benchmark_atrp_series = pd.Series(dtype='float64')
#     calc_portfolio_atrp = portfolio_atrp_series.loc[:actual_calc_end_ts]
#     fwd_portfolio_atrp = portfolio_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     calc_benchmark_atrp = benchmark_atrp_series.loc[:actual_calc_end_ts]
#     fwd_benchmark_atrp = benchmark_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)
#     perf_data['calc_p_sharpe_atr'] = calculate_sharpe_atr(calc_portfolio_returns, calc_portfolio_atrp)
#     perf_data['fwd_p_sharpe_atr'] = calculate_sharpe_atr(fwd_portfolio_returns, fwd_portfolio_atrp)
#     perf_data['full_p_sharpe_atr'] = calculate_sharpe_atr(portfolio_return_series, portfolio_atrp_series)
#     perf_data['calc_b_sharpe_atr'] = calculate_sharpe_atr(calc_benchmark_returns, calc_benchmark_atrp)
#     perf_data['fwd_b_sharpe_atr'] = calculate_sharpe_atr(fwd_benchmark_returns, fwd_benchmark_atrp)
#     perf_data['full_b_sharpe_atr'] = calculate_sharpe_atr(benchmark_return_series, benchmark_atrp_series)
#     if debug:
#         df_ranking_base = pd.DataFrame({'MeanDailyReturn': daily_returns.mean(),'StdDevDailyReturn': daily_returns.std(),'MeanATRP': atrp})
#         df_metrics = pd.DataFrame(metric_values)
#         df_metrics.columns = [f'Metric_{col}' for col in df_metrics.columns]
#         df_ranking = df_ranking_base.join(df_metrics, how='left')
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)
#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace
#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)

# # --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

# def get_eligible_universe(features_df, filter_date, thresholds):
#     """Filters the universe of tickers based on quality metrics for a given date."""
#     filter_date_ts = pd.to_datetime(filter_date)
#     # The index is now the comprehensive features_df
#     date_index = features_df.index.get_level_values('Date').unique().sort_values()
    
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
        
#     # Find the most recent date with quality data on or before the filter date
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
        
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")

#     metrics_on_date = features_df.xs(actual_date_to_use, level='Date')
    
#     # Apply filters using the new column names from features_df
#     mask = ((metrics_on_date['RollMedDollarVol'] >= thresholds['min_median_dollar_volume']) & # <-- RENAMED
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
            
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers

# # --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

# def plot_walk_forward_analyzer(df_ohlcv,
#                                default_start_date=None, default_calc_period=126, default_fwd_period=63,
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
#     # (No changes to the initial setup part of this function...)
#     print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")
#     print("--- Generating all features upfront... ---")
#     features_df = generate_features(df_ohlcv)
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
#     calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
#     fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
#     if default_metric not in METRIC_REGISTRY:
#         fallback_metric = list(METRIC_REGISTRY.keys())[0]
#         print(f"⚠️ Warning: Default metric '{default_metric}' not in registry. Using '{fallback_metric}'.")
#         default_metric = fallback_metric
#     metric_dropdown = widgets.Dropdown(options=list(METRIC_REGISTRY.keys()), value=default_metric, description='Metric:')
#     rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
#     rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container, debug_data_container = [None], [None]
#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date_raw = pd.to_datetime(start_date_picker.value)
#         start_date_idx = master_trading_days.searchsorted(start_date_raw)
#         if start_date_idx >= len(master_trading_days):
#             with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
#         actual_start_date = master_trading_days[start_date_idx]
#         with ticker_list_output:
#             if start_date_raw.date() != actual_start_date.date():
#                 print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")
#         calc_period = calc_period_input.value; fwd_period = fwd_period_input.value; metric = metric_dropdown.value
#         rank_start = rank_start_input.value; rank_end = rank_end_input.value; benchmark_ticker = benchmark_ticker_input.value.strip().upper()
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
#         if rank_start < 1 or calc_period < 2 or fwd_period < 1:
#             with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return
#         required_days = calc_period + fwd_period
#         if start_date_idx + required_days > len(master_trading_days):
#             available_days = len(master_trading_days) - start_date_idx; last_available_date = master_trading_days[-1].date()
#             with ticker_list_output:
#                 print(f"Error: Not enough data for the requested period.\n  Start Date: {actual_start_date.date()}\n  Required Days: {calc_period} (calc) + {fwd_period} (fwd) = {required_days}\n  Available Days from Start: {available_days} (until {last_available_date})\n  Please shorten the 'Calc Period' / 'Fwd Period' or choose an earlier 'Start Date'.")
#             return
#         eligible_tickers = get_eligible_universe(features_df, actual_start_date, quality_thresholds)
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
#         results, debug_output = run_walk_forward_step(df_close_full, df_high_full, df_low_full, master_trading_days, actual_start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker, features_df=features_df, debug=debug)
#         if results.get('error'):
#             with ticker_list_output: print(f"Error: {results['error']}"); return
#         period_dates = {'calc_period_start': results['safe_start_date'], 'calc_period_end': results['actual_calc_end_ts'], 'forward_period_start': results['actual_calc_end_ts'], 'forward_period_end': results['safe_viz_end_date']}
#         run_parameters = {'calc_period': calc_period, 'fwd_period': fwd_period, 'rank_metric': metric, 'rank_start': rank_start, 'rank_end': rank_end, 'benchmark_ticker': benchmark_ticker}
#         results.update(period_dates); results.update(run_parameters)
#         if debug_output is not None and isinstance(debug_output, dict):
#             debug_output.update(period_dates); debug_output.update(run_parameters)
#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark.values, f"Benchmark ({benchmark_ticker})", True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
#         results_container[0] = results; debug_data_container[0] = debug_output
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
#             p = results['performance_data']
            
#             # --- START OF MODIFIED BLOCK ---
#             rows = []
#             # Gain Metrics
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p.get('full_b_gain')):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': '== Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            
#             # Standard Sharpe Metrics
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p.get('full_b_sharpe')):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': '== Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})

#             # Sharpe (ATR) Metrics
#             rows.append({'Metric': 'Group Portfolio Sharpe (ATR)', 'Full': p['full_p_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr']})
#             if not np.isnan(p.get('full_b_sharpe_atr')):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe (ATR)', 'Full': p['full_b_sharpe_atr'], 'Calc': p['calc_b_sharpe_atr'], 'Fwd': p['fwd_b_sharpe_atr']})
#                 rows.append({'Metric': '== Sharpe (ATR) Delta (vs Bm)', 'Full': p['full_p_sharpe_atr'] - p['full_b_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'] - p['calc_b_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr'] - p['fwd_b_sharpe_atr']})

#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row] # This now correctly includes both types of Sharpe
#             # --- END OF MODIFIED BLOCK ---
            
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.4f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return (results_container, debug_data_container)

# def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
#     """Runs a full backtest of a strategy over a specified date range."""
#     print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
    
#     # (No changes to the initial setup part...)
#     start_date, end_date = pd.to_datetime(strategy_params['start_date']), pd.to_datetime(strategy_params['end_date'])
#     calc_period, fwd_period = strategy_params['calc_period'], strategy_params['fwd_period']
#     metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
#     benchmark_ticker = strategy_params['benchmark_ticker']
#     master_calendar_ticker = strategy_params.get('master_calendar_ticker', 'VOO')
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     start_idx = master_trading_days.searchsorted(start_date)
#     end_idx = master_trading_days.searchsorted(end_date, side='right')
#     print("--- Generating all features upfront... ---")
#     features_df = generate_features(df_ohlcv)
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     # Loop through all periods in the backtest range
#     step_indices = range(start_idx, end_idx, fwd_period)
#     all_fwd_gains, period_by_period_debug = [], {}

#     print(f"Simulating {len(step_indices)} periods from {master_trading_days[step_indices[0]].date()} to {master_trading_days[step_indices[-1]].date()}...")
#     for current_idx in tqdm(step_indices, desc="Backtest Progress"):
#         step_date = master_trading_days[current_idx]
#         eligible_tickers = get_eligible_universe(features_df, step_date, quality_thresholds)
#         if not eligible_tickers: continue
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        
#         # +++ UPDATE THE CALL HERE +++
#         results, debug_output = run_walk_forward_step(
#             df_close_step, df_high_step, df_low_step, master_trading_days,
#             step_date, calc_period, fwd_period,
#             metric, rank_start, rank_end, benchmark_ticker,
#             features_df=features_df,  # Pass the features_df
#             debug=True
#         )
        
#         # (The rest of the function remains unchanged...)
#         if results['error'] is None:
#             fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
#             all_fwd_gains.append(fwd_series.pct_change().dropna())
#             period_by_period_debug[step_date.date().isoformat()] = debug_output
            
#     if not all_fwd_gains:
#         print("Error: No valid periods were simulated."); return None

#     strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
#     benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
#     cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
#     fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
#     fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
#     fig.show()
#     final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
#     print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
#     return final_backtest_results

# # --- E. VERIFICATION TOOLS (User Requested) ---

# def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
#                                                   start_date, calc_period, fwd_period,
#                                                   master_calendar_ticker='VOO', export_csv=False):
#     """Verifies portfolio and benchmark performance and optionally exports the data."""
#     display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
#     # 1. Setup trading day calendar and determine exact period dates
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
#     actual_calc_end_date = master_trading_days[calc_end_idx]
#     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
#     display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
#                     f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
#                     f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

#     # 2. Recreate the portfolio and benchmark series from scratch
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
#     portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
    
#     # 3. Optionally export the underlying daily data to a CSV for external checking
#     if export_csv:
#         export_df = pd.DataFrame({
#             'Portfolio_Normalized_Price': portfolio_value_series,
#             'Portfolio_Daily_Return': portfolio_value_series.pct_change()
#         })
#         if benchmark_price_series is not None:
#             norm_bm = benchmark_price_series.loc[actual_start_date:actual_fwd_end_date]
#             norm_bm = norm_bm / norm_bm.bfill().iloc[0]
#             export_df['Benchmark_Normalized_Price'] = norm_bm
#             export_df['Benchmark_Daily_Return'] = norm_bm.pct_change()

#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         tickers_str = '_'.join(tickers_to_verify)
#         filename = f"verify_group_{actual_start_date.date()}_{tickers_str}.csv"
#         filepath = os.path.join(output_dir, filename)
#         export_df.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")

#     # 4. Define a helper to print detailed calculation steps
#     def print_verification_steps(title, price_series):
#         display(Markdown(f"#### Verification for: `{title}`"))
#         if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
#         start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
#         gain = (end_price / start_price) - 1
#         print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
#         returns = price_series.pct_change()
#         sharpe = calculate_sharpe(returns)
#         print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
#         return {'gain': gain, 'sharpe': sharpe}

#     # 5. Run verification for each period
#     display(Markdown("### A. Calculation Period"))
#     perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
#     if benchmark_price_series is not None:
#         perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
    
#     display(Markdown("### B. Forward Period"))
#     perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
#     if benchmark_price_series is not None:
#         perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])

# def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
#                                   master_calendar_ticker='VOO', export_csv=False):
#     """Verifies ranking metrics for a single ticker and optionally exports the data."""
#     display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
#     # 1. Setup trading day calendar and determine exact period dates
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     if start_idx >= len(master_trading_days):
#         print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
#     actual_start_date = master_trading_days[start_idx]
    
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     actual_calc_end_date = master_trading_days[calc_end_idx]

#     # 2. Extract and prepare the raw data for the specific ticker and period
#     df_ticker = df_ohlcv.loc[ticker].sort_index()
#     calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
#     if calc_df.empty or len(calc_df) < 2: 
#         print("No data or not enough data in calc period."); return

#     display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
#     display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
#                     f"**Period End:** `{actual_calc_end_date.date()}`\n"
#                     f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
#     display(Markdown("#### Detailed Metric Calculation Data"))
    
#     # 3. Calculate all intermediate metrics as new columns for full transparency
#     vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
#     vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
    
#     # Corrected True Range (TR) calculation for a single ticker (Series)
#     tr_df = pd.DataFrame({
#         'h_l': vdf['Adj High'] - vdf['Adj Low'],
#         'h_cp': abs(vdf['Adj High'] - vdf['Adj Close'].shift(1)),
#         'l_cp': abs(vdf['Adj Low'] - vdf['Adj Close'].shift(1))
#     })
#     vdf['TR'] = tr_df.max(axis=1)
    
#     vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
#     vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
#     print("--- Start of Calculation Period ---")
#     display(vdf.head())
#     print("\n--- End of Calculation Period ---")
#     display(vdf.tail())

#     # 4. Optionally export this detailed breakdown to CSV
#     if export_csv:
#         output_dir = 'export_csv'
#         os.makedirs(output_dir, exist_ok=True)
#         filename = f"verify_ticker_{actual_start_date.date()}_{ticker}.csv"
#         filepath = os.path.join(output_dir, filename)
#         vdf.to_csv(filepath)
#         print(f"\n✅ Data exported to: {filepath}")
    
#     # 5. Print final metric calculations with formulas
#     display(Markdown("#### `MetricValue` Verification Summary:"))
    
#     calc_start_price = vdf['Adj Close'].bfill().iloc[0]
#     calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
#     price_metric = (calc_end_price / calc_start_price)
#     print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
#     daily_returns = vdf['Daily_Return'].dropna()
#     sharpe_ratio = calculate_sharpe(daily_returns)
#     print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

#     atrp_mean = vdf['ATRP'].mean()
#     mean_daily_return = vdf['Daily_Return'].mean()
#     sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
#     print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")


# def verify_sharpe_atr_calculation_checked(df_ohlcv, features_df, tickers_to_verify, benchmark_ticker,
#                                     start_date, calc_period, fwd_period,
#                                     master_calendar_ticker='VOO', debug=False):
#     """
#     Verifies the Sharpe (ATR) calculations for a portfolio and benchmark.

#     This function transparently recalculates the key components for Sharpe (ATR)
#     and can optionally export the underlying source data for manual inspection.
#     """
#     display(Markdown(f"## Verification Report for Sharpe (ATR) Calculation"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))

#     # --- 1. Determine Exact Period Dates (No changes here) ---
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     actual_start_date = master_trading_days[start_idx]
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
#     actual_calc_end_date = master_trading_days[calc_end_idx]
#     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
#     display(Markdown(f"**Full Period:** `{actual_start_date.date()}` to `{actual_fwd_end_date.date()}`\n"
#                     f"**Calc End Date:** `{actual_calc_end_date.date()}`"))

#     # Original debug export block (can be kept or removed)
#     if debug:
#         # ... (original export code remains here) ...
#         pass # Assuming original block is here

#     # --- 2. Recreate Portfolio & Benchmark Series from Scratch ---
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     portfolio_prices_raw = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
#     portfolio_prices_norm = portfolio_prices_raw.div(portfolio_prices_raw.bfill().iloc[0])
#     portfolio_value_series = portfolio_prices_norm.mean(axis=1)
#     portfolio_return_series = portfolio_value_series.pct_change()
#     p_idx = pd.MultiIndex.from_product([tickers_to_verify, portfolio_return_series.index])
#     p_atrp_df = features_df.loc[features_df.index.intersection(p_idx)]['ATRP'].unstack(level=0)
#     portfolio_atrp_series = p_atrp_df.mean(axis=1)

# ###############################    
#     # benchmark_return_series = df_close_full[benchmark_ticker].pct_change().loc[actual_start_date:actual_fwd_end_date]

#     # 1. First, slice the raw prices for the desired date range.
#     benchmark_prices_raw = df_close_full[benchmark_ticker].loc[actual_start_date:actual_fwd_end_date]
#     # 2. Then, calculate the percentage change on the sliced data.
#     benchmark_return_series = benchmark_prices_raw.pct_change()

# ###############################    
#     benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[actual_start_date:actual_fwd_end_date]


#     # +++ NEW: DETAILED RETURN CALCULATION TRACE (PORTFOLIO) +++
#     if debug:
#         display(Markdown("---"))
#         display(Markdown("### 🐛 Detailed Portfolio Return Calculation Trace"))

#         # Step 1: Show the raw prices being used
#         print("\n[STEP 1] Raw Adjusted Close prices used for calculation.")
#         print("Compare these values with your 'Adj Close' columns.")
#         display(portfolio_prices_raw)

#         # Step 2: Show the normalization base and the result
#         normalization_base = portfolio_prices_raw.bfill().iloc[0]
#         print("\n[STEP 2] Normalization base (first row of prices).")
#         print("Each column in Step 1 is divided by this corresponding value.")
#         display(pd.DataFrame(normalization_base).T)

#         print("\n[STEP 2a] Normalized prices (Result of division).")
#         print("Compare these values with your 'N Close' columns.")
#         display(portfolio_prices_norm)

#         # Step 3: Show the averaged portfolio value series
#         print("\n[STEP 3] Averaged normalized portfolio value series.")
#         print("This is the row-by-row average of the table in Step 2a.")
#         print("Compare this series with your 'N_portf' column.")
#         display(pd.DataFrame(portfolio_value_series, columns=['N_portf']))

#         # Step 4: Show the final portfolio return series
#         print("\n[STEP 4] Final portfolio daily return series (pct_change).")
#         print("This is the percentage change of the series in Step 3.")
#         print("Compare this series with your 'N_portf_rtn' column.")
#         display(pd.DataFrame(portfolio_return_series, columns=['N_portf_rtn']))
#         display(Markdown("---"))
#     # +++ END OF NEW DEBUG BLOCK +++


#     # --- 3. Define a Helper to Print Detailed Calculation Steps ---
#     # MODIFIED to include more debug details inside
#     def _calculate_and_print_metrics(period_name, returns, atrps):
#         display(Markdown(f"#### {period_name}"))
#         if returns.dropna().empty or atrps.dropna().empty:
#             print("  - Not enough data to calculate.")
#             return np.nan

#         # Standard calculations
#         mean_return = returns.mean()
#         mean_atrp = atrps.mean()
#         sharpe_atr = mean_return / mean_atrp if mean_atrp > 0 else np.nan

#         # +++ ADDED: Detailed Mean Calculation Breakdown +++
#         if debug:
#             valid_returns = returns.dropna()
#             num_returns = valid_returns.count()
#             sum_returns = valid_returns.sum()
#             manual_mean = sum_returns / num_returns if num_returns > 0 else 0
#             print(f"  - (Debug) Number of valid daily returns: {num_returns}")
#             print(f"  - (Debug) Sum of all daily returns:      {sum_returns:,.8f}")
#             print(f"  - (Debug) Manually calculated mean:      {manual_mean:,.8f} (Sum / Count)")
#         # +++ END OF ADDED DETAIL +++

#         print(f"  - Mean Daily Return: {mean_return:,.6f}")
#         print(f"  - Mean Daily ATRP:  {mean_atrp:,.6f}")
#         print(f"  - Sharpe (ATR) = (Mean Return / Mean ATRP) = {sharpe_atr:,.4f}")
#         return sharpe_atr

#     # --- 4. Run Verification for Portfolio (No changes here) ---
#     display(Markdown("### A. Group Portfolio Verification"))
#     _calculate_and_print_metrics("Full Period", portfolio_return_series, portfolio_atrp_series)
#     _calculate_and_print_metrics("Calculation Period", portfolio_return_series.loc[:actual_calc_end_date], portfolio_atrp_series.loc[:actual_calc_end_date])
#     _calculate_and_print_metrics("Forward Period", portfolio_return_series.loc[actual_calc_end_date:].iloc[1:], portfolio_atrp_series.loc[actual_calc_end_date:].iloc[1:])

#     # --- 5. Run Verification for Benchmark (No changes here) ---
#     display(Markdown(f"### B. Benchmark ({benchmark_ticker}) Verification"))
#     _calculate_and_print_metrics("Full Period", benchmark_return_series, benchmark_atrp_series)
#     _calculate_and_print_metrics("Calculation Period", benchmark_return_series.loc[:actual_calc_end_date], benchmark_atrp_series.loc[:actual_calc_end_date])
#     _calculate_and_print_metrics("Forward Period", benchmark_return_series.loc[actual_calc_end_date:].iloc[1:], benchmark_atrp_series.loc[actual_calc_end_date:].iloc[1:])




# # --- F. AUTOMATION SCRIPT - STRATEGY SEARCH ---

# def run_strategy_search(df_ohlcv, config):
#     """
#     Runs the main backtesting loop with checkpointing to be resumable.
#     """
#     start_time = time.time()
    
#     # --- 1. SETUP & LOAD PROGRESS ---
#     print("--- Phase 1: Pre-processing and Loading Progress ---")

#     # +++ ADD THIS BLOCK +++
#     # Pre-calculate all features for the entire dataset ONCE.
#     print("--- Generating all features upfront... ---")
#     features_df = generate_features(df_ohlcv)

#     # --- DELETE THIS LINE ---
#     # quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    
#     print("Unstacking data for performance...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
#     master_calendar_ticker = config['master_calendar_ticker']
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

#     results_path = config['results_output_path']
#     completed_params = set()
    
#     if os.path.exists(results_path):
#         print(f"Found existing results file. Loading progress from: {results_path}")
#         df_progress = pd.read_csv(results_path)
#         for _, row in df_progress.iterrows():
#             param_key = (
#                 row['calc_period'], row['fwd_period'], row['metric'],
#                 (row['rank_start'], row['rank_end'])
#             )
#             completed_params.add(param_key)
#         print(f"Found {len(completed_params)} completed parameter sets to skip.")
#     else:
#         print("No existing results file found. Starting a new run.")

#     print("✅ Pre-processing complete.\n")

#     # --- 2. SETUP THE MAIN LOOP (No changes here) ---
#     print("--- Phase 2: Setting up Simulation Loops ---")
#     param_combinations = list(product(
#         config['calc_periods'], config['fwd_periods'],
#         config['metrics'], config['rank_slices']
#     ))
#     search_start_date = pd.to_datetime(config['search_start_date'])
#     search_end_date = pd.to_datetime(config['search_end_date'])
#     start_idx = master_trading_days.searchsorted(search_start_date, side='left')
#     end_idx = master_trading_days.searchsorted(search_end_date, side='right')
#     step_dates_map = {}
#     print("Pre-calculating rebalancing schedules for each holding period...")
#     for fwd_period in sorted(config['fwd_periods']):
#         step_indices = range(start_idx, end_idx, fwd_period)
#         step_dates_map[fwd_period] = master_trading_days[step_indices]
#         print(f"  - Holding Period {fwd_period} days: {len(step_dates_map[fwd_period])} rebalances")
#     print(f"Found {len(param_combinations)} total parameter sets to simulate.")
#     print("✅ Setup complete. Starting main loop...\n")

#     # --- 3. RUN THE MAIN LOOP ---
#     print("--- Phase 3: Running Simulations ---")
#     pbar = tqdm(param_combinations, desc="Parameter Sets")
    
#     for params in pbar:
#         calc_period, fwd_period, metric, rank_slice = params
#         rank_start, rank_end = rank_slice
        
#         param_key = (calc_period, fwd_period, metric, rank_slice)
#         if param_key in completed_params:
#             pbar.set_description(f"Skipping {param_key}")
#             continue

#         pbar.set_description(f"Running {param_key}")
        
#         current_params_results = []
        
#         current_step_dates = step_dates_map[fwd_period]
#         for step_date in current_step_dates:
#             # +++ UPDATE THIS CALL +++
#             eligible_tickers = get_eligible_universe(
#                 features_df, filter_date=step_date, thresholds=config['quality_thresholds']
#             )
#             if not eligible_tickers: continue
            
#             df_close_step = df_close_full[eligible_tickers]
#             df_high_step = df_high_full[eligible_tickers]
#             df_low_step = df_low_full[eligible_tickers]

#             # +++ UPDATE THIS CALL +++
#             step_result, _ = run_walk_forward_step(
#                 df_close_full=df_close_step, df_high_full=df_high_step, df_low_full=df_low_step,
#                 master_trading_days=master_trading_days, start_date=step_date,
#                 calc_period=calc_period, fwd_period=fwd_period,
#                 metric=metric, rank_start=rank_start, rank_end=rank_end,
#                 benchmark_ticker=config['benchmark_ticker'],
#                 features_df=features_df, # Pass the features_df
#                 debug=False
#             )
            
#             if step_result['error'] is None:
#                 p = step_result['performance_data']
#                 log_entry = {
#                     'step_date': step_date.date(), 'calc_period': calc_period,
#                     'fwd_period': fwd_period, 'metric': metric,
#                     'rank_start': rank_start, 'rank_end': rank_end,
#                     'num_universe': len(eligible_tickers),
#                     'num_portfolio': len(step_result['tickers_to_display']),
#                     'fwd_p_gain': p['fwd_p_gain'], 'fwd_b_gain': p['fwd_b_gain'],
#                     'fwd_gain_delta': p['fwd_p_gain'] - p['fwd_b_gain'] if not np.isnan(p['fwd_b_gain']) else np.nan,
#                     'fwd_p_sharpe': p['fwd_p_sharpe'],
#                 }
#                 current_params_results.append(log_entry)
        
#         # --- CHECKPOINTING (No changes here) ---
#         if current_params_results:
#             df_to_append = pd.DataFrame(current_params_results)
#             df_to_append.to_csv(
#                 results_path,
#                 mode='a',
#                 header=not os.path.exists(results_path),
#                 index=False
#             )
#             completed_params.add(param_key)

#     print("✅ Main loop finished.\n")
    
#     # --- 4. RETURN FINAL DATAFRAME (No changes here) ---
#     print("--- Phase 4: Loading Final Results ---")
#     if os.path.exists(results_path):
#         final_df = pd.read_csv(results_path)
#         end_time = time.time()
#         print(f"✅ Process complete. Total execution time: {time.time() - start_time:.2f} seconds.")
#         return final_df
#     else:
#         print("Warning: No results were generated.")
#         return None    



# Below are only functions that required to run plot_walk_forward_analyzer  

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import pprint
from IPython.display import display, Markdown
import os # Make sure os is imported for the export function later


def calculate_gain(price_series: pd.Series):
    """Calculates the total gain over a series of prices."""
    # Ensure there are at least two data points to calculate a gain
    if price_series.dropna().shape[0] < 2: return np.nan
    # Use forward-fill for the end price and back-fill for the start price
    # to handle potential NaNs at the beginning or end of the series.
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series: pd.Series):
    """Calculates the annualized Sharpe ratio from a series of daily returns."""
    # Ensure there are at least two returns to calculate a standard deviation
    if return_series.dropna().shape[0] < 2: return np.nan
    std_dev = return_series.std()
    # Avoid division by zero if returns are constant
    if std_dev > 0 and std_dev != np.inf:
        return (return_series.mean() / std_dev) * np.sqrt(252)
    return np.nan

def calculate_sharpe_atr(return_series: pd.Series, atrp_series: pd.Series):
    """Calculates a Sharpe-like ratio using mean ATRP as the denominator."""
    # Ensure there are returns and that ATRP data is valid
    if return_series.dropna().shape[0] < 2 or atrp_series.dropna().empty:
        return np.nan
        
    mean_return = return_series.mean()
    mean_atrp = atrp_series.mean()
    
    # Avoid division by zero
    if mean_atrp > 0 and mean_atrp != np.inf:
        return mean_return / mean_atrp
        
    return np.nan


# --- B. MODULAR METRIC CALCULATION ENGINE ---

def calculate_price_metric(metric_data: dict) -> pd.Series:
    """
    Calculates the 'Price' metric (total gain) over the calculation period.

    Args:
        metric_data: A dictionary containing pre-calculated data Series.
                     Requires 'calc_close' (pd.DataFrame): The close prices for the calc period.

    Returns:
        A pandas Series of the calculated metric values, indexed by Ticker.
    """
    calc_close = metric_data['calc_close']
    
    # Ensure there are at least two data points to calculate a gain
    if len(calc_close) < 2:
        return pd.Series(dtype='float64', index=calc_close.columns)

    first_prices = calc_close.bfill().iloc[0]
    last_prices = calc_close.ffill().iloc[-1]
    
    # The division of two Series aligns by index (Ticker), which is what we want.
    price_metric = last_prices / first_prices
    
    return price_metric.dropna()

def calculate_sharpe_metric(metric_data: dict) -> pd.Series:
    """
    Calculates the annualized 'Sharpe' ratio metric over the calculation period.

    Args:
        metric_data: A dictionary containing pre-calculated data Series.
                     Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.

    Returns:
        A pandas Series of the calculated metric values, indexed by Ticker.
    """
    daily_returns = metric_data['daily_returns']
    
    # Ensure there's enough data to calculate standard deviation
    if len(daily_returns.dropna()) < 2:
        return pd.Series(dtype='float64', index=daily_returns.columns)
    
    mean_returns = daily_returns.mean()
    std_returns = daily_returns.std()
    
    # Standard annualized Sharpe Ratio calculation. Avoid division by zero.
    # We replace resulting NaNs/infs with 0 to handle cases of zero volatility.
    sharpe_ratio = (mean_returns / std_returns * np.sqrt(252))
    
    return sharpe_ratio.replace([np.inf, -np.inf], np.nan).fillna(0)
    
def calculate_sharpe_atr_metric(metric_data: dict) -> pd.Series:
    """
    Calculates the 'Sharpe (ATR)' metric over the calculation period.

    Args:
        metric_data: A dictionary containing pre-calculated data Series.
                     Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.
                     Requires 'atrp' (pd.Series): Mean ATRP for each ticker over the period.

    Returns:
        A pandas Series of the calculated metric values, indexed by Ticker.
    """
    daily_returns = metric_data['daily_returns']
    atrp = metric_data['atrp']
    
    mean_returns = daily_returns.mean()
    
    # ATRP-based Sharpe. Avoid division by zero.
    # We replace resulting NaNs/infs with 0.
    sharpe_atr = mean_returns / atrp
    
    return sharpe_atr.replace([np.inf, -np.inf], np.nan).fillna(0)


METRIC_REGISTRY = {
    'Price': calculate_price_metric,
    'Sharpe': calculate_sharpe_metric,
    'Sharpe (ATR)': calculate_sharpe_atr_metric,
}

def plot_walk_forward_analyzer(df_ohlcv,
                               default_start_date=None, default_calc_period=126, default_fwd_period=63,
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
                               quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
                               debug=False):
    # (No changes to the initial setup part of this function...)
    print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")
    print("--- Generating all features upfront... ---")
    features_df = generate_features(df_ohlcv)
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
    fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
    if default_metric not in METRIC_REGISTRY:
        fallback_metric = list(METRIC_REGISTRY.keys())[0]
        print(f"⚠️ Warning: Default metric '{default_metric}' not in registry. Using '{fallback_metric}'.")
        default_metric = fallback_metric
    metric_dropdown = widgets.Dropdown(options=list(METRIC_REGISTRY.keys()), value=default_metric, description='Metric:')
    rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
    rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container, debug_data_container = [None], [None]
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date_raw = pd.to_datetime(start_date_picker.value)
        start_date_idx = master_trading_days.searchsorted(start_date_raw)
        if start_date_idx >= len(master_trading_days):
            with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
        actual_start_date = master_trading_days[start_date_idx]
        with ticker_list_output:
            if start_date_raw.date() != actual_start_date.date():
                print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")
        calc_period = calc_period_input.value; fwd_period = fwd_period_input.value; metric = metric_dropdown.value
        rank_start = rank_start_input.value; rank_end = rank_end_input.value; benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
        if rank_start < 1 or calc_period < 2 or fwd_period < 1:
            with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return
        required_days = calc_period + fwd_period
        if start_date_idx + required_days > len(master_trading_days):
            available_days = len(master_trading_days) - start_date_idx; last_available_date = master_trading_days[-1].date()
            with ticker_list_output:
                print(f"Error: Not enough data for the requested period.\n  Start Date: {actual_start_date.date()}\n  Required Days: {calc_period} (calc) + {fwd_period} (fwd) = {required_days}\n  Available Days from Start: {available_days} (until {last_available_date})\n  Please shorten the 'Calc Period' / 'Fwd Period' or choose an earlier 'Start Date'.")
            return
        eligible_tickers = get_eligible_universe(features_df, actual_start_date, quality_thresholds)
        if not eligible_tickers:
            with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        results, debug_output = run_walk_forward_step(df_close_full, df_high_full, df_low_full, master_trading_days, actual_start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker, features_df=features_df, debug=debug)
        if results.get('error'):
            with ticker_list_output: print(f"Error: {results['error']}"); return
        period_dates = {'calc_period_start': results['safe_start_date'], 'calc_period_end': results['actual_calc_end_ts'], 'forward_period_start': results['actual_calc_end_ts'], 'forward_period_end': results['safe_viz_end_date']}
        run_parameters = {'calc_period': calc_period, 'fwd_period': fwd_period, 'rank_metric': metric, 'rank_start': rank_start, 'rank_end': rank_end, 'benchmark_ticker': benchmark_ticker}
        results.update(period_dates); results.update(run_parameters)
        if debug_output is not None and isinstance(debug_output, dict):
            debug_output.update(period_dates); debug_output.update(run_parameters)
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
                    trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark.values, f"Benchmark ({benchmark_ticker})", True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
        results_container[0] = results; debug_data_container[0] = debug_output
        with ticker_list_output:
            print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            pprint.pprint(results['tickers_to_display'])
            p = results['performance_data']
            
            # --- START OF MODIFIED BLOCK ---
            rows = []
            # Gain Metrics
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p.get('full_b_gain')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': '== Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            
            # Standard Sharpe Metrics
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p.get('full_b_sharpe')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': '== Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})

            # Sharpe (ATR) Metrics
            rows.append({'Metric': 'Group Portfolio Sharpe (ATR)', 'Full': p['full_p_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr']})
            if not np.isnan(p.get('full_b_sharpe_atr')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe (ATR)', 'Full': p['full_b_sharpe_atr'], 'Calc': p['calc_b_sharpe_atr'], 'Fwd': p['fwd_b_sharpe_atr']})
                rows.append({'Metric': '== Sharpe (ATR) Delta (vs Bm)', 'Full': p['full_p_sharpe_atr'] - p['full_b_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'] - p['calc_b_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr'] - p['fwd_b_sharpe_atr']})

            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row] # This now correctly includes both types of Sharpe
            # --- END OF MODIFIED BLOCK ---
            
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.4f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return (results_container, debug_data_container)


def generate_features(df_ohlcv: pd.DataFrame, 
                      atr_period: int = 14, 
                      quality_window: int = 252, 
                      quality_min_periods: int = 126) -> pd.DataFrame:
    """
    Generates a comprehensive DataFrame of derived features from raw OHLCV data.

    This function performs all heavy, window-based calculations upfront to be used
    by downstream analysis functions. It calculates:
    1. Technical Indicators: True Range (TR), ATR, and ATRP.
    2. Data Quality Metrics: Rolling stale percentage, median dollar volume, etc.

    Args:
        df_ohlcv: The primary DataFrame with a (Ticker, Date) MultiIndex and
                  columns for 'Adj High', 'Adj Low', 'Adj Close', 'Volume'.
        atr_period: The lookback period for the ATR's Exponential Moving Average.
        quality_window: The rolling window size for data quality metrics.
        quality_min_periods: The minimum number of observations required to have
                             a valid quality metric.

    Returns:
        A new DataFrame with the same (Ticker, Date) MultiIndex containing all
        calculated feature columns.
    """
    print("--- Starting Feature Generation ---")
    
    # Ensure the DataFrame is sorted for correct window and shift operations
    # FIX: Replaced is_lexsorted() with the current pandas attribute
    if not df_ohlcv.index.is_monotonic_increasing:
        print("Sorting index for calculation accuracy...")
        df_ohlcv = df_ohlcv.sort_index()

    # --- 1. Technical Indicator Calculation (TR, ATR, ATRP) ---
    print(f"Calculating technical indicators (ATR Period: {atr_period})...")
    
    # Group by ticker to handle each security independently
    grouped = df_ohlcv.groupby(level='Ticker')
    
    # Get the previous day's close required for True Range
    prev_close = grouped['Adj Close'].shift(1)
    
    # Calculate the three components of True Range
    high_low = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
    high_prev_close = abs(df_ohlcv['Adj High'] - prev_close)
    low_prev_close = abs(df_ohlcv['Adj Low'] - prev_close)
    
    # Combine the components to get the final TR
    tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1, skipna=False)
    
    # # Calculate the ATR using an Exponential Moving Average
    # --- FIX IS HERE ---
    # Use .transform() to apply the EWM function. 
    # This guarantees the resulting Series has the exact same index as 'tr',
    # preventing the index alignment error during the subsequent division.
    atr = tr.groupby(level='Ticker').transform(
        lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean()
    )

    # --- CHANGE 1: Removed .fillna(0) ---
    # ATRP will now be NaN on the first day, consistent with TR and ATR.
    atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)

    indicator_df = pd.DataFrame({
        'TR': tr,
        'ATR': atr,
        'ATRP': atrp
    })
    
    # --- 2. Data Quality Metric Calculation ---
    print(f"Calculating data quality metrics (Window: {quality_window} days)...")
    
    # Create intermediate flags needed for quality calculations
    is_stale = np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0)
    dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']
    has_same_volume = (grouped['Volume'].diff() == 0).astype(int)
    
    # Combine flags into a temporary DataFrame for rolling calculations
    quality_temp_df = pd.DataFrame({
        'IsStale': is_stale,
        'DollarVolume': dollar_volume,
        'HasSameVolume': has_same_volume
    }, index=df_ohlcv.index) # Explicitly set index to be safe
    
    # Perform the rolling calculations on the grouped data
    # --- FIX IS HERE ---
    # We switch to the older, more compatible dictionary-based aggregation method.
    # This syntax is understood by nearly all versions of pandas.
    rolling_result = quality_temp_df.groupby(level='Ticker').rolling(
        window=quality_window,
        min_periods=quality_min_periods
    ).agg({
        'IsStale': 'mean',
        'DollarVolume': 'median',
        'HasSameVolume': 'sum'
    })
    
    # The dictionary syntax produces columns with the original names ('IsStale', etc.).
    # We now explicitly rename them to our desired final names.
    rolling_result = rolling_result.rename(columns={
        'IsStale': 'RollingStalePct',
        'DollarVolume': 'RollMedDollarVol', # <-- RENAMED HERE
        'HasSameVolume': 'RollingSameVolCount'
    })

    # The index after a grouped rolling operation is hierarchical.
    # We remove the outermost 'Ticker' level to restore the original index structure.
    rolling_quality = rolling_result.reset_index(level=0, drop=True)

    # --- 3. Combine All Features ---
    print("Combining all feature sets...")
    features_df = pd.concat([indicator_df, rolling_quality], axis=1)
    
    print("✅ Feature generation complete.")
    return features_df


def get_eligible_universe(features_df, filter_date, thresholds):
    """Filters the universe of tickers based on quality metrics for a given date."""
    filter_date_ts = pd.to_datetime(filter_date)
    # The index is now the comprehensive features_df
    date_index = features_df.index.get_level_values('Date').unique().sort_values()
    
    if filter_date_ts < date_index[0]:
        print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
        return []
        
    # Find the most recent date with quality data on or before the filter date
    valid_prior_dates = date_index[date_index <= filter_date_ts]
    if valid_prior_dates.empty:
        print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
        return []
        
    actual_date_to_use = valid_prior_dates[-1]
    if actual_date_to_use.date() != filter_date_ts.date():
        print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")

    metrics_on_date = features_df.xs(actual_date_to_use, level='Date')
    
    # Apply filters using the new column names from features_df
    mask = ((metrics_on_date['RollMedDollarVol'] >= thresholds['min_median_dollar_volume']) & # <-- RENAMED
            (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
            (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
            
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers


def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
                          master_trading_days,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker,
                          features_df,
                          debug=False):
    """
    Runs a single step of the walk-forward analysis with a strict, pre-emptive
    check to ensure the full period is available.
    """
    debug_data = {} if debug else None

    # 1. Determine exact date ranges with a NEW pre-emptive check
    try:
        start_idx = master_trading_days.get_loc(start_date)
    except KeyError:
        return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)

    # +++ THIS IS THE NEW PRE-EMPTIVE CHECK LOGIC +++
    # Calculate the desired end index without clamping first.
    desired_viz_end_idx = start_idx + calc_period + fwd_period
    
    # Check if the desired end index is out of bounds.
    if desired_viz_end_idx >= len(master_trading_days):
        last_available_date = master_trading_days[-1].date()
        required_days = calc_period + fwd_period
        available_days = len(master_trading_days) - start_idx
        error_msg = (f"Not enough data for the full requested period. "
                     f"Required: {required_days} days, Available: {available_days} days until {last_available_date}.")
        return ({'error': error_msg}, None)
    # --- END OF NEW CHECK ---

    # If the check passes, we know the full period is available.
    # The 'min' calls are now just a redundant safety measure.
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

    safe_start_date = master_trading_days[start_idx]
    safe_calc_end_date = master_trading_days[calc_end_idx]
    safe_viz_end_date = master_trading_days[viz_end_idx]
    
    if safe_start_date >= safe_calc_end_date:
        return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

    # (The rest of the function remains completely unchanged...)
    # 2. Slice data for the calculation period
    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all')
    if calc_close.shape[1] == 0 or len(calc_close) < 2:
        return ({'error': "Not enough data in calc period."}, None)

    # 3. Calculate INTERMEDIATE data required for the ranking metrics
    daily_returns = calc_close.bfill().ffill().pct_change()
    valid_tickers = calc_close.columns
    calc_period_index = pd.MultiIndex.from_product([valid_tickers, calc_close.index], names=['Ticker', 'Date'])
    features_in_period = features_df.loc[features_df.index.intersection(calc_period_index)]
    atrp = features_in_period.groupby(level='Ticker')['ATRP'].mean()

    # 4. Calculate all ranking metrics by iterating through the METRIC_REGISTRY
    metric_ingredients = { 'calc_close': calc_close, 'daily_returns': daily_returns, 'atrp': atrp, }
    metric_values = {}
    for name, func in METRIC_REGISTRY.items():
        metric_values[name] = func(metric_ingredients)
    if metric not in metric_values or metric_values[metric].empty:
        return ({'error': f"Metric '{metric}' could not be calculated or resulted in no valid tickers."}, None)

    # 5. Rank tickers and select the portfolio
    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display:
        return ({'error': "No tickers found for the selected rank."}, None)

    # 6. Calculate Portfolio & Benchmark Performance
    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    actual_calc_end_ts = calc_close.index.max()
    portfolio_series = normalized_plot_data.mean(axis=1)
    portfolio_return_series = portfolio_series.pct_change()
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')
    try:
        boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
        calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
        fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
        if benchmark_price_series is not None:
            bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
            calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
            fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
        else:
            calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
    except (KeyError, IndexError):
        calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
        fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
        if benchmark_price_series is not None:
            calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
            fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]
        else:
            calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
    full_period_index = pd.MultiIndex.from_product([tickers_to_display, portfolio_return_series.index], names=['Ticker', 'Date'])
    portfolio_atrp_features = features_df.loc[features_df.index.intersection(full_period_index)]
    portfolio_atrp_daily_unstacked = portfolio_atrp_features['ATRP'].unstack(level='Ticker')
    portfolio_atrp_series = portfolio_atrp_daily_unstacked.mean(axis=1)
    if benchmark_ticker in df_close_full.columns:
        benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[safe_start_date:safe_viz_end_date]
    else:
        benchmark_atrp_series = pd.Series(dtype='float64')
    calc_portfolio_atrp = portfolio_atrp_series.loc[:actual_calc_end_ts]
    fwd_portfolio_atrp = portfolio_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
    calc_benchmark_atrp = benchmark_atrp_series.loc[:actual_calc_end_ts]
    fwd_benchmark_atrp = benchmark_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
    perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
    perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)
    perf_data['calc_p_sharpe_atr'] = calculate_sharpe_atr(calc_portfolio_returns, calc_portfolio_atrp)
    perf_data['fwd_p_sharpe_atr'] = calculate_sharpe_atr(fwd_portfolio_returns, fwd_portfolio_atrp)
    perf_data['full_p_sharpe_atr'] = calculate_sharpe_atr(portfolio_return_series, portfolio_atrp_series)
    perf_data['calc_b_sharpe_atr'] = calculate_sharpe_atr(calc_benchmark_returns, calc_benchmark_atrp)
    perf_data['fwd_b_sharpe_atr'] = calculate_sharpe_atr(fwd_benchmark_returns, fwd_benchmark_atrp)
    perf_data['full_b_sharpe_atr'] = calculate_sharpe_atr(benchmark_return_series, benchmark_atrp_series)
    if debug:
        df_ranking_base = pd.DataFrame({'MeanDailyReturn': daily_returns.mean(),'StdDevDailyReturn': daily_returns.std(),'MeanATRP': atrp})
        df_metrics = pd.DataFrame(metric_values)
        df_metrics.columns = [f'Metric_{col}' for col in df_metrics.columns]
        df_ranking = df_ranking_base.join(df_metrics, how='left')
        df_ranking.index.name = 'Ticker'
        debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)
    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])
    if debug:
        df_trace = normalized_plot_data.copy()
        df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
        df_trace['Norm_Price_Portfolio'] = portfolio_series
        if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
            norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
            df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
        for col in df_trace.columns:
            if 'Norm_Price' in col:
                df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
        debug_data['portfolio_trace'] = df_trace
    final_results = {
        'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
        'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
        'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
        'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
        'error': None
    }
    return (final_results, debug_data)

def verify_sharpe_atr_calculation_checked(df_ohlcv, features_df, tickers_to_verify, benchmark_ticker,
                                    start_date, calc_period, fwd_period,
                                    master_calendar_ticker='VOO', debug=False):
    """
    Verifies the Sharpe (ATR) calculations for a portfolio and benchmark.

    This function transparently recalculates the key components for Sharpe (ATR)
    and can optionally export the underlying source data for manual inspection.
    """
    display(Markdown(f"## Verification Report for Sharpe (ATR) Calculation"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))

    # --- 1. Determine Exact Period Dates (No changes here) ---
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    actual_start_date = master_trading_days[start_idx]
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    actual_calc_end_date = master_trading_days[calc_end_idx]
    actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
    display(Markdown(f"**Full Period:** `{actual_start_date.date()}` to `{actual_fwd_end_date.date()}`\n"
                    f"**Calc End Date:** `{actual_calc_end_date.date()}`"))

    # Original debug export block (can be kept or removed)
    if debug:
        # ... (original export code remains here) ...
        pass # Assuming original block is here

    # --- 2. Recreate Portfolio & Benchmark Series from Scratch ---
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    portfolio_prices_raw = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
    portfolio_prices_norm = portfolio_prices_raw.div(portfolio_prices_raw.bfill().iloc[0])
    portfolio_value_series = portfolio_prices_norm.mean(axis=1)
    portfolio_return_series = portfolio_value_series.pct_change()
    p_idx = pd.MultiIndex.from_product([tickers_to_verify, portfolio_return_series.index])
    p_atrp_df = features_df.loc[features_df.index.intersection(p_idx)]['ATRP'].unstack(level=0)
    portfolio_atrp_series = p_atrp_df.mean(axis=1)

###############################    
    # benchmark_return_series = df_close_full[benchmark_ticker].pct_change().loc[actual_start_date:actual_fwd_end_date]

    # 1. First, slice the raw prices for the desired date range.
    benchmark_prices_raw = df_close_full[benchmark_ticker].loc[actual_start_date:actual_fwd_end_date]
    # 2. Then, calculate the percentage change on the sliced data.
    benchmark_return_series = benchmark_prices_raw.pct_change()

###############################    
    benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[actual_start_date:actual_fwd_end_date]


    # +++ NEW: DETAILED RETURN CALCULATION TRACE (PORTFOLIO) +++
    if debug:
        display(Markdown("---"))
        display(Markdown("### 🐛 Detailed Portfolio Return Calculation Trace"))

        # Step 1: Show the raw prices being used
        print("\n[STEP 1] Raw Adjusted Close prices used for calculation.")
        print("Compare these values with your 'Adj Close' columns.")
        display(portfolio_prices_raw)

        # Step 2: Show the normalization base and the result
        normalization_base = portfolio_prices_raw.bfill().iloc[0]
        print("\n[STEP 2] Normalization base (first row of prices).")
        print("Each column in Step 1 is divided by this corresponding value.")
        display(pd.DataFrame(normalization_base).T)

        print("\n[STEP 2a] Normalized prices (Result of division).")
        print("Compare these values with your 'N Close' columns.")
        display(portfolio_prices_norm)

        # Step 3: Show the averaged portfolio value series
        print("\n[STEP 3] Averaged normalized portfolio value series.")
        print("This is the row-by-row average of the table in Step 2a.")
        print("Compare this series with your 'N_portf' column.")
        display(pd.DataFrame(portfolio_value_series, columns=['N_portf']))

        # Step 4: Show the final portfolio return series
        print("\n[STEP 4] Final portfolio daily return series (pct_change).")
        print("This is the percentage change of the series in Step 3.")
        print("Compare this series with your 'N_portf_rtn' column.")
        display(pd.DataFrame(portfolio_return_series, columns=['N_portf_rtn']))
        display(Markdown("---"))
    # +++ END OF NEW DEBUG BLOCK +++


    # --- 3. Define a Helper to Print Detailed Calculation Steps ---
    # MODIFIED to include more debug details inside
    def _calculate_and_print_metrics(period_name, returns, atrps):
        display(Markdown(f"#### {period_name}"))
        if returns.dropna().empty or atrps.dropna().empty:
            print("  - Not enough data to calculate.")
            return np.nan

        # Standard calculations
        mean_return = returns.mean()
        mean_atrp = atrps.mean()
        sharpe_atr = mean_return / mean_atrp if mean_atrp > 0 else np.nan

        # +++ ADDED: Detailed Mean Calculation Breakdown +++
        if debug:
            valid_returns = returns.dropna()
            num_returns = valid_returns.count()
            sum_returns = valid_returns.sum()
            manual_mean = sum_returns / num_returns if num_returns > 0 else 0
            print(f"  - (Debug) Number of valid daily returns: {num_returns}")
            print(f"  - (Debug) Sum of all daily returns:      {sum_returns:,.8f}")
            print(f"  - (Debug) Manually calculated mean:      {manual_mean:,.8f} (Sum / Count)")
        # +++ END OF ADDED DETAIL +++

        print(f"  - Mean Daily Return: {mean_return:,.6f}")
        print(f"  - Mean Daily ATRP:  {mean_atrp:,.6f}")
        print(f"  - Sharpe (ATR) = (Mean Return / Mean ATRP) = {sharpe_atr:,.4f}")
        return sharpe_atr

    # --- 4. Run Verification for Portfolio (No changes here) ---
    display(Markdown("### A. Group Portfolio Verification"))
    _calculate_and_print_metrics("Full Period", portfolio_return_series, portfolio_atrp_series)
    _calculate_and_print_metrics("Calculation Period", portfolio_return_series.loc[:actual_calc_end_date], portfolio_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", portfolio_return_series.loc[actual_calc_end_date:].iloc[1:], portfolio_atrp_series.loc[actual_calc_end_date:].iloc[1:])

    # --- 5. Run Verification for Benchmark (No changes here) ---
    display(Markdown(f"### B. Benchmark ({benchmark_ticker}) Verification"))
    _calculate_and_print_metrics("Full Period", benchmark_return_series, benchmark_atrp_series)
    _calculate_and_print_metrics("Calculation Period", benchmark_return_series.loc[:actual_calc_end_date], benchmark_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", benchmark_return_series.loc[actual_calc_end_date:].iloc[1:], benchmark_atrp_series.loc[actual_calc_end_date:].iloc[1:])




In [11]:
# Now call the verification function with the exact parameters from the UI
verify_sharpe_atr_calculation_checked(
    df_ohlcv=df_ohlcv,
    features_df=features_df,
    tickers_to_verify=tickers_to_verify, # <-- From the UI output
    benchmark_ticker=benchmark_ticker,
    start_date=start_date,
    calc_period=10,
    fwd_period=5,
    debug=True,
)

## Verification Report for Sharpe (ATR) Calculation

**Portfolio Tickers:** `['STIP', 'RCL']`
**Benchmark Ticker:** `VOO`

**Full Period:** `2025-08-13` to `2025-09-04`
**Calc End Date:** `2025-08-27`

---

### 🐛 Detailed Portfolio Return Calculation Trace

...

  - Mean Daily Return: 0.000438
  - Mean Daily ATRP:  0.008277
  - Sharpe (ATR) = (Mean Return / Mean ATRP) = 0.0529


#### Calculation Period

  - Mean Daily Return: 0.000283
  - Mean Daily ATRP:  0.008317
  - Sharpe (ATR) = (Mean Return / Mean ATRP) = 0.0340


#### Forward Period

  - Mean Daily Return: 0.000748
  - Mean Daily ATRP:  0.008191
  - Sharpe (ATR) = (Mean Return / Mean ATRP) = 0.0913


# PORTFOLIO & BENCHMARK SHARPE (ATR) Calculation have been verified

### My Verification of Portfolio Sharpe (ATR)

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import pprint
from IPython.display import display, Markdown
import os # Make sure os is imported for the export function later


# def calculate_gain(price_series: pd.Series):
#     """Calculates the total gain over a series of prices."""
#     # Ensure there are at least two data points to calculate a gain
#     if price_series.dropna().shape[0] < 2: return np.nan
#     # Use forward-fill for the end price and back-fill for the start price
#     # to handle potential NaNs at the beginning or end of the series.
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     """Calculates the annualized Sharpe ratio from a series of daily returns."""
#     # Ensure there are at least two returns to calculate a standard deviation
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     # Avoid division by zero if returns are constant
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# def calculate_sharpe_atr(return_series: pd.Series, atrp_series: pd.Series):
#     """Calculates a Sharpe-like ratio using mean ATRP as the denominator."""
#     # Ensure there are returns and that ATRP data is valid
#     if return_series.dropna().shape[0] < 2 or atrp_series.dropna().empty:
#         return np.nan
        
#     mean_return = return_series.mean()
#     mean_atrp = atrp_series.mean()
    
#     # Avoid division by zero
#     if mean_atrp > 0 and mean_atrp != np.inf:
#         return mean_return / mean_atrp
        
#     return np.nan


# # --- B. MODULAR METRIC CALCULATION ENGINE ---

# def calculate_price_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the 'Price' metric (total gain) over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'calc_close' (pd.DataFrame): The close prices for the calc period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     calc_close = metric_data['calc_close']
    
#     # Ensure there are at least two data points to calculate a gain
#     if len(calc_close) < 2:
#         return pd.Series(dtype='float64', index=calc_close.columns)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
    
#     # The division of two Series aligns by index (Ticker), which is what we want.
#     price_metric = last_prices / first_prices
    
#     return price_metric.dropna()

# def calculate_sharpe_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the annualized 'Sharpe' ratio metric over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     daily_returns = metric_data['daily_returns']
    
#     # Ensure there's enough data to calculate standard deviation
#     if len(daily_returns.dropna()) < 2:
#         return pd.Series(dtype='float64', index=daily_returns.columns)
    
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     # Standard annualized Sharpe Ratio calculation. Avoid division by zero.
#     # We replace resulting NaNs/infs with 0 to handle cases of zero volatility.
#     sharpe_ratio = (mean_returns / std_returns * np.sqrt(252))
    
#     return sharpe_ratio.replace([np.inf, -np.inf], np.nan).fillna(0)
    
# def calculate_sharpe_atr_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the 'Sharpe (ATR)' metric over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.
#                      Requires 'atrp' (pd.Series): Mean ATRP for each ticker over the period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     daily_returns = metric_data['daily_returns']
#     atrp = metric_data['atrp']
    
#     mean_returns = daily_returns.mean()
    
#     # ATRP-based Sharpe. Avoid division by zero.
#     # We replace resulting NaNs/infs with 0.
#     sharpe_atr = mean_returns / atrp
    
#     return sharpe_atr.replace([np.inf, -np.inf], np.nan).fillna(0)


# METRIC_REGISTRY = {
#     'Price': calculate_price_metric,
#     'Sharpe': calculate_sharpe_metric,
#     'Sharpe (ATR)': calculate_sharpe_atr_metric,
# }

# def plot_walk_forward_analyzer(df_ohlcv,
#                                default_start_date=None, default_calc_period=126, default_fwd_period=63,
#                                default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
#                                default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
#                                quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
#                                debug=False):
#     # (No changes to the initial setup part of this function...)
#     print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
#     if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
#     df_ohlcv = df_ohlcv.sort_index()
#     if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
#         raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")
#     print("--- Generating all features upfront... ---")
#     features_df = generate_features(df_ohlcv)
#     print("Pre-processing data (unstacking)...")
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     df_high_full = df_ohlcv['Adj High'].unstack(level=0)
#     df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
#     start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
#     calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
#     fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
#     if default_metric not in METRIC_REGISTRY:
#         fallback_metric = list(METRIC_REGISTRY.keys())[0]
#         print(f"⚠️ Warning: Default metric '{default_metric}' not in registry. Using '{fallback_metric}'.")
#         default_metric = fallback_metric
#     metric_dropdown = widgets.Dropdown(options=list(METRIC_REGISTRY.keys()), value=default_metric, description='Metric:')
#     rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
#     rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
#     benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
#     update_button = widgets.Button(description="Update Chart", button_style='primary')
#     ticker_list_output = widgets.Output()
#     results_container, debug_data_container = [None], [None]
#     fig = go.FigureWidget()
#     max_traces = 50
#     for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
#     fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

#     def update_plot(button_click):
#         ticker_list_output.clear_output()
#         start_date_raw = pd.to_datetime(start_date_picker.value)
#         start_date_idx = master_trading_days.searchsorted(start_date_raw)
#         if start_date_idx >= len(master_trading_days):
#             with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
#         actual_start_date = master_trading_days[start_date_idx]
#         with ticker_list_output:
#             if start_date_raw.date() != actual_start_date.date():
#                 print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")
#         calc_period = calc_period_input.value; fwd_period = fwd_period_input.value; metric = metric_dropdown.value
#         rank_start = rank_start_input.value; rank_end = rank_end_input.value; benchmark_ticker = benchmark_ticker_input.value.strip().upper()
#         if rank_start > rank_end:
#             with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
#         if rank_start < 1 or calc_period < 2 or fwd_period < 1:
#             with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return
#         required_days = calc_period + fwd_period
#         if start_date_idx + required_days > len(master_trading_days):
#             available_days = len(master_trading_days) - start_date_idx; last_available_date = master_trading_days[-1].date()
#             with ticker_list_output:
#                 print(f"Error: Not enough data for the requested period.\n  Start Date: {actual_start_date.date()}\n  Required Days: {calc_period} (calc) + {fwd_period} (fwd) = {required_days}\n  Available Days from Start: {available_days} (until {last_available_date})\n  Please shorten the 'Calc Period' / 'Fwd Period' or choose an earlier 'Start Date'.")
#             return
#         eligible_tickers = get_eligible_universe(features_df, actual_start_date, quality_thresholds)
#         if not eligible_tickers:
#             with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
#         df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
#         results, debug_output = run_walk_forward_step(df_close_full, df_high_full, df_low_full, master_trading_days, actual_start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker, features_df=features_df, debug=debug)
#         if results.get('error'):
#             with ticker_list_output: print(f"Error: {results['error']}"); return
#         period_dates = {'calc_period_start': results['safe_start_date'], 'calc_period_end': results['actual_calc_end_ts'], 'forward_period_start': results['actual_calc_end_ts'], 'forward_period_end': results['safe_viz_end_date']}
#         run_parameters = {'calc_period': calc_period, 'fwd_period': fwd_period, 'rank_metric': metric, 'rank_start': rank_start, 'rank_end': rank_end, 'benchmark_ticker': benchmark_ticker}
#         results.update(period_dates); results.update(run_parameters)
#         if debug_output is not None and isinstance(debug_output, dict):
#             debug_output.update(period_dates); debug_output.update(run_parameters)
#         with fig.batch_update():
#             for i in range(max_traces):
#                 trace = fig.data[i]
#                 if i < len(results['tickers_to_display']):
#                     ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
#                     trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
#                 else: trace.visible, trace.showlegend = False, False
#             benchmark_trace = fig.data[max_traces]
#             if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
#                 normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
#                 benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark.values, f"Benchmark ({benchmark_ticker})", True
#             else: benchmark_trace.visible = False
#             portfolio_trace = fig.data[max_traces + 1]
#             portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
#             fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
#         results_container[0] = results; debug_data_container[0] = debug_output
#         with ticker_list_output:
#             print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
#             pprint.pprint(results['tickers_to_display'])
#             p = results['performance_data']
            
#             # --- START OF MODIFIED BLOCK ---
#             rows = []
#             # Gain Metrics
#             rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
#             if not np.isnan(p.get('full_b_gain')):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
#                 rows.append({'Metric': '== Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            
#             # Standard Sharpe Metrics
#             rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
#             if not np.isnan(p.get('full_b_sharpe')):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
#                 rows.append({'Metric': '== Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})

#             # Sharpe (ATR) Metrics
#             rows.append({'Metric': 'Group Portfolio Sharpe (ATR)', 'Full': p['full_p_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr']})
#             if not np.isnan(p.get('full_b_sharpe_atr')):
#                 rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe (ATR)', 'Full': p['full_b_sharpe_atr'], 'Calc': p['calc_b_sharpe_atr'], 'Fwd': p['fwd_b_sharpe_atr']})
#                 rows.append({'Metric': '== Sharpe (ATR) Delta (vs Bm)', 'Full': p['full_p_sharpe_atr'] - p['full_b_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'] - p['calc_b_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr'] - p['fwd_b_sharpe_atr']})

#             report_df = pd.DataFrame(rows).set_index('Metric')
#             gain_rows = [row for row in report_df.index if 'Gain' in row]
#             sharpe_rows = [row for row in report_df.index if 'Sharpe' in row] # This now correctly includes both types of Sharpe
#             # --- END OF MODIFIED BLOCK ---
            
#             styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
#             print("\n--- Strategy Performance Summary ---")
#             display(styled_df)
#     fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
#     fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
#     update_button.on_click(update_plot)
#     controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
#     controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
#     ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
#     display(ui_container, fig)
#     update_plot(None)
#     return (results_container, debug_data_container)


def generate_features(df_ohlcv: pd.DataFrame, 
                      atr_period: int = 14, 
                      quality_window: int = 252, 
                      quality_min_periods: int = 126) -> pd.DataFrame:
    """
    Generates a comprehensive DataFrame of derived features from raw OHLCV data.

    This function performs all heavy, window-based calculations upfront to be used
    by downstream analysis functions. It calculates:
    1. Technical Indicators: True Range (TR), ATR, and ATRP.
    2. Data Quality Metrics: Rolling stale percentage, median dollar volume, etc.

    Args:
        df_ohlcv: The primary DataFrame with a (Ticker, Date) MultiIndex and
                  columns for 'Adj High', 'Adj Low', 'Adj Close', 'Volume'.
        atr_period: The lookback period for the ATR's Exponential Moving Average.
        quality_window: The rolling window size for data quality metrics.
        quality_min_periods: The minimum number of observations required to have
                             a valid quality metric.

    Returns:
        A new DataFrame with the same (Ticker, Date) MultiIndex containing all
        calculated feature columns.
    """
    print("--- Starting Feature Generation ---")
    
    # Ensure the DataFrame is sorted for correct window and shift operations
    # FIX: Replaced is_lexsorted() with the current pandas attribute
    if not df_ohlcv.index.is_monotonic_increasing:
        print("Sorting index for calculation accuracy...")
        df_ohlcv = df_ohlcv.sort_index()

    # --- 1. Technical Indicator Calculation (TR, ATR, ATRP) ---
    print(f"Calculating technical indicators (ATR Period: {atr_period})...")
    
    # Group by ticker to handle each security independently
    grouped = df_ohlcv.groupby(level='Ticker')
    
    # Get the previous day's close required for True Range
    prev_close = grouped['Adj Close'].shift(1)
    
    # Calculate the three components of True Range
    high_low = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
    high_prev_close = abs(df_ohlcv['Adj High'] - prev_close)
    low_prev_close = abs(df_ohlcv['Adj Low'] - prev_close)
    
    # Combine the components to get the final TR
    tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1, skipna=False)
    
    # # Calculate the ATR using an Exponential Moving Average
    # --- FIX IS HERE ---
    # Use .transform() to apply the EWM function. 
    # This guarantees the resulting Series has the exact same index as 'tr',
    # preventing the index alignment error during the subsequent division.
    atr = tr.groupby(level='Ticker').transform(
        lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean()
    )

    # --- CHANGE 1: Removed .fillna(0) ---
    # ATRP will now be NaN on the first day, consistent with TR and ATR.
    atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)

    indicator_df = pd.DataFrame({
        'TR': tr,
        'ATR': atr,
        'ATRP': atrp
    })
    
    # --- 2. Data Quality Metric Calculation ---
    print(f"Calculating data quality metrics (Window: {quality_window} days)...")
    
    # Create intermediate flags needed for quality calculations
    is_stale = np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0)
    dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']
    has_same_volume = (grouped['Volume'].diff() == 0).astype(int)
    
    # Combine flags into a temporary DataFrame for rolling calculations
    quality_temp_df = pd.DataFrame({
        'IsStale': is_stale,
        'DollarVolume': dollar_volume,
        'HasSameVolume': has_same_volume
    }, index=df_ohlcv.index) # Explicitly set index to be safe
    
    # Perform the rolling calculations on the grouped data
    # --- FIX IS HERE ---
    # We switch to the older, more compatible dictionary-based aggregation method.
    # This syntax is understood by nearly all versions of pandas.
    rolling_result = quality_temp_df.groupby(level='Ticker').rolling(
        window=quality_window,
        min_periods=quality_min_periods
    ).agg({
        'IsStale': 'mean',
        'DollarVolume': 'median',
        'HasSameVolume': 'sum'
    })
    
    # The dictionary syntax produces columns with the original names ('IsStale', etc.).
    # We now explicitly rename them to our desired final names.
    rolling_result = rolling_result.rename(columns={
        'IsStale': 'RollingStalePct',
        'DollarVolume': 'RollMedDollarVol', # <-- RENAMED HERE
        'HasSameVolume': 'RollingSameVolCount'
    })

    # The index after a grouped rolling operation is hierarchical.
    # We remove the outermost 'Ticker' level to restore the original index structure.
    rolling_quality = rolling_result.reset_index(level=0, drop=True)

    # --- 3. Combine All Features ---
    print("Combining all feature sets...")
    features_df = pd.concat([indicator_df, rolling_quality], axis=1)
    
    print("✅ Feature generation complete.")
    return features_df


# def get_eligible_universe(features_df, filter_date, thresholds):
#     """Filters the universe of tickers based on quality metrics for a given date."""
#     filter_date_ts = pd.to_datetime(filter_date)
#     # The index is now the comprehensive features_df
#     date_index = features_df.index.get_level_values('Date').unique().sort_values()
    
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
        
#     # Find the most recent date with quality data on or before the filter date
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
        
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")

#     metrics_on_date = features_df.xs(actual_date_to_use, level='Date')
    
#     # Apply filters using the new column names from features_df
#     mask = ((metrics_on_date['RollMedDollarVol'] >= thresholds['min_median_dollar_volume']) & # <-- RENAMED
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
            
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers


# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           features_df,
#                           debug=False):
#     """
#     Runs a single step of the walk-forward analysis with a strict, pre-emptive
#     check to ensure the full period is available.
#     """
#     debug_data = {} if debug else None

#     # 1. Determine exact date ranges with a NEW pre-emptive check
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)

#     # +++ THIS IS THE NEW PRE-EMPTIVE CHECK LOGIC +++
#     # Calculate the desired end index without clamping first.
#     desired_viz_end_idx = start_idx + calc_period + fwd_period
    
#     # Check if the desired end index is out of bounds.
#     if desired_viz_end_idx >= len(master_trading_days):
#         last_available_date = master_trading_days[-1].date()
#         required_days = calc_period + fwd_period
#         available_days = len(master_trading_days) - start_idx
#         error_msg = (f"Not enough data for the full requested period. "
#                      f"Required: {required_days} days, Available: {available_days} days until {last_available_date}.")
#         return ({'error': error_msg}, None)
#     # --- END OF NEW CHECK ---

#     # If the check passes, we know the full period is available.
#     # The 'min' calls are now just a redundant safety measure.
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     # (The rest of the function remains completely unchanged...)
#     # 2. Slice data for the calculation period
#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     # 3. Calculate INTERMEDIATE data required for the ranking metrics
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     valid_tickers = calc_close.columns
#     calc_period_index = pd.MultiIndex.from_product([valid_tickers, calc_close.index], names=['Ticker', 'Date'])
#     features_in_period = features_df.loc[features_df.index.intersection(calc_period_index)]
#     atrp = features_in_period.groupby(level='Ticker')['ATRP'].mean()

#     # 4. Calculate all ranking metrics by iterating through the METRIC_REGISTRY
#     metric_ingredients = { 'calc_close': calc_close, 'daily_returns': daily_returns, 'atrp': atrp, }
#     metric_values = {}
#     for name, func in METRIC_REGISTRY.items():
#         metric_values[name] = func(metric_ingredients)
#     if metric not in metric_values or metric_values[metric].empty:
#         return ({'error': f"Metric '{metric}' could not be calculated or resulted in no valid tickers."}, None)

#     # 5. Rank tickers and select the portfolio
#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     # 6. Calculate Portfolio & Benchmark Performance
#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')
#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
#         if benchmark_price_series is not None:
#             bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#             calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#             fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         if benchmark_price_series is not None:
#             calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#             fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     full_period_index = pd.MultiIndex.from_product([tickers_to_display, portfolio_return_series.index], names=['Ticker', 'Date'])
#     portfolio_atrp_features = features_df.loc[features_df.index.intersection(full_period_index)]
#     portfolio_atrp_daily_unstacked = portfolio_atrp_features['ATRP'].unstack(level='Ticker')
#     portfolio_atrp_series = portfolio_atrp_daily_unstacked.mean(axis=1)
#     if benchmark_ticker in df_close_full.columns:
#         benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[safe_start_date:safe_viz_end_date]
#     else:
#         benchmark_atrp_series = pd.Series(dtype='float64')
#     calc_portfolio_atrp = portfolio_atrp_series.loc[:actual_calc_end_ts]
#     fwd_portfolio_atrp = portfolio_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     calc_benchmark_atrp = benchmark_atrp_series.loc[:actual_calc_end_ts]
#     fwd_benchmark_atrp = benchmark_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)
#     perf_data['calc_p_sharpe_atr'] = calculate_sharpe_atr(calc_portfolio_returns, calc_portfolio_atrp)
#     perf_data['fwd_p_sharpe_atr'] = calculate_sharpe_atr(fwd_portfolio_returns, fwd_portfolio_atrp)
#     perf_data['full_p_sharpe_atr'] = calculate_sharpe_atr(portfolio_return_series, portfolio_atrp_series)
#     perf_data['calc_b_sharpe_atr'] = calculate_sharpe_atr(calc_benchmark_returns, calc_benchmark_atrp)
#     perf_data['fwd_b_sharpe_atr'] = calculate_sharpe_atr(fwd_benchmark_returns, fwd_benchmark_atrp)
#     perf_data['full_b_sharpe_atr'] = calculate_sharpe_atr(benchmark_return_series, benchmark_atrp_series)
#     if debug:
#         df_ranking_base = pd.DataFrame({'MeanDailyReturn': daily_returns.mean(),'StdDevDailyReturn': daily_returns.std(),'MeanATRP': atrp})
#         df_metrics = pd.DataFrame(metric_values)
#         df_metrics.columns = [f'Metric_{col}' for col in df_metrics.columns]
#         df_ranking = df_ranking_base.join(df_metrics, how='left')
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)
#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace
#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)

def verify_sharpe_atr_calculation_checked(df_ohlcv, features_df, tickers_to_verify, benchmark_ticker,
                                    start_date, calc_period, fwd_period,
                                    master_calendar_ticker='VOO', debug=False):
    """
    Verifies the Sharpe (ATR) calculations for a portfolio and benchmark.

    This function transparently recalculates the key components for Sharpe (ATR)
    and can optionally export the underlying source data for manual inspection.
    """
    display(Markdown(f"## Verification Report for Sharpe (ATR) Calculation"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))

    # --- 1. Determine Exact Period Dates (No changes here) ---
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    actual_start_date = master_trading_days[start_idx]
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    actual_calc_end_date = master_trading_days[calc_end_idx]
    actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
    display(Markdown(f"**Full Period:** `{actual_start_date.date()}` to `{actual_fwd_end_date.date()}`\n"
                    f"**Calc End Date:** `{actual_calc_end_date.date()}`"))

    # Original debug export block (can be kept or removed)
    if debug:
        # ... (original export code remains here) ...
        pass # Assuming original block is here

    # --- 2. Recreate Portfolio & Benchmark Series from Scratch ---
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    portfolio_prices_raw = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
    portfolio_prices_norm = portfolio_prices_raw.div(portfolio_prices_raw.bfill().iloc[0])
    portfolio_value_series = portfolio_prices_norm.mean(axis=1)
    portfolio_return_series = portfolio_value_series.pct_change()
    p_idx = pd.MultiIndex.from_product([tickers_to_verify, portfolio_return_series.index])
    p_atrp_df = features_df.loc[features_df.index.intersection(p_idx)]['ATRP'].unstack(level=0)
    portfolio_atrp_series = p_atrp_df.mean(axis=1)

###############################    
    # benchmark_return_series = df_close_full[benchmark_ticker].pct_change().loc[actual_start_date:actual_fwd_end_date]

    # 1. First, slice the raw prices for the desired date range.
    benchmark_prices_raw = df_close_full[benchmark_ticker].loc[actual_start_date:actual_fwd_end_date]
    # 2. Then, calculate the percentage change on the sliced data.
    benchmark_return_series = benchmark_prices_raw.pct_change()

###############################    
    benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[actual_start_date:actual_fwd_end_date]


    # +++ NEW: DETAILED RETURN CALCULATION TRACE (PORTFOLIO) +++
    if debug:
        display(Markdown("---"))
        display(Markdown("### 🐛 Detailed Portfolio Return Calculation Trace"))

        # Step 1: Show the raw prices being used
        print("\n[STEP 1] Raw Adjusted Close prices used for calculation.")
        print("Compare these values with your 'Adj Close' columns.")
        display(portfolio_prices_raw)

        # Step 2: Show the normalization base and the result
        normalization_base = portfolio_prices_raw.bfill().iloc[0]
        print("\n[STEP 2] Normalization base (first row of prices).")
        print("Each column in Step 1 is divided by this corresponding value.")
        display(pd.DataFrame(normalization_base).T)

        print("\n[STEP 2a] Normalized prices (Result of division).")
        print("Compare these values with your 'N Close' columns.")
        display(portfolio_prices_norm)

        # Step 3: Show the averaged portfolio value series
        print("\n[STEP 3] Averaged normalized portfolio value series.")
        print("This is the row-by-row average of the table in Step 2a.")
        print("Compare this series with your 'N_portf' column.")
        display(pd.DataFrame(portfolio_value_series, columns=['N_portf']))

        # Step 4: Show the final portfolio return series
        print("\n[STEP 4] Final portfolio daily return series (pct_change).")
        print("This is the percentage change of the series in Step 3.")
        print("Compare this series with your 'N_portf_rtn' column.")
        display(pd.DataFrame(portfolio_return_series, columns=['N_portf_rtn']))
        display(Markdown("---"))
    # +++ END OF NEW DEBUG BLOCK +++


    # --- 3. Define a Helper to Print Detailed Calculation Steps ---
    # MODIFIED to include more debug details inside
    def _calculate_and_print_metrics(period_name, returns, atrps):
        display(Markdown(f"#### {period_name}"))
        if returns.dropna().empty or atrps.dropna().empty:
            print("  - Not enough data to calculate.")
            return np.nan

        # Standard calculations
        mean_return = returns.mean()
        mean_atrp = atrps.mean()
        sharpe_atr = mean_return / mean_atrp if mean_atrp > 0 else np.nan

        # +++ ADDED: Detailed Mean Calculation Breakdown +++
        if debug:
            valid_returns = returns.dropna()
            num_returns = valid_returns.count()
            sum_returns = valid_returns.sum()
            manual_mean = sum_returns / num_returns if num_returns > 0 else 0
            print(f"  - (Debug) Number of valid daily returns: {num_returns}")
            print(f"  - (Debug) Sum of all daily returns:      {sum_returns:,.8f}")
            print(f"  - (Debug) Manually calculated mean:      {manual_mean:,.8f} (Sum / Count)")
        # +++ END OF ADDED DETAIL +++

        print(f"  - Mean Daily Return: {mean_return:,.6f}")
        print(f"  - Mean Daily ATRP:  {mean_atrp:,.6f}")
        print(f"  - Sharpe (ATR) = (Mean Return / Mean ATRP) = {sharpe_atr:,.4f}")
        return sharpe_atr

    # --- 4. Run Verification for Portfolio (No changes here) ---
    display(Markdown("### A. Group Portfolio Verification"))
    _calculate_and_print_metrics("Full Period", portfolio_return_series, portfolio_atrp_series)
    _calculate_and_print_metrics("Calculation Period", portfolio_return_series.loc[:actual_calc_end_date], portfolio_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", portfolio_return_series.loc[actual_calc_end_date:].iloc[1:], portfolio_atrp_series.loc[actual_calc_end_date:].iloc[1:])

    # --- 5. Run Verification for Benchmark (No changes here) ---
    display(Markdown(f"### B. Benchmark ({benchmark_ticker}) Verification"))
    _calculate_and_print_metrics("Full Period", benchmark_return_series, benchmark_atrp_series)
    _calculate_and_print_metrics("Calculation Period", benchmark_return_series.loc[:actual_calc_end_date], benchmark_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", benchmark_return_series.loc[actual_calc_end_date:].iloc[1:], benchmark_atrp_series.loc[actual_calc_end_date:].iloc[1:])



In [ ]:
# Assume 'df_ohlcv' is your loaded dataset
# You might need to regenerate features_df if it's not in your notebook's memory
print("--- Regenerating features for verification ---")
features_df = generate_features(df_ohlcv)

In [ ]:
tickers_to_verify = ['STIP', 'RCL']
start_date = '2025-08-13'
end_date = '2025-09-04'
benchmark_ticker = 'VOO'

In [ ]:
def verify_sharpe_atr_calculation_checked(df_ohlcv, features_df, tickers_to_verify, benchmark_ticker,
                                    start_date, calc_period, fwd_period,
                                    master_calendar_ticker='VOO', debug=False):
    """
    Verifies the Sharpe (ATR) calculations for a portfolio and benchmark.

    This function transparently recalculates the key components for Sharpe (ATR)
    and can optionally export the underlying source data for manual inspection.
    """
    display(Markdown(f"## Verification Report for Sharpe (ATR) Calculation"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))

    # --- 1. Determine Exact Period Dates (No changes here) ---
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    actual_start_date = master_trading_days[start_idx]
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    actual_calc_end_date = master_trading_days[calc_end_idx]
    actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
    display(Markdown(f"**Full Period:** `{actual_start_date.date()}` to `{actual_fwd_end_date.date()}`\n"
                    f"**Calc End Date:** `{actual_calc_end_date.date()}`"))

    # Original debug export block (can be kept or removed)
    if debug:
        # ... (original export code remains here) ...
        pass # Assuming original block is here

    # --- 2. Recreate Portfolio & Benchmark Series from Scratch ---
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    portfolio_prices_raw = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
    portfolio_prices_norm = portfolio_prices_raw.div(portfolio_prices_raw.bfill().iloc[0])
    portfolio_value_series = portfolio_prices_norm.mean(axis=1)
    portfolio_return_series = portfolio_value_series.pct_change()
    p_idx = pd.MultiIndex.from_product([tickers_to_verify, portfolio_return_series.index])
    p_atrp_df = features_df.loc[features_df.index.intersection(p_idx)]['ATRP'].unstack(level=0)
    portfolio_atrp_series = p_atrp_df.mean(axis=1)

###############################    
    # benchmark_return_series = df_close_full[benchmark_ticker].pct_change().loc[actual_start_date:actual_fwd_end_date]

    # 1. First, slice the raw prices for the desired date range.
    benchmark_prices_raw = df_close_full[benchmark_ticker].loc[actual_start_date:actual_fwd_end_date]
    # 2. Then, calculate the percentage change on the sliced data.
    benchmark_return_series = benchmark_prices_raw.pct_change()

###############################    
    benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[actual_start_date:actual_fwd_end_date]


    # +++ NEW: DETAILED RETURN CALCULATION TRACE (PORTFOLIO) +++
    if debug:
        display(Markdown("---"))
        display(Markdown("### 🐛 Detailed Portfolio Return Calculation Trace"))

        # Step 1: Show the raw prices being used
        print("\n[STEP 1] Raw Adjusted Close prices used for calculation.")
        print("Compare these values with your 'Adj Close' columns.")
        display(portfolio_prices_raw)

        # Step 2: Show the normalization base and the result
        normalization_base = portfolio_prices_raw.bfill().iloc[0]
        print("\n[STEP 2] Normalization base (first row of prices).")
        print("Each column in Step 1 is divided by this corresponding value.")
        display(pd.DataFrame(normalization_base).T)

        print("\n[STEP 2a] Normalized prices (Result of division).")
        print("Compare these values with your 'N Close' columns.")
        display(portfolio_prices_norm)

        # Step 3: Show the averaged portfolio value series
        print("\n[STEP 3] Averaged normalized portfolio value series.")
        print("This is the row-by-row average of the table in Step 2a.")
        print("Compare this series with your 'N_portf' column.")
        display(pd.DataFrame(portfolio_value_series, columns=['N_portf']))

        # Step 4: Show the final portfolio return series
        print("\n[STEP 4] Final portfolio daily return series (pct_change).")
        print("This is the percentage change of the series in Step 3.")
        print("Compare this series with your 'N_portf_rtn' column.")
        display(pd.DataFrame(portfolio_return_series, columns=['N_portf_rtn']))
        display(Markdown("---"))
    # +++ END OF NEW DEBUG BLOCK +++


    # --- 3. Define a Helper to Print Detailed Calculation Steps ---
    # MODIFIED to include more debug details inside
    def _calculate_and_print_metrics(period_name, returns, atrps):
        display(Markdown(f"#### {period_name}"))
        if returns.dropna().empty or atrps.dropna().empty:
            print("  - Not enough data to calculate.")
            return np.nan

        # Standard calculations
        mean_return = returns.mean()
        mean_atrp = atrps.mean()
        sharpe_atr = mean_return / mean_atrp if mean_atrp > 0 else np.nan

        # +++ ADDED: Detailed Mean Calculation Breakdown +++
        if debug:
            valid_returns = returns.dropna()
            num_returns = valid_returns.count()
            sum_returns = valid_returns.sum()
            manual_mean = sum_returns / num_returns if num_returns > 0 else 0
            print(f"  - (Debug) Number of valid daily returns: {num_returns}")
            print(f"  - (Debug) Sum of all daily returns:      {sum_returns:,.8f}")
            print(f"  - (Debug) Manually calculated mean:      {manual_mean:,.8f} (Sum / Count)")
        # +++ END OF ADDED DETAIL +++

        print(f"  - Mean Daily Return: {mean_return:,.6f}")
        print(f"  - Mean Daily ATRP:  {mean_atrp:,.6f}")
        print(f"  - Sharpe (ATR) = (Mean Return / Mean ATRP) = {sharpe_atr:,.4f}")
        return sharpe_atr

    # --- 4. Run Verification for Portfolio (No changes here) ---
    display(Markdown("### A. Group Portfolio Verification"))
    _calculate_and_print_metrics("Full Period", portfolio_return_series, portfolio_atrp_series)
    _calculate_and_print_metrics("Calculation Period", portfolio_return_series.loc[:actual_calc_end_date], portfolio_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", portfolio_return_series.loc[actual_calc_end_date:].iloc[1:], portfolio_atrp_series.loc[actual_calc_end_date:].iloc[1:])

    # --- 5. Run Verification for Benchmark (No changes here) ---
    display(Markdown(f"### B. Benchmark ({benchmark_ticker}) Verification"))
    _calculate_and_print_metrics("Full Period", benchmark_return_series, benchmark_atrp_series)
    _calculate_and_print_metrics("Calculation Period", benchmark_return_series.loc[:actual_calc_end_date], benchmark_atrp_series.loc[:actual_calc_end_date])
    _calculate_and_print_metrics("Forward Period", benchmark_return_series.loc[actual_calc_end_date:].iloc[1:], benchmark_atrp_series.loc[actual_calc_end_date:].iloc[1:])

In [ ]:
# Now call the verification function with the exact parameters from the UI
verify_sharpe_atr_calculation_checked(
    df_ohlcv=df_ohlcv,
    features_df=features_df,
    tickers_to_verify=tickers_to_verify, # <-- From the UI output
    benchmark_ticker=benchmark_ticker,
    start_date=start_date,
    calc_period=10,
    fwd_period=5,
    debug=True,
)

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import pprint
from IPython.display import display, Markdown
import os # Make sure os is imported for the export function later


# def calculate_gain(price_series: pd.Series):
#     """Calculates the total gain over a series of prices."""
#     # Ensure there are at least two data points to calculate a gain
#     if price_series.dropna().shape[0] < 2: return np.nan
#     # Use forward-fill for the end price and back-fill for the start price
#     # to handle potential NaNs at the beginning or end of the series.
#     return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

# def calculate_sharpe(return_series: pd.Series):
#     """Calculates the annualized Sharpe ratio from a series of daily returns."""
#     # Ensure there are at least two returns to calculate a standard deviation
#     if return_series.dropna().shape[0] < 2: return np.nan
#     std_dev = return_series.std()
#     # Avoid division by zero if returns are constant
#     if std_dev > 0 and std_dev != np.inf:
#         return (return_series.mean() / std_dev) * np.sqrt(252)
#     return np.nan

# def calculate_sharpe_atr(return_series: pd.Series, atrp_series: pd.Series):
#     """Calculates a Sharpe-like ratio using mean ATRP as the denominator."""
#     # Ensure there are returns and that ATRP data is valid
#     if return_series.dropna().shape[0] < 2 or atrp_series.dropna().empty:
#         return np.nan
        
#     mean_return = return_series.mean()
#     mean_atrp = atrp_series.mean()
    
#     # Avoid division by zero
#     if mean_atrp > 0 and mean_atrp != np.inf:
#         return mean_return / mean_atrp
        
#     return np.nan


# # --- B. MODULAR METRIC CALCULATION ENGINE ---

# def calculate_price_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the 'Price' metric (total gain) over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'calc_close' (pd.DataFrame): The close prices for the calc period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     calc_close = metric_data['calc_close']
    
#     # Ensure there are at least two data points to calculate a gain
#     if len(calc_close) < 2:
#         return pd.Series(dtype='float64', index=calc_close.columns)

#     first_prices = calc_close.bfill().iloc[0]
#     last_prices = calc_close.ffill().iloc[-1]
    
#     # The division of two Series aligns by index (Ticker), which is what we want.
#     price_metric = last_prices / first_prices
    
#     return price_metric.dropna()

# def calculate_sharpe_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the annualized 'Sharpe' ratio metric over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     daily_returns = metric_data['daily_returns']
    
#     # Ensure there's enough data to calculate standard deviation
#     if len(daily_returns.dropna()) < 2:
#         return pd.Series(dtype='float64', index=daily_returns.columns)
    
#     mean_returns = daily_returns.mean()
#     std_returns = daily_returns.std()
    
#     # Standard annualized Sharpe Ratio calculation. Avoid division by zero.
#     # We replace resulting NaNs/infs with 0 to handle cases of zero volatility.
#     sharpe_ratio = (mean_returns / std_returns * np.sqrt(252))
    
#     return sharpe_ratio.replace([np.inf, -np.inf], np.nan).fillna(0)
    
# def calculate_sharpe_atr_metric(metric_data: dict) -> pd.Series:
#     """
#     Calculates the 'Sharpe (ATR)' metric over the calculation period.

#     Args:
#         metric_data: A dictionary containing pre-calculated data Series.
#                      Requires 'daily_returns' (pd.DataFrame): Daily returns for the calc period.
#                      Requires 'atrp' (pd.Series): Mean ATRP for each ticker over the period.

#     Returns:
#         A pandas Series of the calculated metric values, indexed by Ticker.
#     """
#     daily_returns = metric_data['daily_returns']
#     atrp = metric_data['atrp']
    
#     mean_returns = daily_returns.mean()
    
#     # ATRP-based Sharpe. Avoid division by zero.
#     # We replace resulting NaNs/infs with 0.
#     sharpe_atr = mean_returns / atrp
    
#     return sharpe_atr.replace([np.inf, -np.inf], np.nan).fillna(0)


# METRIC_REGISTRY = {
#     'Price': calculate_price_metric,
#     'Sharpe': calculate_sharpe_metric,
#     'Sharpe (ATR)': calculate_sharpe_atr_metric,
# }


def plot_walk_forward_analyzer(df_ohlcv,
                               default_start_date=None, default_calc_period=126, default_fwd_period=63,
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
                               quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
                               debug=False):
    # (No changes to the initial setup part of this function...)
    print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")
    print("--- Generating all features upfront... ---")
    features_df = generate_features(df_ohlcv)
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
    fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
    if default_metric not in METRIC_REGISTRY:
        fallback_metric = list(METRIC_REGISTRY.keys())[0]
        print(f"⚠️ Warning: Default metric '{default_metric}' not in registry. Using '{fallback_metric}'.")
        default_metric = fallback_metric
    metric_dropdown = widgets.Dropdown(options=list(METRIC_REGISTRY.keys()), value=default_metric, description='Metric:')
    rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
    rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container, debug_data_container = [None], [None]
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    def update_plot(button_click):
        ticker_list_output.clear_output()
        start_date_raw = pd.to_datetime(start_date_picker.value)
        start_date_idx = master_trading_days.searchsorted(start_date_raw)
        if start_date_idx >= len(master_trading_days):
            with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
        actual_start_date = master_trading_days[start_date_idx]
        with ticker_list_output:
            if start_date_raw.date() != actual_start_date.date():
                print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")
        calc_period = calc_period_input.value; fwd_period = fwd_period_input.value; metric = metric_dropdown.value
        rank_start = rank_start_input.value; rank_end = rank_end_input.value; benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
        if rank_start < 1 or calc_period < 2 or fwd_period < 1:
            with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return
        required_days = calc_period + fwd_period
        if start_date_idx + required_days > len(master_trading_days):
            available_days = len(master_trading_days) - start_date_idx; last_available_date = master_trading_days[-1].date()
            with ticker_list_output:
                print(f"Error: Not enough data for the requested period.\n  Start Date: {actual_start_date.date()}\n  Required Days: {calc_period} (calc) + {fwd_period} (fwd) = {required_days}\n  Available Days from Start: {available_days} (until {last_available_date})\n  Please shorten the 'Calc Period' / 'Fwd Period' or choose an earlier 'Start Date'.")
            return
        eligible_tickers = get_eligible_universe(features_df, actual_start_date, quality_thresholds)
        if not eligible_tickers:
            with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        results, debug_output = run_walk_forward_step(df_close_full, df_high_full, df_low_full, master_trading_days, actual_start_date, calc_period, fwd_period, metric, rank_start, rank_end, benchmark_ticker, features_df=features_df, debug=debug)
        if results.get('error'):
            with ticker_list_output: print(f"Error: {results['error']}"); return
        period_dates = {'calc_period_start': results['safe_start_date'], 'calc_period_end': results['actual_calc_end_ts'], 'forward_period_start': results['actual_calc_end_ts'], 'forward_period_end': results['safe_viz_end_date']}
        run_parameters = {'calc_period': calc_period, 'fwd_period': fwd_period, 'rank_metric': metric, 'rank_start': rank_start, 'rank_end': rank_end, 'benchmark_ticker': benchmark_ticker}
        results.update(period_dates); results.update(run_parameters)
        if debug_output is not None and isinstance(debug_output, dict):
            debug_output.update(period_dates); debug_output.update(run_parameters)
        with fig.batch_update():
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
                    trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark.values, f"Benchmark ({benchmark_ticker})", True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
        results_container[0] = results; debug_data_container[0] = debug_output
        with ticker_list_output:
            print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            pprint.pprint(results['tickers_to_display'])
            p = results['performance_data']
            
            # --- START OF MODIFIED BLOCK ---
            rows = []
            # Gain Metrics
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p.get('full_b_gain')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': '== Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            
            # Standard Sharpe Metrics
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p.get('full_b_sharpe')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': '== Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})

            # Sharpe (ATR) Metrics
            rows.append({'Metric': 'Group Portfolio Sharpe (ATR)', 'Full': p['full_p_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr']})
            if not np.isnan(p.get('full_b_sharpe_atr')):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe (ATR)', 'Full': p['full_b_sharpe_atr'], 'Calc': p['calc_b_sharpe_atr'], 'Fwd': p['fwd_b_sharpe_atr']})
                rows.append({'Metric': '== Sharpe (ATR) Delta (vs Bm)', 'Full': p['full_p_sharpe_atr'] - p['full_b_sharpe_atr'], 'Calc': p['calc_p_sharpe_atr'] - p['calc_b_sharpe_atr'], 'Fwd': p['fwd_p_sharpe_atr'] - p['fwd_b_sharpe_atr']})

            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row] # This now correctly includes both types of Sharpe
            # --- END OF MODIFIED BLOCK ---
            
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None)
    return (results_container, debug_data_container)



# def generate_features(df_ohlcv: pd.DataFrame, 
#                       atr_period: int = 14, 
#                       quality_window: int = 252, 
#                       quality_min_periods: int = 126) -> pd.DataFrame:
#     """
#     Generates a comprehensive DataFrame of derived features from raw OHLCV data.

#     This function performs all heavy, window-based calculations upfront to be used
#     by downstream analysis functions. It calculates:
#     1. Technical Indicators: True Range (TR), ATR, and ATRP.
#     2. Data Quality Metrics: Rolling stale percentage, median dollar volume, etc.

#     Args:
#         df_ohlcv: The primary DataFrame with a (Ticker, Date) MultiIndex and
#                   columns for 'Adj High', 'Adj Low', 'Adj Close', 'Volume'.
#         atr_period: The lookback period for the ATR's Exponential Moving Average.
#         quality_window: The rolling window size for data quality metrics.
#         quality_min_periods: The minimum number of observations required to have
#                              a valid quality metric.

#     Returns:
#         A new DataFrame with the same (Ticker, Date) MultiIndex containing all
#         calculated feature columns.
#     """
#     print("--- Starting Feature Generation ---")
    
#     # Ensure the DataFrame is sorted for correct window and shift operations
#     # FIX: Replaced is_lexsorted() with the current pandas attribute
#     if not df_ohlcv.index.is_monotonic_increasing:
#         print("Sorting index for calculation accuracy...")
#         df_ohlcv = df_ohlcv.sort_index()

#     # --- 1. Technical Indicator Calculation (TR, ATR, ATRP) ---
#     print(f"Calculating technical indicators (ATR Period: {atr_period})...")
    
#     # Group by ticker to handle each security independently
#     grouped = df_ohlcv.groupby(level='Ticker')
    
#     # Get the previous day's close required for True Range
#     prev_close = grouped['Adj Close'].shift(1)
    
#     # Calculate the three components of True Range
#     high_low = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
#     high_prev_close = abs(df_ohlcv['Adj High'] - prev_close)
#     low_prev_close = abs(df_ohlcv['Adj Low'] - prev_close)
    
#     # Combine the components to get the final TR
#     tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1, skipna=False)
    
#     # # Calculate the ATR using an Exponential Moving Average
#     # --- FIX IS HERE ---
#     # Use .transform() to apply the EWM function. 
#     # This guarantees the resulting Series has the exact same index as 'tr',
#     # preventing the index alignment error during the subsequent division.
#     atr = tr.groupby(level='Ticker').transform(
#         lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean()
#     )

#     # --- CHANGE 1: Removed .fillna(0) ---
#     # ATRP will now be NaN on the first day, consistent with TR and ATR.
#     atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)

#     indicator_df = pd.DataFrame({
#         'TR': tr,
#         'ATR': atr,
#         'ATRP': atrp
#     })
    
#     # --- 2. Data Quality Metric Calculation ---
#     print(f"Calculating data quality metrics (Window: {quality_window} days)...")
    
#     # Create intermediate flags needed for quality calculations
#     is_stale = np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0)
#     dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']
#     has_same_volume = (grouped['Volume'].diff() == 0).astype(int)
    
#     # Combine flags into a temporary DataFrame for rolling calculations
#     quality_temp_df = pd.DataFrame({
#         'IsStale': is_stale,
#         'DollarVolume': dollar_volume,
#         'HasSameVolume': has_same_volume
#     }, index=df_ohlcv.index) # Explicitly set index to be safe
    
#     # Perform the rolling calculations on the grouped data
#     # --- FIX IS HERE ---
#     # We switch to the older, more compatible dictionary-based aggregation method.
#     # This syntax is understood by nearly all versions of pandas.
#     rolling_result = quality_temp_df.groupby(level='Ticker').rolling(
#         window=quality_window,
#         min_periods=quality_min_periods
#     ).agg({
#         'IsStale': 'mean',
#         'DollarVolume': 'median',
#         'HasSameVolume': 'sum'
#     })
    
#     # The dictionary syntax produces columns with the original names ('IsStale', etc.).
#     # We now explicitly rename them to our desired final names.
#     rolling_result = rolling_result.rename(columns={
#         'IsStale': 'RollingStalePct',
#         'DollarVolume': 'RollMedDollarVol', # <-- RENAMED HERE
#         'HasSameVolume': 'RollingSameVolCount'
#     })

#     # The index after a grouped rolling operation is hierarchical.
#     # We remove the outermost 'Ticker' level to restore the original index structure.
#     rolling_quality = rolling_result.reset_index(level=0, drop=True)

#     # --- 3. Combine All Features ---
#     print("Combining all feature sets...")
#     features_df = pd.concat([indicator_df, rolling_quality], axis=1)
    
#     print("✅ Feature generation complete.")
#     return features_df


# def get_eligible_universe(features_df, filter_date, thresholds):
#     """Filters the universe of tickers based on quality metrics for a given date."""
#     filter_date_ts = pd.to_datetime(filter_date)
#     # The index is now the comprehensive features_df
#     date_index = features_df.index.get_level_values('Date').unique().sort_values()
    
#     if filter_date_ts < date_index[0]:
#         print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
#         return []
        
#     # Find the most recent date with quality data on or before the filter date
#     valid_prior_dates = date_index[date_index <= filter_date_ts]
#     if valid_prior_dates.empty:
#         print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
#         return []
        
#     actual_date_to_use = valid_prior_dates[-1]
#     if actual_date_to_use.date() != filter_date_ts.date():
#         print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")

#     metrics_on_date = features_df.xs(actual_date_to_use, level='Date')
    
#     # Apply filters using the new column names from features_df
#     mask = ((metrics_on_date['RollMedDollarVol'] >= thresholds['min_median_dollar_volume']) & # <-- RENAMED
#             (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
#             (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
            
#     eligible_tickers = metrics_on_date[mask].index.tolist()
#     all_tickers = metrics_on_date.index.tolist()
#     print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
#     return eligible_tickers


# def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
#                           master_trading_days,
#                           start_date, calc_period, fwd_period,
#                           metric, rank_start, rank_end, benchmark_ticker,
#                           features_df,
#                           debug=False):
#     """
#     Runs a single step of the walk-forward analysis with a strict, pre-emptive
#     check to ensure the full period is available.
#     """
#     debug_data = {} if debug else None

#     # 1. Determine exact date ranges with a NEW pre-emptive check
#     try:
#         start_idx = master_trading_days.get_loc(start_date)
#     except KeyError:
#         return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)

#     # +++ THIS IS THE NEW PRE-EMPTIVE CHECK LOGIC +++
#     # Calculate the desired end index without clamping first.
#     desired_viz_end_idx = start_idx + calc_period + fwd_period
    
#     # Check if the desired end index is out of bounds.
#     if desired_viz_end_idx >= len(master_trading_days):
#         last_available_date = master_trading_days[-1].date()
#         required_days = calc_period + fwd_period
#         available_days = len(master_trading_days) - start_idx
#         error_msg = (f"Not enough data for the full requested period. "
#                      f"Required: {required_days} days, Available: {available_days} days until {last_available_date}.")
#         return ({'error': error_msg}, None)
#     # --- END OF NEW CHECK ---

#     # If the check passes, we know the full period is available.
#     # The 'min' calls are now just a redundant safety measure.
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

#     safe_start_date = master_trading_days[start_idx]
#     safe_calc_end_date = master_trading_days[calc_end_idx]
#     safe_viz_end_date = master_trading_days[viz_end_idx]
    
#     if safe_start_date >= safe_calc_end_date:
#         return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

#     # (The rest of the function remains completely unchanged...)
#     # 2. Slice data for the calculation period
#     calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
#     calc_close = calc_close_raw.dropna(axis=1, how='all')
#     if calc_close.shape[1] == 0 or len(calc_close) < 2:
#         return ({'error': "Not enough data in calc period."}, None)

#     # 3. Calculate INTERMEDIATE data required for the ranking metrics
#     daily_returns = calc_close.bfill().ffill().pct_change()
#     valid_tickers = calc_close.columns
#     calc_period_index = pd.MultiIndex.from_product([valid_tickers, calc_close.index], names=['Ticker', 'Date'])
#     features_in_period = features_df.loc[features_df.index.intersection(calc_period_index)]
#     atrp = features_in_period.groupby(level='Ticker')['ATRP'].mean()

#     # 4. Calculate all ranking metrics by iterating through the METRIC_REGISTRY
#     metric_ingredients = { 'calc_close': calc_close, 'daily_returns': daily_returns, 'atrp': atrp, }
#     metric_values = {}
#     for name, func in METRIC_REGISTRY.items():
#         metric_values[name] = func(metric_ingredients)
#     if metric not in metric_values or metric_values[metric].empty:
#         return ({'error': f"Metric '{metric}' could not be calculated or resulted in no valid tickers."}, None)

#     # 5. Rank tickers and select the portfolio
#     sorted_tickers = metric_values[metric].sort_values(ascending=False)
#     tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
#     if not tickers_to_display:
#         return ({'error': "No tickers found for the selected rank."}, None)

#     # 6. Calculate Portfolio & Benchmark Performance
#     normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
#     normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
#     actual_calc_end_ts = calc_close.index.max()
#     portfolio_series = normalized_plot_data.mean(axis=1)
#     portfolio_return_series = portfolio_series.pct_change()
#     benchmark_price_series = df_close_full.get(benchmark_ticker)
#     benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')
#     try:
#         boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
#         calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
#         fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
#         if benchmark_price_series is not None:
#             bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
#             calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
#             fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     except (KeyError, IndexError):
#         calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
#         fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         if benchmark_price_series is not None:
#             calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
#             fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]
#         else:
#             calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
#     full_period_index = pd.MultiIndex.from_product([tickers_to_display, portfolio_return_series.index], names=['Ticker', 'Date'])
#     portfolio_atrp_features = features_df.loc[features_df.index.intersection(full_period_index)]
#     portfolio_atrp_daily_unstacked = portfolio_atrp_features['ATRP'].unstack(level='Ticker')
#     portfolio_atrp_series = portfolio_atrp_daily_unstacked.mean(axis=1)
#     if benchmark_ticker in df_close_full.columns:
#         benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[safe_start_date:safe_viz_end_date]
#     else:
#         benchmark_atrp_series = pd.Series(dtype='float64')
#     calc_portfolio_atrp = portfolio_atrp_series.loc[:actual_calc_end_ts]
#     fwd_portfolio_atrp = portfolio_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     calc_benchmark_atrp = benchmark_atrp_series.loc[:actual_calc_end_ts]
#     fwd_benchmark_atrp = benchmark_atrp_series.loc[actual_calc_end_ts:].iloc[1:]
#     perf_data = {}
#     perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
#     perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
#     perf_data['full_p_gain'] = calculate_gain(portfolio_series)
#     perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
#     perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
#     perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
#     perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
#     perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
#     perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
#     perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
#     perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)
#     perf_data['calc_p_sharpe_atr'] = calculate_sharpe_atr(calc_portfolio_returns, calc_portfolio_atrp)
#     perf_data['fwd_p_sharpe_atr'] = calculate_sharpe_atr(fwd_portfolio_returns, fwd_portfolio_atrp)
#     perf_data['full_p_sharpe_atr'] = calculate_sharpe_atr(portfolio_return_series, portfolio_atrp_series)
#     perf_data['calc_b_sharpe_atr'] = calculate_sharpe_atr(calc_benchmark_returns, calc_benchmark_atrp)
#     perf_data['fwd_b_sharpe_atr'] = calculate_sharpe_atr(fwd_benchmark_returns, fwd_benchmark_atrp)
#     perf_data['full_b_sharpe_atr'] = calculate_sharpe_atr(benchmark_return_series, benchmark_atrp_series)
#     if debug:
#         df_ranking_base = pd.DataFrame({'MeanDailyReturn': daily_returns.mean(),'StdDevDailyReturn': daily_returns.std(),'MeanATRP': atrp})
#         df_metrics = pd.DataFrame(metric_values)
#         df_metrics.columns = [f'Metric_{col}' for col in df_metrics.columns]
#         df_ranking = df_ranking_base.join(df_metrics, how='left')
#         df_ranking.index.name = 'Ticker'
#         debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)
#     calc_end_prices = calc_close.ffill().iloc[-1]
#     fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
#     viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
#     calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
#     fwd_gains = (viz_end_prices / calc_end_prices) - 1
#     results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
#     if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
#         benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
#         results_df = pd.concat([results_df, benchmark_df_row])
#     if debug:
#         df_trace = normalized_plot_data.copy()
#         df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
#         df_trace['Norm_Price_Portfolio'] = portfolio_series
#         if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
#             norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
#             df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
#         for col in df_trace.columns:
#             if 'Norm_Price' in col:
#                 df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
#         debug_data['portfolio_trace'] = df_trace
#     final_results = {
#         'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
#         'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
#         'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
#         'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
#         'error': None
#     }
#     return (final_results, debug_data)


# def verify_sharpe_atr_calculation_checked(df_ohlcv, features_df, tickers_to_verify, benchmark_ticker,
#                                     start_date, calc_period, fwd_period,
#                                     master_calendar_ticker='VOO', debug=False):
#     """
#     Verifies the Sharpe (ATR) calculations for a portfolio and benchmark.

#     This function transparently recalculates the key components for Sharpe (ATR)
#     and can optionally export the underlying source data for manual inspection.
#     """
#     display(Markdown(f"## Verification Report for Sharpe (ATR) Calculation"))
#     display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))

#     # --- 1. Determine Exact Period Dates (No changes here) ---
#     master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
#     start_date_raw = pd.to_datetime(start_date)
#     start_idx = master_trading_days.searchsorted(start_date_raw)
#     actual_start_date = master_trading_days[start_idx]
#     calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
#     fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
#     actual_calc_end_date = master_trading_days[calc_end_idx]
#     actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
#     display(Markdown(f"**Full Period:** `{actual_start_date.date()}` to `{actual_fwd_end_date.date()}`\n"
#                     f"**Calc End Date:** `{actual_calc_end_date.date()}`"))

#     # Original debug export block (can be kept or removed)
#     if debug:
#         # ... (original export code remains here) ...
#         pass # Assuming original block is here

#     # --- 2. Recreate Portfolio & Benchmark Series from Scratch ---
#     df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
#     portfolio_prices_raw = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
#     portfolio_prices_norm = portfolio_prices_raw.div(portfolio_prices_raw.bfill().iloc[0])
#     portfolio_value_series = portfolio_prices_norm.mean(axis=1)
#     portfolio_return_series = portfolio_value_series.pct_change()
#     p_idx = pd.MultiIndex.from_product([tickers_to_verify, portfolio_return_series.index])
#     p_atrp_df = features_df.loc[features_df.index.intersection(p_idx)]['ATRP'].unstack(level=0)
#     portfolio_atrp_series = p_atrp_df.mean(axis=1)

# ###############################    
#     # benchmark_return_series = df_close_full[benchmark_ticker].pct_change().loc[actual_start_date:actual_fwd_end_date]

#     # 1. First, slice the raw prices for the desired date range.
#     benchmark_prices_raw = df_close_full[benchmark_ticker].loc[actual_start_date:actual_fwd_end_date]
#     # 2. Then, calculate the percentage change on the sliced data.
#     benchmark_return_series = benchmark_prices_raw.pct_change()

# ###############################    
#     benchmark_atrp_series = features_df.xs(benchmark_ticker, level='Ticker')['ATRP'].loc[actual_start_date:actual_fwd_end_date]


#     # +++ NEW: DETAILED RETURN CALCULATION TRACE (PORTFOLIO) +++
#     if debug:
#         display(Markdown("---"))
#         display(Markdown("### 🐛 Detailed Portfolio Return Calculation Trace"))

#         # Step 1: Show the raw prices being used
#         print("\n[STEP 1] Raw Adjusted Close prices used for calculation.")
#         print("Compare these values with your 'Adj Close' columns.")
#         display(portfolio_prices_raw)

#         # Step 2: Show the normalization base and the result
#         normalization_base = portfolio_prices_raw.bfill().iloc[0]
#         print("\n[STEP 2] Normalization base (first row of prices).")
#         print("Each column in Step 1 is divided by this corresponding value.")
#         display(pd.DataFrame(normalization_base).T)

#         print("\n[STEP 2a] Normalized prices (Result of division).")
#         print("Compare these values with your 'N Close' columns.")
#         display(portfolio_prices_norm)

#         # Step 3: Show the averaged portfolio value series
#         print("\n[STEP 3] Averaged normalized portfolio value series.")
#         print("This is the row-by-row average of the table in Step 2a.")
#         print("Compare this series with your 'N_portf' column.")
#         display(pd.DataFrame(portfolio_value_series, columns=['N_portf']))

#         # Step 4: Show the final portfolio return series
#         print("\n[STEP 4] Final portfolio daily return series (pct_change).")
#         print("This is the percentage change of the series in Step 3.")
#         print("Compare this series with your 'N_portf_rtn' column.")
#         display(pd.DataFrame(portfolio_return_series, columns=['N_portf_rtn']))
#         display(Markdown("---"))
#     # +++ END OF NEW DEBUG BLOCK +++


#     # --- 3. Define a Helper to Print Detailed Calculation Steps ---
#     # MODIFIED to include more debug details inside
#     def _calculate_and_print_metrics(period_name, returns, atrps):
#         display(Markdown(f"#### {period_name}"))
#         if returns.dropna().empty or atrps.dropna().empty:
#             print("  - Not enough data to calculate.")
#             return np.nan

#         # Standard calculations
#         mean_return = returns.mean()
#         mean_atrp = atrps.mean()
#         sharpe_atr = mean_return / mean_atrp if mean_atrp > 0 else np.nan

#         # +++ ADDED: Detailed Mean Calculation Breakdown +++
#         if debug:
#             valid_returns = returns.dropna()
#             num_returns = valid_returns.count()
#             sum_returns = valid_returns.sum()
#             manual_mean = sum_returns / num_returns if num_returns > 0 else 0
#             print(f"  - (Debug) Number of valid daily returns: {num_returns}")
#             print(f"  - (Debug) Sum of all daily returns:      {sum_returns:,.8f}")
#             print(f"  - (Debug) Manually calculated mean:      {manual_mean:,.8f} (Sum / Count)")
#         # +++ END OF ADDED DETAIL +++

#         print(f"  - Mean Daily Return: {mean_return:,.6f}")
#         print(f"  - Mean Daily ATRP:  {mean_atrp:,.6f}")
#         print(f"  - Sharpe (ATR) = (Mean Return / Mean ATRP) = {sharpe_atr:,.4f}")
#         return sharpe_atr

#     # --- 4. Run Verification for Portfolio (No changes here) ---
#     display(Markdown("### A. Group Portfolio Verification"))
#     _calculate_and_print_metrics("Full Period", portfolio_return_series, portfolio_atrp_series)
#     _calculate_and_print_metrics("Calculation Period", portfolio_return_series.loc[:actual_calc_end_date], portfolio_atrp_series.loc[:actual_calc_end_date])
#     _calculate_and_print_metrics("Forward Period", portfolio_return_series.loc[actual_calc_end_date:].iloc[1:], portfolio_atrp_series.loc[actual_calc_end_date:].iloc[1:])

#     # --- 5. Run Verification for Benchmark (No changes here) ---
#     display(Markdown(f"### B. Benchmark ({benchmark_ticker}) Verification"))
#     _calculate_and_print_metrics("Full Period", benchmark_return_series, benchmark_atrp_series)
#     _calculate_and_print_metrics("Calculation Period", benchmark_return_series.loc[:actual_calc_end_date], benchmark_atrp_series.loc[:actual_calc_end_date])
#     _calculate_and_print_metrics("Forward Period", benchmark_return_series.loc[actual_calc_end_date:].iloc[1:], benchmark_atrp_series.loc[actual_calc_end_date:].iloc[1:])



In [ ]:
results_container, debug_container = plot_walk_forward_analyzer(
    df_ohlcv=df_ohlcv,
    default_start_date='2018-10-03',
    default_calc_period=252,
    default_fwd_period=63,
    default_metric='Sharpe (ATR)',
    default_rank_start=1,
    default_rank_end=5,
    default_benchmark_ticker='VOO',
    quality_thresholds={ 'min_median_dollar_volume': 10_000_000, 
                         'max_stale_pct': 0.05, 
                         'max_same_vol_count': 1 },
    debug=True  # <-- Activate the new mode!
)

In [ ]:
features_df.info()

In [ ]:
# 2. Use pd.IndexSlice for a clean and readable slice
# This is the recommended pandas method for complex slicing on a MultiIndex.
idx = pd.IndexSlice
sliced_features_df = features_df.loc[idx[tickers_to_verify, start_date:end_date], :]

# 3. Display the results to verify
print(f"Shape of the original features_df: {features_df.shape}")
print(f"Shape of the sliced_features_df: {sliced_features_df.shape}")
print("\n--- Displaying the sliced DataFrame ---")
display(sliced_features_df)

# You can also verify the boundaries of the slice
print("\n--- Verifying the boundaries ---")
print(f"First Ticker/Date: {sliced_features_df.index.min()}")
print(f"Last Ticker/Date:  {sliced_features_df.index.max()}")


In [ ]:
# # 1. Define the tickers and date range for your slice
# tickers_to_slice = ['BIL', 'SATS', 'MINT', 'BOXX']
# start_date = '2025-08-03'
# end_date = '2025-10-02'

# 2. Use pd.IndexSlice for a clean and readable slice
# This is the recommended pandas method for complex slicing on a MultiIndex.
idx = pd.IndexSlice
sliced_df_ohlcv = df_ohlcv.loc[idx[tickers_to_verify, start_date:end_date], :]

# 3. Display the results to verify
print(f"Shape of the original df_ohlcv: {df_ohlcv.shape}")
print(f"Shape of the sliced_df_ohlcv: {sliced_df_ohlcv.shape}")
print("\n--- Displaying the sliced DataFrame ---")
display(sliced_df_ohlcv)

# You can also verify the boundaries of the slice
print("\n--- Verifying the boundaries ---")
print(f"First Ticker/Date: {sliced_df_ohlcv.index.min()}")
print(f"Last Ticker/Date:  {sliced_df_ohlcv.index.max()}")


In [ ]:
# 2. Use .loc to slice AND explicitly create a .copy()
# This tells pandas that we are creating a new DataFrame that we intend to modify.
# This is the line that prevents the warning.
idx = pd.IndexSlice
sliced_df_ohlcv = df_ohlcv.loc[idx[tickers_to_verify, start_date:end_date], :].copy()

# 3. Now, add the new column to this independent copy.
# This will no longer raise a warning.
sliced_df_ohlcv['Daily_Return'] = sliced_df_ohlcv.groupby(level='Ticker')['Adj Close'].pct_change()

# 4. Display the results to verify
print("--- DataFrame with 'Daily_Return' column added (Warning Corrected) ---")
display(sliced_df_ohlcv.head())
print("...")
display(sliced_df_ohlcv.tail())

# print("\n--- Updated DataFrame Info ---")
# sliced_df_ohlcv.info()

In [ ]:
# Group by the 'Ticker' level of the index, select the 'Daily_Return' column,
# and calculate the mean for each group.
mean_daily_returns = sliced_df_ohlcv.groupby(level='Ticker')['Daily_Return'].mean()

# Display the resulting Series
print("--- Mean Daily Return per Ticker ---")
display(mean_daily_returns)

In [ ]:
# Group by the 'Ticker' level of the index, select the 'Daily_Return' column,
# and calculate the mean for each group.
mean_ATRP = sliced_features_df.groupby(level='Ticker')['ATRP'].mean()

# Display the resulting Series
print("--- ATRP per Ticker ---")
display(mean_ATRP)

In [ ]:
display(mean_daily_returns / mean_ATRP)

In [ ]:
sliced_features_df.info()

In [ ]:
sliced_df_ohlcv.info()

| Date | Portfolio Value | Portfolio Daily_Return | Ticker A ATRP | Ticker B ATRP | Portfolio Daily ATRP |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Day 1 | 1.000 | **`NaN`** | 0.005 | 0.005 | 0.005 |
| Day 2 | 1.010 | `+0.010` | 0.006 | 0.006 | 0.006 |
| Day 3 | 1.015 | `+0.005` | 0.004 | 0.004 | 0.004 |

| Function                                     | Is the "Clamp" Logic Used? | Why? (Its Role in this Function)                                                                                                                                                                                                                                                                                                                             |
| -------------------------------------------- | -------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **`run_walk_forward_step`**                  | **Yes (Primary)**          | This is the **source of truth**. As the core calculation engine, it *must* be robust and handle running out of data at the end of the timeline without crashing. This is the most critical implementation of the clamp.                                                                                                                                       |
| **`verify_sharpe_atr_calculation`**          | **Yes (Replication)**      | As you noted, it's here. Its purpose is to **exactly mimic** the behavior of `run_walk_forward_step`. If the verification tool crashed while the main tool gracefully clamped the period, the verification tool would be useless for debugging end-of-timeline scenarios.                                                                                         |
| **`verify_group_tickers_walk_forward_calculation`** | **Yes (Replication)**      | Same reason as above. This function verifies portfolio performance and must use the identical date boundary logic as the main engine to produce comparable results for gain and Sharpe ratio.                                                                                                                                                                  |
| **`verify_ticker_ranking_metrics`**          | **Yes (Replication)**      | Same reason. It verifies the metrics calculated over the `calc_period`. If the user selects a start date where a full `calc_period` is not available, this tool must clamp the period in the same way `run_walk_forward_step` does to verify the resulting (shorter) calculation.                                                                              |

| Function | How its Behavior Would Change | Pro | Con |
| :--- | :--- | :--- | :--- |
| **`run_walk_forward_step` (Core Engine)** | It would now return an error for the final, shorter periods instead of clamping and calculating them. It becomes "all or nothing." | The function's behavior is now stricter and more predictable. The results it returns are never from "partial" periods. | The function is less flexible; it can no longer handle end-of-timeline scenarios gracefully on its own. |
| **`plot_walk_forward_analyzer` (UI)** | The user experience would be nearly identical, because it *already has* this pre-emptive check. The engine's check would just be a redundant confirmation. | The engine's behavior now perfectly matches the UI's pre-emptive check, removing any potential for divergence. | None, really. This is a positive change from the UI's perspective. |
| **`run_full_backtest` & `run_strategy_search` (Automation)** | **This is the most significant impact.** When the backtest reaches the end of the data, `run_walk_forward_step` will return an error. The backtester's loop will then **skip this final, incomplete period entirely.** | The final aggregated equity curve and performance statistics are "purer." Every single point comes from a full-length forward period. | The backtest **throws away the last few days/weeks of data** because they don't form a complete forward period. The resulting equity curve is shorter. |

In [ ]:
# CHECK Metric_Sharpe (ATR) for Calc. Period has error.
- start date 2018-10-03
- Calc Period 252
- Fwd Period 63
- Metric Sharpe ATR
- Rank Start 1
- Rank End 5
-- Analysis Period 2018-10-03 to 2020-01-06 (this is full period, calc + fwd)
-- [BIL, MINT, SHV, BNDX, VCSH]
-- Metric_Sharpe (ATR) for BIL, MINT, SHV, BNDX matched my own calculation at bottom cell
-- Metric Sharpe (ATR) for VCSH is a bit off (code calc 0.194195, my calc 0.196112)

### Refactor Phase 1: Consolidated and Verified Feature Generation Engine

### Start of Code for Refactoring Phase 1

In [ ]:
import pandas as pd

data_path = r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet'
df_ohlcv = pd.read_parquet(data_path, engine='pyarrow')
df_ohlcv

In [ ]:
import pandas as pd
import numpy as np

def generate_features(df_ohlcv: pd.DataFrame, 
                      atr_period: int = 14, 
                      quality_window: int = 252, 
                      quality_min_periods: int = 126) -> pd.DataFrame:
    """
    Generates a comprehensive DataFrame of derived features from raw OHLCV data.

    This function performs all heavy, window-based calculations upfront to be used
    by downstream analysis functions. It calculates:
    1. Technical Indicators: True Range (TR), ATR, and ATRP.
    2. Data Quality Metrics: Rolling stale percentage, median dollar volume, etc.

    Args:
        df_ohlcv: The primary DataFrame with a (Ticker, Date) MultiIndex and
                  columns for 'Adj High', 'Adj Low', 'Adj Close', 'Volume'.
        atr_period: The lookback period for the ATR's Exponential Moving Average.
        quality_window: The rolling window size for data quality metrics.
        quality_min_periods: The minimum number of observations required to have
                             a valid quality metric.

    Returns:
        A new DataFrame with the same (Ticker, Date) MultiIndex containing all
        calculated feature columns.
    """
    print("--- Starting Feature Generation ---")
    
    # Ensure the DataFrame is sorted for correct window and shift operations
    # FIX: Replaced is_lexsorted() with the current pandas attribute
    if not df_ohlcv.index.is_monotonic_increasing:
        print("Sorting index for calculation accuracy...")
        df_ohlcv = df_ohlcv.sort_index()

    # --- 1. Technical Indicator Calculation (TR, ATR, ATRP) ---
    print(f"Calculating technical indicators (ATR Period: {atr_period})...")
    
    # Group by ticker to handle each security independently
    grouped = df_ohlcv.groupby(level='Ticker')
    
    # Get the previous day's close required for True Range
    prev_close = grouped['Adj Close'].shift(1)
    
    # Calculate the three components of True Range
    high_low = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
    high_prev_close = abs(df_ohlcv['Adj High'] - prev_close)
    low_prev_close = abs(df_ohlcv['Adj Low'] - prev_close)
    
    # Combine the components to get the final TR
    tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1, skipna=False)
    
    # # Calculate the ATR using an Exponential Moving Average
    # --- FIX IS HERE ---
    # Use .transform() to apply the EWM function. 
    # This guarantees the resulting Series has the exact same index as 'tr',
    # preventing the index alignment error during the subsequent division.
    atr = tr.groupby(level='Ticker').transform(
        lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean()
    )

    # --- CHANGE 1: Removed .fillna(0) ---
    # ATRP will now be NaN on the first day, consistent with TR and ATR.
    atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)

    indicator_df = pd.DataFrame({
        'TR': tr,
        'ATR': atr,
        'ATRP': atrp
    })
    
    # --- 2. Data Quality Metric Calculation ---
    print(f"Calculating data quality metrics (Window: {quality_window} days)...")
    
    # Create intermediate flags needed for quality calculations
    is_stale = np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0)
    dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']
    has_same_volume = (grouped['Volume'].diff() == 0).astype(int)
    
    # Combine flags into a temporary DataFrame for rolling calculations
    quality_temp_df = pd.DataFrame({
        'IsStale': is_stale,
        'DollarVolume': dollar_volume,
        'HasSameVolume': has_same_volume
    }, index=df_ohlcv.index) # Explicitly set index to be safe
    
    # Perform the rolling calculations on the grouped data
    # --- FIX IS HERE ---
    # We switch to the older, more compatible dictionary-based aggregation method.
    # This syntax is understood by nearly all versions of pandas.
    rolling_result = quality_temp_df.groupby(level='Ticker').rolling(
        window=quality_window,
        min_periods=quality_min_periods
    ).agg({
        'IsStale': 'mean',
        'DollarVolume': 'median',
        'HasSameVolume': 'sum'
    })
    
    # The dictionary syntax produces columns with the original names ('IsStale', etc.).
    # We now explicitly rename them to our desired final names.
    rolling_result = rolling_result.rename(columns={
        'IsStale': 'RollingStalePct',
        'DollarVolume': 'RollMedDollarVol', # <-- RENAMED HERE
        'HasSameVolume': 'RollingSameVolCount'
    })

    # The index after a grouped rolling operation is hierarchical.
    # We remove the outermost 'Ticker' level to restore the original index structure.
    rolling_quality = rolling_result.reset_index(level=0, drop=True)

    # --- 3. Combine All Features ---
    print("Combining all feature sets...")
    features_df = pd.concat([indicator_df, rolling_quality], axis=1)
    
    print("✅ Feature generation complete.")
    return features_df

In [ ]:
def test_features_df(features_df: pd.DataFrame, 
                     df_ohlcv: pd.DataFrame, 
                     test_ticker: str = 'AAPL',
                     spot_check_date: str = '2020-03-20'):
    """
    Runs a suite of tests to verify the correctness of the generated features_df.

    Args:
        features_df: The generated DataFrame from the generate_features function.
        df_ohlcv: The original source OHLCV DataFrame.
        test_ticker: A common, liquid ticker to use for specific value checks.
    """
    print(f"\n--- Running Verification Suite for features_df (Test Ticker: {test_ticker}) ---")
    
    # --- Test 1: Structural Integrity ---
    print("\n[Test 1: Structural Integrity]")
    assert features_df.index.equals(df_ohlcv.index), "FAIL: Index does not match original df_ohlcv."
    print("  ✅ PASS: Index matches original df_ohlcv.")
    
    expected_cols = ['TR', 'ATR', 'ATRP', 'RollingStalePct', 'RollMedDollarVol', 'RollingSameVolCount']
    assert all(col in features_df.columns for col in expected_cols), "FAIL: Missing one or more expected columns."
    print("  ✅ PASS: All expected feature columns are present.")
    print(f"  - DataFrame Info:")
    features_df.info(verbose=False, memory_usage='deep')


    # --- Test 2: ATR Calculation Logic ---
    print("\n[Test 2: ATR Logic Verification]")
    ticker_features = features_df.loc[test_ticker]
    
    # Test 2a: First TR value should be NaN (since prev_close is NaN)
    first_tr = ticker_features['TR'].iloc[0]
    assert pd.isna(first_tr), f"FAIL: First TR value for {test_ticker} should be NaN, but got {first_tr}."
    print(f"  ✅ PASS: First TR value for {test_ticker} is NaN as expected.")

    # Test 2b: The first valid ATR should equal the first valid TR (EWM cold start behavior)
    first_valid_tr_val = ticker_features['TR'].dropna().iloc[0]
    first_valid_atr_val = ticker_features['ATR'].dropna().iloc[0]
    assert np.isclose(first_valid_tr_val, first_valid_atr_val), \
        f"FAIL: First valid ATR ({first_valid_atr_val}) should equal first valid TR ({first_valid_tr_val})."
    print("  ✅ PASS: First valid ATR correctly seeded with first valid TR.")


    # --- Test 3: Rolling Quality Metrics Logic ---
    print("\n[Test 3: Rolling Quality Metrics Logic Verification]")
    quality_min_periods = 126 # Should match the parameter used in generation
    
    # Test 3a: Check for leading NaNs
    first_valid_quality_idx = ticker_features['RollingStalePct'].first_valid_index()
    if first_valid_quality_idx is None:
        print(f"  - INFO: No valid quality metrics found for {test_ticker} (likely too little data). Skipping test.")
    else:
        position_of_first_valid = ticker_features.index.get_loc(first_valid_quality_idx)
        assert position_of_first_valid == quality_min_periods - 1, \
            f"FAIL: First valid quality metric should appear at index {quality_min_periods - 1}, but appeared at {position_of_first_valid}."
        print(f"  ✅ PASS: Leading NaNs are present for the first {quality_min_periods - 1} periods as expected.")


    # --- Test 4: Spot Check Against Manual Calculation ---
    print("\n[Test 4: Spot Check vs. Manual Calculation]")
    # # Choose a specific date for a manual calculation
    # spot_check_date = '2020-03-20' # A volatile day for a good test
    
    # Manual TR Calculation
    today_data = df_ohlcv.loc[(test_ticker, spot_check_date)]
    yesterday_data = df_ohlcv.loc[(test_ticker, pd.to_datetime(spot_check_date) - pd.Timedelta(days=1))] # simple lookback for test
    
    manual_h_l = today_data['Adj High'] - today_data['Adj Low']
    manual_h_pc = abs(today_data['Adj High'] - yesterday_data['Adj Close'])
    manual_l_pc = abs(today_data['Adj Low'] - yesterday_data['Adj Close'])
    manual_tr = max(manual_h_l, manual_h_pc, manual_l_pc)
    
    code_tr = ticker_features.loc[spot_check_date]['TR']
    
    assert np.isclose(manual_tr, code_tr), f"FAIL: Manual TR ({manual_tr:.4f}) does not match code TR ({code_tr:.4f}) on {spot_check_date}."
    print(f"  ✅ PASS: Manually calculated TR on {spot_check_date} matches code's TR.")
    
    print("\n--- ✅ All Verification Tests Passed ---")

In [ ]:
# This might take a minute or two depending on your data size and CPU
features_df = generate_features(df_ohlcv)

In [ ]:
features_df

In [ ]:
test_features_df(features_df, df_ohlcv, test_ticker='VCSH', spot_check_date='2018-10-03') 
# You can change the test_ticker to another well-known stock like 'MSFT' or 'GOOG'

In [ ]:
ticker_features = features_df.loc['VCSH']
ticker_features.loc['2018-10-03':'2019-10-04']

In [ ]:
import pandas as pd
import os

def export_ticker_data(ticker_to_export: str, 
                         df_ohlcv: pd.DataFrame, 
                         features_df: pd.DataFrame, 
                         output_dir: str = 'export_csv'):
    """
    Exports the raw OHLCV data and the corresponding calculated features for a 
    single ticker to two separate CSV files.

    This function is designed for easy manual verification of data and calculations.
    It will create the output directory if it does not exist.

    Args:
        ticker_to_export: The ticker symbol to export (e.g., 'AAPL').
        df_ohlcv: The main DataFrame containing the raw OHLCV data with a 
                  (Ticker, Date) MultiIndex.
        features_df: The DataFrame containing the calculated features with a 
                     (Ticker, Date) MultiIndex.
        output_dir: The directory where the CSV files will be saved. 
                    Defaults to 'export_csv'.
    """
    print(f"--- Attempting to export data for ticker: {ticker_to_export} ---")
    
    # --- 1. Ensure the output directory exists ---
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Output directory '{output_dir}' is ready.")
    except OSError as e:
        print(f"Error: Could not create directory '{output_dir}'. Reason: {e}")
        return

    # --- 2. Isolate the data for the specified ticker ---
    try:
        # Use .loc to select all rows for the given ticker from the MultiIndex
        ticker_ohlcv = df_ohlcv.loc[ticker_to_export]
        ticker_features = features_df.loc[ticker_to_export]
        
        if ticker_ohlcv.empty:
            print(f"Warning: No OHLCV data found for ticker '{ticker_to_export}'. Cannot export.")
            return
            
        print(f"Found {len(ticker_ohlcv)} rows of data for '{ticker_to_export}'.")
        
    except KeyError:
        print(f"Error: Ticker '{ticker_to_export}' not found in one or both of the DataFrames. Please check the symbol.")
        return
    except Exception as e:
        print(f"An unexpected error occurred while accessing data: {e}")
        return

    # --- 3. Construct file paths and export to CSV ---
    try:
        # Define the full path for each output file
        ohlcv_filename = f"{ticker_to_export}_ohlcv.csv"
        features_filename = f"{ticker_to_export}_features.csv"
        
        ohlcv_filepath = os.path.join(output_dir, ohlcv_filename)
        features_filepath = os.path.join(output_dir, features_filename)
        
        # Export the DataFrames to CSV. The index (Date) will be included.
        ticker_ohlcv.to_csv(ohlcv_filepath)
        ticker_features.to_csv(features_filepath)
        
        print("\n✅ Export successful!")
        print(f"   - Raw OHLCV data saved to: {ohlcv_filepath}")
        print(f"   - Calculated features saved to: {features_filepath}")

    except Exception as e:
        print(f"Error: Failed to write data to CSV files. Reason: {e}")

In [ ]:
export_ticker_data('VCSH', df_ohlcv, features_df)

### Part 1: Function to Create Synthetic Ticker Data

This function creates a DataFrame for a single ticker (`SYNTH`) with specific, predictable patterns for stale days, dollar volume, and repeated volumes.

In [ ]:
import pandas as pd
import numpy as np
import os # Make sure os is imported for the export function later

def create_synthetic_ticker_data(
    ticker_name: str = 'SYNTH', 
    num_days: int = 50,
    num_zero_volume_days: int = 5,
    num_flat_price_days: int = 3
) -> pd.DataFrame:
    """
    Creates a synthetic OHLCV DataFrame with predictable patterns and randomly injected
    stale data conditions for robust testing.

    Args:
        ticker_name: The name for the synthetic ticker.
        num_days: The total number of days for the ticker's history.
        num_zero_volume_days: The number of random days to set Volume to 0.
        num_flat_price_days: The number of random days to set High == Low.

    Returns:
        A pandas DataFrame with a (Ticker, Date) MultiIndex.
    """
    print(f"--- Creating synthetic data for '{ticker_name}' with {num_days} days ---")
    
    # 1. Create a base DataFrame with "normal" data
    dates = pd.to_datetime(pd.date_range(start='2023-01-01', periods=num_days, freq='B'))
    data = {
        'Adj Open': 100.0, 'Adj High': 102.0, 'Adj Low': 98.0,
        'Adj Close': 100.0, 'Volume': 1_000_000
    }
    df = pd.DataFrame(data, index=dates)
    df['Adj Close'] = df['Adj Close'] + np.random.randn(num_days) * 0.5 # Add some noise

    # 2. Define a "protected" window for specific verification tests.
    # The `verify_synthetic_ticker_features` function depends on this exact window.
    # We will not inject random stale days here.
    protected_start_idx, protected_end_idx = 10, 20
    
    # 3. Inject random "stale" days OUTSIDE the protected window
    available_indices = df.index.drop(df.index[protected_start_idx:protected_end_idx])
    
    # Inject zero-volume days
    if num_zero_volume_days > 0:
        if len(available_indices) < num_zero_volume_days:
            raise ValueError("Not enough available days to inject zero-volume days.")
        zero_vol_dates = np.random.choice(available_indices, num_zero_volume_days, replace=False)
        df.loc[zero_vol_dates, 'Volume'] = 0
        print(f"  - Injected {num_zero_volume_days} random zero-volume 'stale' days.")
        # Update available indices to avoid overlap
        available_indices = available_indices.drop(zero_vol_dates)

    # Inject flat-price days (High == Low)
    if num_flat_price_days > 0:
        if len(available_indices) < num_flat_price_days:
            raise ValueError("Not enough available days to inject flat-price days.")
        flat_price_dates = np.random.choice(available_indices, num_flat_price_days, replace=False)
        # Set High and Low to be the same as the Close price for that day
        df.loc[flat_price_dates, 'Adj High'] = df.loc[flat_price_dates, 'Adj Close']
        df.loc[flat_price_dates, 'Adj Low'] = df.loc[flat_price_dates, 'Adj Close']
        print(f"  - Injected {num_flat_price_days} random flat-price 'stale' days.")

    # 4. Inject the specific, hand-crafted patterns inside the protected window for verification
    print("  - Injecting specific patterns for programmatic verification...")
    # Pattern for RollingStalePct: 2 stale days in 10 (20%)
    df.iloc[10, df.columns.get_loc('Volume')] = 0  # Stale day (zero volume)
    df.iloc[11, df.columns.get_loc('Adj High')] = 99.0 # Stale day (High == Low)
    df.iloc[11, df.columns.get_loc('Adj Low')] = 99.0
    
    # Pattern for RollingMedianVolume
    for i in range(10):
        df.iloc[10 + i, df.columns.get_loc('Adj Close')] = 100.0 # Standardize price for easy median calc
        df.iloc[10 + i, df.columns.get_loc('Volume')] = (i + 1) * 10000

    # Pattern for RollingSameVolCount
    df.iloc[15, df.columns.get_loc('Volume')] = 77777
    df.iloc[16, df.columns.get_loc('Volume')] = 77777
    df.iloc[17, df.columns.get_loc('Volume')] = 77777
    
    # 5. Set the MultiIndex
    df['Ticker'] = ticker_name
    df = df.set_index(['Ticker', df.index])
    df.index.names = ['Ticker', 'Date']
    
    print("✅ Synthetic data created successfully.")
    return df

### Part 2: Code to Test the Synthetic Data

This new verification function is specifically designed to check the results from our synthetic data. It knows exactly what values to expect on a specific date.

In [ ]:
def verify_synthetic_ticker_features(features_df: pd.DataFrame, 
                                       ticker_name: str = 'SYNTH',
                                       quality_window: int = 10):
    """
    Verifies the quality metric calculations on the features_df generated from
    the synthetic ticker data.

    Args:
        features_df: The DataFrame of calculated features.
        ticker_name: The name of the synthetic ticker.
        quality_window: The rolling window used, which must match the window
                        of the synthetic data pattern.
    """
    print(f"\n--- Running Verification on Synthetic Ticker '{ticker_name}' ---")
    
    # --- Expected values based on our synthetic data design ---
    EXPECTED_STALE_PCT = 0.20  # 2 stale days out of 10
    EXPECTED_MEDIAN_DOLLAR_VOL = 5_500_000.0 # median of (10k..100k) * price of 100
    EXPECTED_SAME_VOL_COUNT = 2.0 # Three consecutive days gives two 'diff() == 0' events

    try:
        # Isolate the features for our synthetic ticker
        ticker_features = features_df.loc[ticker_name]
        
        # The first valid calculation will be on the last day of our 10-day window.
        # The window starts at index 10 and has a length of 10, so it ends at index 19.
        verification_date = ticker_features.index[19]
        
        print(f"Verifying calculations on date: {verification_date.date()}")
        
        # Get the calculated values from the DataFrame
        calculated_values = ticker_features.loc[verification_date]
        stale_pct = calculated_values['RollingStalePct']
        # --- CHANGE 2 (continued): Accessing the renamed column ---
        median_vol = calculated_values['RollMedDollarVol'] 
        same_vol_count = calculated_values['RollingSameVolCount']
        
        # --- Perform Assertions ---
        print("\n[Test 1: RollingStalePct]")
        assert np.isclose(stale_pct, EXPECTED_STALE_PCT), f"FAIL: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}"
        print(f"  ✅ PASS: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}")

        print("\n[Test 2: RollMedDollarVol]")
        assert np.isclose(median_vol, EXPECTED_MEDIAN_DOLLAR_VOL), f"FAIL: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}"
        print(f"  ✅ PASS: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}")

        print("\n[Test 3: RollingSameVolCount]")
        assert np.isclose(same_vol_count, EXPECTED_SAME_VOL_COUNT), f"FAIL: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}"
        print(f"  ✅ PASS: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}")

        print("\n--- ✅ All Synthetic Data Verification Tests Passed ---")

    except KeyError:
        print(f"FAIL: Ticker '{ticker_name}' not found in features_df.")
    except IndexError:
        print("FAIL: Not enough data in features_df to run verification. Check num_days.")
    except Exception as e:
        print(f"An unexpected error occurred during verification: {e}")

## Part 3: Main Script to Run Everything

This block of code ties it all together. You will need your `generate_features` and `export_ticker_data` functions available in the same environment.


In [ ]:
# --- Main Execution Block ---

# Make sure you have 'generate_features', 'verify_synthetic_ticker_features', 
# and 'export_ticker_data' functions available in your environment.

# --- Step 1: Define Parameters ---
SYNTHETIC_TICKER_NAME = 'SYNTH_STALE_TEST'
QUALITY_WINDOW_FOR_TEST = 20
MIN_PERIODS_FOR_TEST = 10 

# --- Step 2: Create Synthetic Data with More Stale Days ---
# We'll create 8 zero-volume days and 4 flat-price days.
df_ohlcv_synth = create_synthetic_ticker_data(
    ticker_name=SYNTHETIC_TICKER_NAME, 
    num_days=100, # Use more days to have space for random stale days
    num_zero_volume_days=8,
    num_flat_price_days=4
)

# --- Step 3: Run Feature Generation ---
features_df_synth = generate_features(
    df_ohlcv_synth,
    quality_window=QUALITY_WINDOW_FOR_TEST,
    quality_min_periods=MIN_PERIODS_FOR_TEST
)

# --- Step 4: Verify the Core Logic (this will still pass) ---
# The verification function checks the specific 10-day window, which we preserved.
verify_synthetic_ticker_features(
    features_df_synth,
    ticker_name=SYNTHETIC_TICKER_NAME,
    quality_window=QUALITY_WINDOW_FOR_TEST
)

# --- Step 5: Export for Manual Inspection ---
# When you open the CSVs, you will now see the additional random stale days
# you created, allowing you to manually check the rolling calculations anywhere.
print("\n--- Exporting Enhanced Synthetic Data for Manual Review ---")
export_ticker_data(
    ticker_to_export=SYNTHETIC_TICKER_NAME,
    df_ohlcv=df_ohlcv_synth,
    features_df=features_df_synth
)

# --- Step 6: Generate_features Parameters ---
print("\n--- Generate_features Parameters ---")
print(f'quality_window: {QUALITY_WINDOW_FOR_TEST}')
print(f'quality_min_periods: {MIN_PERIODS_FOR_TEST}')

### End of Code for Refactoring Phase 1

## Project Hand-off: Walk-Forward Backtesting Bot

This package contains the final version of the code, designed to be resilient, portable, and easy to use in both local (VS Code) and cloud (Google Colab) environments.

### 1. Summary of Key Features

The system you have built now includes:

*   **Environment-Agnostic Operation:** A "magic switch" automatically detects whether the code is running locally or in Colab and adjusts all file paths accordingly.
*   **Resumable Backtests (Checkpointing):** Long-running parameter searches are now resilient. If the process is interrupted, it can be restarted and will automatically skip completed work, picking up where it left off.
*   **Granular, Trading-Day-Based Logic:** The backtester operates on precise integer counts of trading days, allowing for non-calendar-based periods (e.g., 10-day holds) and eliminating approximation errors.
*   **Multi-Period Testing:** The automation script is capable of testing a list of different holding/rebalancing periods in a single run.
*   **Modular & Verifiable Core Engine:** The core calculation logic (`run_walk_forward_step`) is a pure, self-contained function, making it easy to test and verify independently.
*   **Dynamic Data Quality Filtering:** Before each ranking period, the universe of stocks is filtered based on rolling liquidity and data quality metrics, ensuring the strategy is only applied to tradable assets.

### 2. Required Project Structure

For the environment switch to work seamlessly, your project should be organized in the following way, both on your local machine and in Google Drive.

```
my_trading_project/
│
├── 📜 bot.ipynb                 # <-- This is the main notebook file
│
├── 📁 data/
│   └── 📊 df_OHLCV_stocks_etfs.parquet # <-- Your input data file goes here
│
└── 📁 export_csv/                 # <-- Folder for local results (created automatically)
```

### 3. Final, Complete Code

This is the entire code for your notebook, consolidated into logical cells.

#### **CELL 1: ENVIRONMENT SETUP & CONFIGURATION**
*This cell is the "brain" of the system. It detects the environment and configures all paths. It's the only cell you might need to edit if your file paths change.*

In [ ]:
# ==============================================================================
# --- CELL 1: ENVIRONMENT SETUP & CONFIGURATION (IMPROVED) ---
# This cell automatically detects the environment (local VS Code or Google Colab)
# and configures paths and settings accordingly. It also creates directories.
# ==============================================================================
import sys
import os

# 1. AUTOMATIC ENVIRONMENT DETECTION
try:
    import google.colab
    IS_COLAB = True
    print("✅ Environment: Google Colab detected.")
except ImportError:
    IS_COLAB = False
    print("✅ Environment: Local (VS Code) detected.")

# 2. ENVIRONMENT-SPECIFIC CONFIGURATION
if IS_COLAB:
    # --- Colab Settings ---
    from google.colab import drive, output
    drive.mount('/content/drive')
    output.enable_custom_widget_manager()
    
    # IMPORTANT: This should be the path to your main project folder in Google Drive
    DRIVE_ROOT = '/content/drive/MyDrive/my_trading_project'
    
    env_config = {
        'data_path': os.path.join(DRIVE_ROOT, 'data', 'df_OHLCV_stocks_etfs.parquet'),
        'output_dir': os.path.join(DRIVE_ROOT, 'results') # Colab results go in a 'results' folder
    }
    
else:
    # --- Local Settings ---
    # IMPORTANT: Update this path to your local data file if it's different
    env_config = {
        'data_path': r'c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet',
        'output_dir': os.path.join('.', 'export_csv') # Local results go in 'export_csv'
    }

# 3. CREATE ALL NECESSARY DIRECTORIES
data_parent_dir = os.path.dirname(env_config['data_path'])
os.makedirs(data_parent_dir, exist_ok=True)
os.makedirs(env_config['output_dir'], exist_ok=True)

print(f"\nData will be loaded from: {env_config['data_path']}")
print(f"Output files will be saved to: {env_config['output_dir']}")

# 4. DEFINE THE FULL PATH FOR THE RESULTS FILE
env_config['results_path'] = os.path.join(env_config['output_dir'], 'dev_strategy_search_results.csv')



#### **CELL 2: GOLDEN COPY - CORE ENGINE & TOOLS**
*This cell contains all the stable, tested functions that form the core of your backtester.*

In [ ]:
# ==============================================================================
# GOLDEN COPY - COMPLETE PROJECT CODE (All Fixes Included)
# Version: Added Refactoring Phase 1 code  
# Date: 2025-10-14
# ==============================================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import time
import pprint
import os # Make sure os is imported for the export function later
import re

from datetime import datetime, date
from IPython.display import display, Markdown
from tqdm.auto import tqdm
from pathlib import Path
from itertools import product


pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 3000)


# --- REFACTORING PHASE 1 CODE: Feature Generation Engine ---

def generate_features(df_ohlcv: pd.DataFrame, 
                      atr_period: int = 14, 
                      quality_window: int = 252, 
                      quality_min_periods: int = 126) -> pd.DataFrame:
    """
    Generates a comprehensive DataFrame of derived features from raw OHLCV data.

    This function performs all heavy, window-based calculations upfront to be used
    by downstream analysis functions. It calculates:
    1. Technical Indicators: True Range (TR), ATR, and ATRP.
    2. Data Quality Metrics: Rolling stale percentage, median dollar volume, etc.

    Args:
        df_ohlcv: The primary DataFrame with a (Ticker, Date) MultiIndex and
                  columns for 'Adj High', 'Adj Low', 'Adj Close', 'Volume'.
        atr_period: The lookback period for the ATR's Exponential Moving Average.
        quality_window: The rolling window size for data quality metrics.
        quality_min_periods: The minimum number of observations required to have
                             a valid quality metric.

    Returns:
        A new DataFrame with the same (Ticker, Date) MultiIndex containing all
        calculated feature columns.
    """
    print("--- Starting Feature Generation ---")
    
    # Ensure the DataFrame is sorted for correct window and shift operations
    # FIX: Replaced is_lexsorted() with the current pandas attribute
    if not df_ohlcv.index.is_monotonic_increasing:
        print("Sorting index for calculation accuracy...")
        df_ohlcv = df_ohlcv.sort_index()

    # --- 1. Technical Indicator Calculation (TR, ATR, ATRP) ---
    print(f"Calculating technical indicators (ATR Period: {atr_period})...")
    
    # Group by ticker to handle each security independently
    grouped = df_ohlcv.groupby(level='Ticker')
    
    # Get the previous day's close required for True Range
    prev_close = grouped['Adj Close'].shift(1)
    
    # Calculate the three components of True Range
    high_low = df_ohlcv['Adj High'] - df_ohlcv['Adj Low']
    high_prev_close = abs(df_ohlcv['Adj High'] - prev_close)
    low_prev_close = abs(df_ohlcv['Adj Low'] - prev_close)
    
    # Combine the components to get the final TR
    tr = pd.concat([high_low, high_prev_close, low_prev_close], axis=1).max(axis=1, skipna=False)
    
    # # Calculate the ATR using an Exponential Moving Average
    # --- FIX IS HERE ---
    # Use .transform() to apply the EWM function. 
    # This guarantees the resulting Series has the exact same index as 'tr',
    # preventing the index alignment error during the subsequent division.
    atr = tr.groupby(level='Ticker').transform(
        lambda x: x.ewm(alpha=1/atr_period, adjust=False).mean()
    )

    # --- CHANGE 1: Removed .fillna(0) ---
    # ATRP will now be NaN on the first day, consistent with TR and ATR.
    atrp = (atr / df_ohlcv['Adj Close']).replace([np.inf, -np.inf], np.nan)

    indicator_df = pd.DataFrame({
        'TR': tr,
        'ATR': atr,
        'ATRP': atrp
    })
    
    # --- 2. Data Quality Metric Calculation ---
    print(f"Calculating data quality metrics (Window: {quality_window} days)...")
    
    # Create intermediate flags needed for quality calculations
    is_stale = np.where((df_ohlcv['Volume'] == 0) | (df_ohlcv['Adj High'] == df_ohlcv['Adj Low']), 1, 0)
    dollar_volume = df_ohlcv['Adj Close'] * df_ohlcv['Volume']
    has_same_volume = (grouped['Volume'].diff() == 0).astype(int)
    
    # Combine flags into a temporary DataFrame for rolling calculations
    quality_temp_df = pd.DataFrame({
        'IsStale': is_stale,
        'DollarVolume': dollar_volume,
        'HasSameVolume': has_same_volume
    }, index=df_ohlcv.index) # Explicitly set index to be safe
    
    # Perform the rolling calculations on the grouped data
    # --- FIX IS HERE ---
    # We switch to the older, more compatible dictionary-based aggregation method.
    # This syntax is understood by nearly all versions of pandas.
    rolling_result = quality_temp_df.groupby(level='Ticker').rolling(
        window=quality_window,
        min_periods=quality_min_periods
    ).agg({
        'IsStale': 'mean',
        'DollarVolume': 'median',
        'HasSameVolume': 'sum'
    })
    
    # The dictionary syntax produces columns with the original names ('IsStale', etc.).
    # We now explicitly rename them to our desired final names.
    rolling_result = rolling_result.rename(columns={
        'IsStale': 'RollingStalePct',
        'DollarVolume': 'RollMedDollarVol', # <-- RENAMED HERE
        'HasSameVolume': 'RollingSameVolCount'
    })

    # The index after a grouped rolling operation is hierarchical.
    # We remove the outermost 'Ticker' level to restore the original index structure.
    rolling_quality = rolling_result.reset_index(level=0, drop=True)

    # --- 3. Combine All Features ---
    print("Combining all feature sets...")
    features_df = pd.concat([indicator_df, rolling_quality], axis=1)
    
    print("✅ Feature generation complete.")
    return features_df

def test_features_df(features_df: pd.DataFrame, 
                     df_ohlcv: pd.DataFrame, 
                     test_ticker: str = 'AAPL',
                     spot_check_date: str = '2020-03-20'):
    """
    Runs a suite of tests to verify the correctness of the generated features_df.

    Args:
        features_df: The generated DataFrame from the generate_features function.
        df_ohlcv: The original source OHLCV DataFrame.
        test_ticker: A common, liquid ticker to use for specific value checks.
    """
    print(f"\n--- Running Verification Suite for features_df (Test Ticker: {test_ticker}) ---")
    
    # --- Test 1: Structural Integrity ---
    print("\n[Test 1: Structural Integrity]")
    assert features_df.index.equals(df_ohlcv.index), "FAIL: Index does not match original df_ohlcv."
    print("  ✅ PASS: Index matches original df_ohlcv.")
    
    expected_cols = ['TR', 'ATR', 'ATRP', 'RollingStalePct', 'RollMedDollarVol', 'RollingSameVolCount']
    assert all(col in features_df.columns for col in expected_cols), "FAIL: Missing one or more expected columns."
    print("  ✅ PASS: All expected feature columns are present.")
    print(f"  - DataFrame Info:")
    features_df.info(verbose=False, memory_usage='deep')


    # --- Test 2: ATR Calculation Logic ---
    print("\n[Test 2: ATR Logic Verification]")
    ticker_features = features_df.loc[test_ticker]
    
    # Test 2a: First TR value should be NaN (since prev_close is NaN)
    first_tr = ticker_features['TR'].iloc[0]
    assert pd.isna(first_tr), f"FAIL: First TR value for {test_ticker} should be NaN, but got {first_tr}."
    print(f"  ✅ PASS: First TR value for {test_ticker} is NaN as expected.")

    # Test 2b: The first valid ATR should equal the first valid TR (EWM cold start behavior)
    first_valid_tr_val = ticker_features['TR'].dropna().iloc[0]
    first_valid_atr_val = ticker_features['ATR'].dropna().iloc[0]
    assert np.isclose(first_valid_tr_val, first_valid_atr_val), \
        f"FAIL: First valid ATR ({first_valid_atr_val}) should equal first valid TR ({first_valid_tr_val})."
    print("  ✅ PASS: First valid ATR correctly seeded with first valid TR.")


    # --- Test 3: Rolling Quality Metrics Logic ---
    print("\n[Test 3: Rolling Quality Metrics Logic Verification]")
    quality_min_periods = 126 # Should match the parameter used in generation
    
    # Test 3a: Check for leading NaNs
    first_valid_quality_idx = ticker_features['RollingStalePct'].first_valid_index()
    if first_valid_quality_idx is None:
        print(f"  - INFO: No valid quality metrics found for {test_ticker} (likely too little data). Skipping test.")
    else:
        position_of_first_valid = ticker_features.index.get_loc(first_valid_quality_idx)
        assert position_of_first_valid == quality_min_periods - 1, \
            f"FAIL: First valid quality metric should appear at index {quality_min_periods - 1}, but appeared at {position_of_first_valid}."
        print(f"  ✅ PASS: Leading NaNs are present for the first {quality_min_periods - 1} periods as expected.")


    # --- Test 4: Spot Check Against Manual Calculation ---
    print("\n[Test 4: Spot Check vs. Manual Calculation]")
    # # Choose a specific date for a manual calculation
    # spot_check_date = '2020-03-20' # A volatile day for a good test
    
    # Manual TR Calculation
    today_data = df_ohlcv.loc[(test_ticker, spot_check_date)]
    yesterday_data = df_ohlcv.loc[(test_ticker, pd.to_datetime(spot_check_date) - pd.Timedelta(days=1))] # simple lookback for test
    
    manual_h_l = today_data['Adj High'] - today_data['Adj Low']
    manual_h_pc = abs(today_data['Adj High'] - yesterday_data['Adj Close'])
    manual_l_pc = abs(today_data['Adj Low'] - yesterday_data['Adj Close'])
    manual_tr = max(manual_h_l, manual_h_pc, manual_l_pc)
    
    code_tr = ticker_features.loc[spot_check_date]['TR']
    
    assert np.isclose(manual_tr, code_tr), f"FAIL: Manual TR ({manual_tr:.4f}) does not match code TR ({code_tr:.4f}) on {spot_check_date}."
    print(f"  ✅ PASS: Manually calculated TR on {spot_check_date} matches code's TR.")
    
    print("\n--- ✅ All Verification Tests Passed ---")

def export_ticker_data(ticker_to_export: str, 
                         df_ohlcv: pd.DataFrame, 
                         features_df: pd.DataFrame, 
                         output_dir: str = 'export_csv'):
    """
    Exports the raw OHLCV data and the corresponding calculated features for a 
    single ticker to two separate CSV files.

    This function is designed for easy manual verification of data and calculations.
    It will create the output directory if it does not exist.

    Args:
        ticker_to_export: The ticker symbol to export (e.g., 'AAPL').
        df_ohlcv: The main DataFrame containing the raw OHLCV data with a 
                  (Ticker, Date) MultiIndex.
        features_df: The DataFrame containing the calculated features with a 
                     (Ticker, Date) MultiIndex.
        output_dir: The directory where the CSV files will be saved. 
                    Defaults to 'export_csv'.
    """
    print(f"--- Attempting to export data for ticker: {ticker_to_export} ---")
    
    # --- 1. Ensure the output directory exists ---
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Output directory '{output_dir}' is ready.")
    except OSError as e:
        print(f"Error: Could not create directory '{output_dir}'. Reason: {e}")
        return

    # --- 2. Isolate the data for the specified ticker ---
    try:
        # Use .loc to select all rows for the given ticker from the MultiIndex
        ticker_ohlcv = df_ohlcv.loc[ticker_to_export]
        ticker_features = features_df.loc[ticker_to_export]
        
        if ticker_ohlcv.empty:
            print(f"Warning: No OHLCV data found for ticker '{ticker_to_export}'. Cannot export.")
            return
            
        print(f"Found {len(ticker_ohlcv)} rows of data for '{ticker_to_export}'.")
        
    except KeyError:
        print(f"Error: Ticker '{ticker_to_export}' not found in one or both of the DataFrames. Please check the symbol.")
        return
    except Exception as e:
        print(f"An unexpected error occurred while accessing data: {e}")
        return

    # --- 3. Construct file paths and export to CSV ---
    try:
        # Define the full path for each output file
        ohlcv_filename = f"{ticker_to_export}_ohlcv.csv"
        features_filename = f"{ticker_to_export}_features.csv"
        
        ohlcv_filepath = os.path.join(output_dir, ohlcv_filename)
        features_filepath = os.path.join(output_dir, features_filename)
        
        # Export the DataFrames to CSV. The index (Date) will be included.
        ticker_ohlcv.to_csv(ohlcv_filepath)
        ticker_features.to_csv(features_filepath)
        
        print("\n✅ Export successful!")
        print(f"   - Raw OHLCV data saved to: {ohlcv_filepath}")
        print(f"   - Calculated features saved to: {features_filepath}")

    except Exception as e:
        print(f"Error: Failed to write data to CSV files. Reason: {e}")

def create_synthetic_ticker_data(
    ticker_name: str = 'SYNTH', 
    num_days: int = 50,
    num_zero_volume_days: int = 5,
    num_flat_price_days: int = 3
) -> pd.DataFrame:
    """
    Creates a synthetic OHLCV DataFrame with predictable patterns and randomly injected
    stale data conditions for robust testing.

    Args:
        ticker_name: The name for the synthetic ticker.
        num_days: The total number of days for the ticker's history.
        num_zero_volume_days: The number of random days to set Volume to 0.
        num_flat_price_days: The number of random days to set High == Low.

    Returns:
        A pandas DataFrame with a (Ticker, Date) MultiIndex.
    """
    print(f"--- Creating synthetic data for '{ticker_name}' with {num_days} days ---")
    
    # 1. Create a base DataFrame with "normal" data
    dates = pd.to_datetime(pd.date_range(start='2023-01-01', periods=num_days, freq='B'))
    data = {
        'Adj Open': 100.0, 'Adj High': 102.0, 'Adj Low': 98.0,
        'Adj Close': 100.0, 'Volume': 1_000_000
    }
    df = pd.DataFrame(data, index=dates)
    df['Adj Close'] = df['Adj Close'] + np.random.randn(num_days) * 0.5 # Add some noise

    # 2. Define a "protected" window for specific verification tests.
    # The `verify_synthetic_ticker_features` function depends on this exact window.
    # We will not inject random stale days here.
    protected_start_idx, protected_end_idx = 10, 20
    
    # 3. Inject random "stale" days OUTSIDE the protected window
    available_indices = df.index.drop(df.index[protected_start_idx:protected_end_idx])
    
    # Inject zero-volume days
    if num_zero_volume_days > 0:
        if len(available_indices) < num_zero_volume_days:
            raise ValueError("Not enough available days to inject zero-volume days.")
        zero_vol_dates = np.random.choice(available_indices, num_zero_volume_days, replace=False)
        df.loc[zero_vol_dates, 'Volume'] = 0
        print(f"  - Injected {num_zero_volume_days} random zero-volume 'stale' days.")
        # Update available indices to avoid overlap
        available_indices = available_indices.drop(zero_vol_dates)

    # Inject flat-price days (High == Low)
    if num_flat_price_days > 0:
        if len(available_indices) < num_flat_price_days:
            raise ValueError("Not enough available days to inject flat-price days.")
        flat_price_dates = np.random.choice(available_indices, num_flat_price_days, replace=False)
        # Set High and Low to be the same as the Close price for that day
        df.loc[flat_price_dates, 'Adj High'] = df.loc[flat_price_dates, 'Adj Close']
        df.loc[flat_price_dates, 'Adj Low'] = df.loc[flat_price_dates, 'Adj Close']
        print(f"  - Injected {num_flat_price_days} random flat-price 'stale' days.")

    # 4. Inject the specific, hand-crafted patterns inside the protected window for verification
    print("  - Injecting specific patterns for programmatic verification...")
    # Pattern for RollingStalePct: 2 stale days in 10 (20%)
    df.iloc[10, df.columns.get_loc('Volume')] = 0  # Stale day (zero volume)
    df.iloc[11, df.columns.get_loc('Adj High')] = 99.0 # Stale day (High == Low)
    df.iloc[11, df.columns.get_loc('Adj Low')] = 99.0
    
    # Pattern for RollingMedianVolume
    for i in range(10):
        df.iloc[10 + i, df.columns.get_loc('Adj Close')] = 100.0 # Standardize price for easy median calc
        df.iloc[10 + i, df.columns.get_loc('Volume')] = (i + 1) * 10000

    # Pattern for RollingSameVolCount
    df.iloc[15, df.columns.get_loc('Volume')] = 77777
    df.iloc[16, df.columns.get_loc('Volume')] = 77777
    df.iloc[17, df.columns.get_loc('Volume')] = 77777
    
    # 5. Set the MultiIndex
    df['Ticker'] = ticker_name
    df = df.set_index(['Ticker', df.index])
    df.index.names = ['Ticker', 'Date']
    
    print("✅ Synthetic data created successfully.")
    return df

def verify_synthetic_ticker_features(features_df: pd.DataFrame, 
                                       ticker_name: str = 'SYNTH',
                                       quality_window: int = 10):
    """
    Verifies the quality metric calculations on the features_df generated from
    the synthetic ticker data.

    Args:
        features_df: The DataFrame of calculated features.
        ticker_name: The name of the synthetic ticker.
        quality_window: The rolling window used, which must match the window
                        of the synthetic data pattern.
    """
    print(f"\n--- Running Verification on Synthetic Ticker '{ticker_name}' ---")
    
    # --- Expected values based on our synthetic data design ---
    EXPECTED_STALE_PCT = 0.20  # 2 stale days out of 10
    EXPECTED_MEDIAN_DOLLAR_VOL = 5_500_000.0 # median of (10k..100k) * price of 100
    EXPECTED_SAME_VOL_COUNT = 2.0 # Three consecutive days gives two 'diff() == 0' events

    try:
        # Isolate the features for our synthetic ticker
        ticker_features = features_df.loc[ticker_name]
        
        # The first valid calculation will be on the last day of our 10-day window.
        # The window starts at index 10 and has a length of 10, so it ends at index 19.
        verification_date = ticker_features.index[19]
        
        print(f"Verifying calculations on date: {verification_date.date()}")
        
        # Get the calculated values from the DataFrame
        calculated_values = ticker_features.loc[verification_date]
        stale_pct = calculated_values['RollingStalePct']
        # --- CHANGE 2 (continued): Accessing the renamed column ---
        median_vol = calculated_values['RollMedDollarVol'] 
        same_vol_count = calculated_values['RollingSameVolCount']
        
        # --- Perform Assertions ---
        print("\n[Test 1: RollingStalePct]")
        assert np.isclose(stale_pct, EXPECTED_STALE_PCT), f"FAIL: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}"
        print(f"  ✅ PASS: Expected {EXPECTED_STALE_PCT}, Got {stale_pct}")

        print("\n[Test 2: RollMedDollarVol]")
        assert np.isclose(median_vol, EXPECTED_MEDIAN_DOLLAR_VOL), f"FAIL: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}"
        print(f"  ✅ PASS: Expected {EXPECTED_MEDIAN_DOLLAR_VOL}, Got {median_vol}")

        print("\n[Test 3: RollingSameVolCount]")
        assert np.isclose(same_vol_count, EXPECTED_SAME_VOL_COUNT), f"FAIL: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}"
        print(f"  ✅ PASS: Expected {EXPECTED_SAME_VOL_COUNT}, Got {same_vol_count}")

        print("\n--- ✅ All Synthetic Data Verification Tests Passed ---")

    except KeyError:
        print(f"FAIL: Ticker '{ticker_name}' not found in features_df.")
    except IndexError:
        print("FAIL: Not enough data in features_df to run verification. Check num_days.")
    except Exception as e:
        print(f"An unexpected error occurred during verification: {e}")

# --- A. HELPER FUNCTIONS ---

def calculate_gain(price_series: pd.Series):
    """Calculates the total gain over a series of prices."""
    # Ensure there are at least two data points to calculate a gain
    if price_series.dropna().shape[0] < 2: return np.nan
    # Use forward-fill for the end price and back-fill for the start price
    # to handle potential NaNs at the beginning or end of the series.
    return (price_series.ffill().iloc[-1] / price_series.bfill().iloc[0]) - 1

def calculate_sharpe(return_series: pd.Series):
    """Calculates the annualized Sharpe ratio from a series of daily returns."""
    # Ensure there are at least two returns to calculate a standard deviation
    if return_series.dropna().shape[0] < 2: return np.nan
    std_dev = return_series.std()
    # Avoid division by zero if returns are constant
    if std_dev > 0 and std_dev != np.inf:
        return (return_series.mean() / std_dev) * np.sqrt(252)
    return np.nan

def print_nested(d, indent=0, width=4):
    """Pretty-print any nested dict/list/tuple combination."""
    spacing = ' ' * indent
    if isinstance(d, dict):
        for k, v in d.items():
            print(f'{spacing}{k}:')
            print_nested(v, indent + width, width)
    elif isinstance(d, (list, tuple)):
        for item in d:
            print_nested(item, indent, width)
    else:
        print(f'{spacing}{d}')

# --- B. THE CORE CALCULATION ENGINE ---

def run_walk_forward_step(df_close_full, df_high_full, df_low_full,
                          master_trading_days,
                          start_date, calc_period, fwd_period,
                          metric, rank_start, rank_end, benchmark_ticker,
                          debug=False):
    """Runs a single step of the walk-forward analysis using precise trading days."""
    debug_data = {} if debug else None
    
    # 1. Determine exact date ranges using the master trading day calendar
    try:
        start_idx = master_trading_days.get_loc(start_date)
    except KeyError:
        return ({'error': f"Start date {start_date.date()} is not a valid trading day."}, None)
        
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    viz_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)

    safe_start_date = master_trading_days[start_idx]
    safe_calc_end_date = master_trading_days[calc_end_idx]
    safe_viz_end_date = master_trading_days[viz_end_idx]
    
    if safe_start_date >= safe_calc_end_date:
        return ({'error': "Invalid date range (calc period has zero or negative length)."}, None)

    # 2. Slice data for the calculation period and filter for valid tickers
    calc_close_raw = df_close_full.loc[safe_start_date:safe_calc_end_date]
    calc_close = calc_close_raw.dropna(axis=1, how='all') # Drop tickers with no data in the period
    if calc_close.shape[1] == 0 or len(calc_close) < 2:
        return ({'error': "Not enough data in calc period."}, None)

    # 3. Calculate ranking metrics for all valid tickers
    first_prices = calc_close.bfill().iloc[0]
    last_prices = calc_close.ffill().iloc[-1]
    daily_returns = calc_close.bfill().ffill().pct_change()
    mean_returns = daily_returns.mean()
    std_returns = daily_returns.std()
    
    valid_tickers = calc_close.columns
    calc_high = df_high_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    calc_low = df_low_full[valid_tickers].loc[safe_start_date:safe_calc_end_date]
    
    # Correctly calculate True Range (TR) for a multi-ticker DataFrame
    # First, align the previous day's close to the current calculation window.
    prev_close = df_close_full[valid_tickers].shift(1).loc[safe_start_date:safe_calc_end_date]
    
    # Calculate the three components of True Range. Each result is a DataFrame.
    component1 = calc_high - calc_low
    component2 = abs(calc_high - prev_close)
    component3 = abs(calc_low - prev_close)

    # Find the element-wise maximum across the three component DataFrames.
    # np.maximum is efficient and preserves the DataFrame structure.
    tr = np.maximum(component1, np.maximum(component2, component3))
    
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    atrp = (atr / calc_close).mean() # Mean ATRP over the calculation period

    metric_values = {}
    metric_values['Price'] = (last_prices / first_prices).dropna()
    metric_values['Sharpe'] = (mean_returns / std_returns * np.sqrt(252)).fillna(0)
    metric_values['Sharpe (ATR)'] = (mean_returns / atrp).fillna(0)

    if debug:
        df_ranking = pd.DataFrame({
            'FirstPrice': first_prices, 'LastPrice': last_prices, 'MeanDailyReturn': mean_returns,
            'StdDevDailyReturn': std_returns, 'MeanATRP': atrp, 'Metric_Price': metric_values['Price'],
            'Metric_Sharpe': metric_values['Sharpe'], 'Metric_Sharpe (ATR)': metric_values['Sharpe (ATR)']
        })
        df_ranking.index.name = 'Ticker'
        debug_data['ranking_metrics'] = df_ranking.sort_values(f'Metric_{metric}', ascending=False)

    # 4. Rank tickers and select the target group
    sorted_tickers = metric_values[metric].sort_values(ascending=False)
    tickers_to_display = sorted_tickers.index[rank_start-1:rank_end].tolist()
    if not tickers_to_display:
        return ({'error': "No tickers found for the selected rank."}, None)

    # 5. Prepare data for plotting and portfolio performance calculation
    normalized_plot_data = df_close_full[tickers_to_display].loc[safe_start_date:safe_viz_end_date]
    normalized_plot_data = normalized_plot_data.div(normalized_plot_data.bfill().iloc[0])
    actual_calc_end_ts = calc_close.index.max()
    portfolio_series = normalized_plot_data.mean(axis=1)
    portfolio_return_series = portfolio_series.pct_change()
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    benchmark_return_series = benchmark_price_series.loc[safe_start_date:safe_viz_end_date].bfill().ffill().pct_change() if benchmark_price_series is not None else pd.Series(dtype='float64')

    # 6. Correctly slice return series for Sharpe calculation to prevent lookahead
    try:
        # Use index location for a clean, non-overlapping split
        boundary_loc = portfolio_return_series.index.get_loc(actual_calc_end_ts)
        calc_portfolio_returns = portfolio_return_series.iloc[:boundary_loc + 1]
        fwd_portfolio_returns = portfolio_return_series.iloc[boundary_loc + 1:]
        
        if benchmark_price_series is not None:
            bm_boundary_loc = benchmark_return_series.index.get_loc(actual_calc_end_ts)
            calc_benchmark_returns = benchmark_return_series.iloc[:bm_boundary_loc + 1]
            fwd_benchmark_returns = benchmark_return_series.iloc[bm_boundary_loc + 1:]
        else:
            calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')
            
    except (KeyError, IndexError): # Fallback for edge cases
        calc_portfolio_returns = portfolio_return_series.loc[:actual_calc_end_ts]
        fwd_portfolio_returns = portfolio_return_series.loc[actual_calc_end_ts:].iloc[1:]
        if benchmark_price_series is not None:
            calc_benchmark_returns = benchmark_return_series.loc[:actual_calc_end_ts]
            fwd_benchmark_returns = benchmark_return_series.loc[actual_calc_end_ts:].iloc[1:]
        else:
            calc_benchmark_returns, fwd_benchmark_returns = pd.Series(dtype='float64'), pd.Series(dtype='float64')

    # 7. Calculate performance metrics (Gain & Sharpe) for all periods
    perf_data = {}
    perf_data['calc_p_gain'] = calculate_gain(portfolio_series.loc[:actual_calc_end_ts])
    perf_data['fwd_p_gain'] = calculate_gain(portfolio_series.loc[actual_calc_end_ts:])
    perf_data['full_p_gain'] = calculate_gain(portfolio_series)
    perf_data['calc_p_sharpe'] = calculate_sharpe(calc_portfolio_returns)
    perf_data['fwd_p_sharpe'] = calculate_sharpe(fwd_portfolio_returns)
    perf_data['full_p_sharpe'] = calculate_sharpe(portfolio_return_series)
    
    perf_data['calc_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:actual_calc_end_ts]) if benchmark_price_series is not None else np.nan
    perf_data['fwd_b_gain'] = calculate_gain(benchmark_price_series.loc[actual_calc_end_ts:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['full_b_gain'] = calculate_gain(benchmark_price_series.loc[safe_start_date:safe_viz_end_date]) if benchmark_price_series is not None else np.nan
    perf_data['calc_b_sharpe'] = calculate_sharpe(calc_benchmark_returns)
    perf_data['fwd_b_sharpe'] = calculate_sharpe(fwd_benchmark_returns)
    perf_data['full_b_sharpe'] = calculate_sharpe(benchmark_return_series)

    # 8. Assemble results DataFrame for display
    calc_end_prices = calc_close.ffill().iloc[-1]
    fwd_close_slice = df_close_full.loc[actual_calc_end_ts:safe_viz_end_date]
    viz_end_prices = fwd_close_slice.ffill().iloc[-1] if not fwd_close_slice.empty and len(fwd_close_slice) >= 2 else calc_end_prices
    calc_gains = (calc_end_prices / calc_close.bfill().iloc[0]) - 1
    fwd_gains = (viz_end_prices / calc_end_prices) - 1
    results_df = pd.DataFrame({'Rank': range(rank_start, rank_start + len(tickers_to_display)), 'Metric': metric, 'MetricValue': sorted_tickers.loc[tickers_to_display].values, 'CalcPrice': calc_end_prices.loc[tickers_to_display], 'CalcGain': calc_gains.loc[tickers_to_display], 'FwdGain': fwd_gains.loc[tickers_to_display]}, index=pd.Index(tickers_to_display, name='Ticker'))
    if benchmark_price_series is not None and benchmark_ticker in calc_close.columns:
        benchmark_df_row = pd.DataFrame({'Rank': np.nan, 'Metric': metric, 'MetricValue': metric_values[metric].get(benchmark_ticker, np.nan), 'CalcPrice': calc_end_prices[benchmark_ticker], 'CalcGain': calc_gains[benchmark_ticker], 'FwdGain': fwd_gains[benchmark_ticker]}, index=pd.Index([f"{benchmark_ticker} (BM)"], name='Ticker'))
        results_df = pd.concat([results_df, benchmark_df_row])
    
    # 9. Assemble debug data if requested
    if debug:
        df_trace = normalized_plot_data.copy()
        df_trace.columns = [f'Norm_Price_{c}' for c in df_trace.columns]
        df_trace['Norm_Price_Portfolio'] = portfolio_series
        if benchmark_price_series is not None and not benchmark_price_series.loc[safe_start_date:safe_viz_end_date].dropna().empty:
            norm_bm = benchmark_price_series.loc[safe_start_date:safe_viz_end_date] / benchmark_price_series.loc[safe_start_date:].bfill().iloc[0]
            df_trace[f'Norm_Price_Benchmark_{benchmark_ticker}'] = norm_bm
        for col in df_trace.columns:
            if 'Norm_Price' in col:
                df_trace[col.replace('Norm_Price', 'Return')] = df_trace[col].pct_change()
        debug_data['portfolio_trace'] = df_trace

    # 10. Package final results
    final_results = {
        'tickers_to_display': tickers_to_display, 'normalized_plot_data': normalized_plot_data,
        'portfolio_series': portfolio_series, 'benchmark_price_series': benchmark_price_series,
        'performance_data': perf_data, 'results_df': results_df, 'actual_calc_end_ts': actual_calc_end_ts,
        'safe_start_date': safe_start_date, 'safe_viz_end_date': safe_viz_end_date,
        'error': None
    }
    return (final_results, debug_data)

# --- C. DYNAMIC DATA QUALITY FILTER FUNCTIONS ---

def calculate_rolling_quality_metrics(df_ohlcv, window=252, min_periods=126, debug=False):
    """Calculates rolling data quality metrics for the entire dataset."""
    print(f"--- Calculating Rolling Quality Metrics (Window: {window} days) ---")
    df = df_ohlcv.copy()
    if not df.index.is_monotonic_increasing:
        df.sort_index(inplace=True)
        
    # Define quality flags
    df['IsStale'] = np.where((df['Volume'] == 0) | (df['Adj High'] == df['Adj Low']), 1, 0)
    df['DollarVolume'] = df['Adj Close'] * df['Volume']
    df['HasSameVolumeAsPrevDay'] = (df.groupby(level='Ticker')['Volume'].diff() == 0).astype(int)
    
    # Calculate rolling metrics per ticker
    grouped = df.groupby(level='Ticker')
    stale_pct = grouped['IsStale'].rolling(window=window, min_periods=min_periods).mean()
    median_vol = grouped['DollarVolume'].rolling(window=window, min_periods=min_periods).median()
    same_vol_count = grouped['HasSameVolumeAsPrevDay'].rolling(window=window, min_periods=min_periods).sum()
    
    quality_df = pd.concat([stale_pct, median_vol, same_vol_count], axis=1)
    quality_df.columns = ['RollingStalePct', 'RollingMedianVolume', 'RollingSameVolCount']
    quality_df.index = quality_df.index.droplevel(0) # Remove the extra 'Ticker' level
    print("✅ Rolling metrics calculation complete.")
    return quality_df

def get_eligible_universe(quality_metrics_df, filter_date, thresholds):
    """Filters the universe of tickers based on quality metrics for a given date."""
    filter_date_ts = pd.to_datetime(filter_date)
    date_index = quality_metrics_df.index.get_level_values('Date').unique().sort_values()
    
    if filter_date_ts < date_index[0]:
        print(f"Warning: Filter date {filter_date_ts.date()} is before the earliest data point. Returning empty universe.")
        return []
        
    # Find the most recent date with quality data on or before the filter date
    valid_prior_dates = date_index[date_index <= filter_date_ts]
    if valid_prior_dates.empty:
        print(f"Warning: No available data found on or before {filter_date_ts.date()}. Returning empty universe.")
        return []
        
    actual_date_to_use = valid_prior_dates[-1]
    if actual_date_to_use.date() != filter_date_ts.date():
        print(f"ℹ️ Info: Filter date {filter_date_ts.date()} not found. Using previous available date {actual_date_to_use.date()}.")

    metrics_on_date = quality_metrics_df.xs(actual_date_to_use, level='Date')
    
    # Apply filters
    mask = ((metrics_on_date['RollingMedianVolume'] >= thresholds['min_median_dollar_volume']) &
            (metrics_on_date['RollingStalePct'] <= thresholds['max_stale_pct']) &
            (metrics_on_date['RollingSameVolCount'] <= thresholds['max_same_vol_count']))
            
    eligible_tickers = metrics_on_date[mask].index.tolist()
    all_tickers = metrics_on_date.index.tolist()
    print(f"Dynamic Filter ({filter_date_ts.date()}): Kept {len(eligible_tickers)} of {len(all_tickers)} tickers.")
    return eligible_tickers    

# --- D. INTERACTIVE ANALYSIS & BACKTESTING TOOLS ---

def plot_walk_forward_analyzer(df_ohlcv, 
                               default_start_date=None, default_calc_period=126, default_fwd_period=63,
                               default_metric='Sharpe (ATR)', default_rank_start=1, default_rank_end=10,
                               default_benchmark_ticker='VOO', master_calendar_ticker='VOO',
                               quality_thresholds={'min_median_dollar_volume': 1_000_000, 'max_stale_pct': 0.05, 'max_same_vol_count': 10},
                               debug=False):
    """Creates an interactive widget for single-period walk-forward analysis."""
    print("Initializing Walk-Forward Analyzer (using Trading Day Logic)...")
    if not isinstance(df_ohlcv.index, pd.MultiIndex): raise ValueError("Input DataFrame must have a (Ticker, Date) MultiIndex.")
    df_ohlcv = df_ohlcv.sort_index()

    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

    # # The following functions are assumed to exist. We define placeholders for them.
    # def calculate_rolling_quality_metrics(df, window):
    #     tickers = df.index.get_level_values(0).unique()
    #     dates = df.index.get_level_values(1).unique()
    #     return pd.DataFrame(index=pd.MultiIndex.from_product([tickers, dates], names=['Ticker', 'Date']))
    # def get_eligible_universe(quality_df, date, thresholds):
    #     tickers = quality_df.index.get_level_values(0).unique()
    #     return list(tickers)
    # def run_walk_forward_step(*args, **kwargs):
    #     # Dummy return structure for demonstration
    #     return {'error': "This is a placeholder function.", 'safe_start_date': pd.Timestamp.now(), 'actual_calc_end_ts': pd.Timestamp.now(), 'safe_viz_end_date': pd.Timestamp.now()}, None

    print("Pre-calculating data quality metrics...")
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    print("Pre-processing data (unstacking)...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    # --- Widget Setup ---
    start_date_picker = widgets.DatePicker(description='Start Date:', value=pd.to_datetime(default_start_date), disabled=False)
    calc_period_input = widgets.IntText(value=default_calc_period, description='Calc Period (days):')
    fwd_period_input = widgets.IntText(value=default_fwd_period, description='Fwd Period (days):')
    metrics = ['Price', 'Sharpe', 'Sharpe (ATR)']
    metric_dropdown = widgets.Dropdown(options=metrics, value=default_metric, description='Metric:')
    rank_start_input = widgets.IntText(value=default_rank_start, description='Rank Start:')
    rank_end_input = widgets.IntText(value=default_rank_end, description='Rank End:')
    benchmark_ticker_input = widgets.Text(value=default_benchmark_ticker, description='Benchmark:', placeholder='Enter Ticker')
    update_button = widgets.Button(description="Update Chart", button_style='primary')
    ticker_list_output = widgets.Output()
    results_container, debug_data_container = [None], [None]

    # --- Plotting Setup ---
    fig = go.FigureWidget()
    max_traces = 50
    for i in range(max_traces): fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name=f'placeholder_{i}', visible=False, showlegend=False))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Benchmark', visible=True, showlegend=True, line=dict(color='black', width=3, dash='dash')))
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', name='Group Portfolio', visible=True, showlegend=True, line=dict(color='green', width=3)))

    # --- Update Logic (Callback) ---
    def update_plot(button_click):
        ticker_list_output.clear_output()
        
        # 1. Get and validate user inputs
        start_date_raw = pd.to_datetime(start_date_picker.value)
        start_date_idx = master_trading_days.searchsorted(start_date_raw)
        if start_date_idx >= len(master_trading_days):
            with ticker_list_output: print(f"Error: Start date is after the last available trading day."); return
        actual_start_date = master_trading_days[start_date_idx]
        with ticker_list_output: 
            if start_date_raw.date() != actual_start_date.date():
                print(f"ℹ️ Info: Start date {start_date_raw.date()} is not a trading day. Snapping forward to {actual_start_date.date()}.")

        # Capture input values into variables
        calc_period = calc_period_input.value
        fwd_period = fwd_period_input.value
        metric = metric_dropdown.value
        rank_start = rank_start_input.value
        rank_end = rank_end_input.value
        benchmark_ticker = benchmark_ticker_input.value.strip().upper()
        
        if rank_start > rank_end:
            with ticker_list_output: print("Error: 'Rank Start' must be <= 'Rank End'."); return
        if rank_start < 1 or calc_period < 2 or fwd_period < 1:
            with ticker_list_output: print("Error: Ranks must be >= 1, Calc Period >= 2, Fwd Period >= 1."); return

        # 1a. Validate data availability
        required_days = calc_period + fwd_period
        if start_date_idx + required_days > len(master_trading_days):
            available_days = len(master_trading_days) - start_date_idx
            last_available_date = master_trading_days[-1].date()
            with ticker_list_output:
                print(f"Error: Not enough data for the requested period.")
                print(f"  Start Date: {actual_start_date.date()}")
                print(f"  Required Days: {calc_period} (calc) + {fwd_period} (fwd) = {required_days}")
                print(f"  Available Days from Start: {available_days} (until {last_available_date})")
                print(f"  Please shorten the 'Calc Period' / 'Fwd Period' or choose an earlier 'Start Date'.")
            return

        # 2. Apply dynamic data quality filter
        eligible_tickers = get_eligible_universe(quality_metrics_df, actual_start_date, quality_thresholds)
        if not eligible_tickers:
            with ticker_list_output: print(f"Error: No eligible tickers found on {actual_start_date.date()} with the current quality filters."); return
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]

        # 3. Run the core calculation
        results, debug_output = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step, master_trading_days,
            actual_start_date, calc_period, fwd_period, 
            metric, rank_start, rank_end, benchmark_ticker, debug=debug
        )
        if results.get('error'):
            with ticker_list_output: print(f"Error: {results['error']}"); return
            
        # ======================= MODIFICATION START =======================
        # 3a. Augment the output containers with period dates and run parameters for external use.
        
        # Add period dates (from previous request)
        period_dates = {
            'calc_period_start': results['safe_start_date'],
            'calc_period_end': results['actual_calc_end_ts'],
            'forward_period_start': results['actual_calc_end_ts'],
            'forward_period_end': results['safe_viz_end_date']
        }
        
        # Add run parameters (new request)
        run_parameters = {
            'calc_period': calc_period,
            'fwd_period': fwd_period,
            'rank_metric': metric,
            'rank_start': rank_start,
            'rank_end': rank_end,
            'benchmark_ticker': benchmark_ticker
        }
        
        # Update the main results dictionary
        results.update(period_dates)
        results.update(run_parameters)

        # Update the debug dictionary if it exists
        if debug_output is not None and isinstance(debug_output, dict):
            debug_output.update(period_dates)
            debug_output.update(run_parameters)
        # ======================= MODIFICATION END =======================

        # 4. Update the interactive plot
        with fig.batch_update():
            # (Plotting code remains unchanged)
            for i in range(max_traces):
                trace = fig.data[i]
                if i < len(results['tickers_to_display']):
                    ticker = results['tickers_to_display'][i]; plot_data_series = results['normalized_plot_data'][ticker]
                    trace.x, trace.y, trace.name, trace.visible, trace.showlegend = plot_data_series.index, plot_data_series.values, ticker, True, True
                else: trace.visible, trace.showlegend = False, False
            benchmark_trace = fig.data[max_traces]
            if results['benchmark_price_series'] is not None and not results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']].dropna().empty:
                normalized_benchmark = results['benchmark_price_series'].loc[results['safe_start_date']:results['safe_viz_end_date']] / results['benchmark_price_series'].loc[results['safe_start_date']:].bfill().iloc[0]
                benchmark_trace.x, benchmark_trace.y, benchmark_trace.name, benchmark_trace.visible = normalized_benchmark.index, normalized_benchmark, f"Benchmark ({benchmark_ticker})", True
            else: benchmark_trace.visible = False
            portfolio_trace = fig.data[max_traces + 1]
            portfolio_trace.x, portfolio_trace.y, portfolio_trace.name, portfolio_trace.visible = results['portfolio_series'].index, results['portfolio_series'], 'Group Portfolio', True
            fig.layout.shapes = []; fig.add_shape(type="line", x0=results['actual_calc_end_ts'], y0=0, x1=results['actual_calc_end_ts'], y1=1, xref='x', yref='paper', line=dict(color="grey", width=2, dash="dash"))
            
        results_container[0] = results; debug_data_container[0] = debug_output
        
        # 5. Display summary statistics in a formatted table
        with ticker_list_output:
            # (Summary display code remains unchanged)
            print(f"Analysis Period: {results['safe_start_date'].date()} to {results['safe_viz_end_date'].date()}.")
            pprint.pprint(results['tickers_to_display'])
            p = results['performance_data']
            rows = []
            rows.append({'Metric': 'Group Portfolio Gain', 'Full': p['full_p_gain'], 'Calc': p['calc_p_gain'], 'Fwd': p['fwd_p_gain']})
            if not np.isnan(p['full_b_gain']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Gain', 'Full': p['full_b_gain'], 'Calc': p['calc_b_gain'], 'Fwd': p['fwd_b_gain']})
                rows.append({'Metric': 'Gain Delta (vs Bm)', 'Full': p['full_p_gain'] - p['full_b_gain'], 'Calc': p['calc_p_gain'] - p['calc_b_gain'], 'Fwd': p['fwd_p_gain'] - p['fwd_b_gain']})
            rows.append({'Metric': 'Group Portfolio Sharpe', 'Full': p['full_p_sharpe'], 'Calc': p['calc_p_sharpe'], 'Fwd': p['fwd_p_sharpe']})
            if not np.isnan(p['full_b_sharpe']):
                rows.append({'Metric': f'Benchmark ({benchmark_ticker}) Sharpe', 'Full': p['full_b_sharpe'], 'Calc': p['calc_b_sharpe'], 'Fwd': p['fwd_b_sharpe']})
                rows.append({'Metric': 'Sharpe Delta (vs Bm)', 'Full': p['full_p_sharpe'] - p['full_b_sharpe'], 'Calc': p['calc_p_sharpe'] - p['calc_b_sharpe'], 'Fwd': p['fwd_p_sharpe'] - p['fwd_b_sharpe']})
            report_df = pd.DataFrame(rows).set_index('Metric')
            gain_rows = [row for row in report_df.index if 'Gain' in row or 'Delta' in row]
            sharpe_rows = [row for row in report_df.index if 'Sharpe' in row]
            styled_df = report_df.style.format('{:+.2%}', na_rep='N/A', subset=(gain_rows, report_df.columns)).format('{:+.2f}', na_rep='N/A', subset=(sharpe_rows, report_df.columns)).set_properties(**{'text-align': 'right', 'width': '100px'}).set_table_styles([{'selector': 'th.col_heading', 'props': [('text-align', 'right')]}, {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}])
            print("\n--- Strategy Performance Summary ---")
            display(styled_df)
            
    # --- Final Layout & Display ---
    fig.update_layout(title_text='Walk-Forward Performance Analysis', xaxis_title='Date', yaxis_title='Normalized Price (Start = 1)', hovermode='x unified', legend_title_text='Tickers (Ranked)', height=600, margin=dict(t=50))
    fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
    update_button.on_click(update_plot)
    controls_row1 = widgets.HBox([start_date_picker, calc_period_input, fwd_period_input])
    controls_row2 = widgets.HBox([metric_dropdown, rank_start_input, rank_end_input, benchmark_ticker_input, update_button])
    ui_container = widgets.VBox([controls_row1, controls_row2, ticker_list_output], layout=widgets.Layout(margin='10px 0 20px 0'))
    display(ui_container, fig)
    update_plot(None) # Initial run
    return (results_container, debug_data_container)

def run_full_backtest(df_ohlcv, strategy_params, quality_thresholds):
    """Runs a full backtest of a strategy over a specified date range."""
    print(f"--- Running Full Forensic Backtest for Strategy: {strategy_params['metric']} (Top {strategy_params['rank_start']}-{strategy_params['rank_end']}) ---")
    
    # 1. Unpack strategy parameters
    start_date, end_date = pd.to_datetime(strategy_params['start_date']), pd.to_datetime(strategy_params['end_date'])
    calc_period, fwd_period = strategy_params['calc_period'], strategy_params['fwd_period']
    metric, rank_start, rank_end = strategy_params['metric'], strategy_params['rank_start'], strategy_params['rank_end']
    benchmark_ticker = strategy_params['benchmark_ticker']
    master_calendar_ticker = strategy_params.get('master_calendar_ticker', 'VOO')
    
    # 2. Perform initial setup (same as analyzer)
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    
    start_idx = master_trading_days.searchsorted(start_date)
    end_idx = master_trading_days.searchsorted(end_date, side='right')
    
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0); df_high_full = df_ohlcv['Adj High'].unstack(level=0); df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    # 3. Loop through all periods in the backtest range
    step_indices = range(start_idx, end_idx, fwd_period)
    all_fwd_gains, period_by_period_debug = [], {}

    print(f"Simulating {len(step_indices)} periods from {master_trading_days[step_indices[0]].date()} to {master_trading_days[step_indices[-1]].date()}...")
    for current_idx in tqdm(step_indices, desc="Backtest Progress"):
        step_date = master_trading_days[current_idx]
        
        # Apply data quality filter for the current step
        eligible_tickers = get_eligible_universe(quality_metrics_df, step_date, quality_thresholds)
        if not eligible_tickers: continue
        
        df_close_step = df_close_full[eligible_tickers]; df_high_step = df_high_full[eligible_tickers]; df_low_step = df_low_full[eligible_tickers]
        
        # Run a single walk-forward analysis step
        results, debug_output = run_walk_forward_step(
            df_close_step, df_high_step, df_low_step, master_trading_days,
            step_date, calc_period, fwd_period,
            metric, rank_start, rank_end, benchmark_ticker, debug=True
        )
        
        # Collect results for this period
        if results['error'] is None:
            fwd_series = results['portfolio_series'].loc[results['actual_calc_end_ts']:]
            all_fwd_gains.append(fwd_series.pct_change().dropna())
            period_by_period_debug[step_date.date().isoformat()] = debug_output
            
    if not all_fwd_gains:
        print("Error: No valid periods were simulated."); return None

    # 4. Stitch together the results to form a continuous equity curve
    strategy_returns = pd.concat(all_fwd_gains); strategy_equity_curve = (1 + strategy_returns).cumprod()
    benchmark_returns = df_close_full[benchmark_ticker].pct_change().loc[strategy_equity_curve.index]; benchmark_equity_curve = (1 + benchmark_returns).cumprod()
    cumulative_equity_df = pd.DataFrame({'Strategy_Equity': strategy_equity_curve, 'Benchmark_Equity': benchmark_equity_curve})
    
    # 5. Plot the final equity curve
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Strategy_Equity'], name='Strategy', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=cumulative_equity_df.index, y=cumulative_equity_df['Benchmark_Equity'], name=f'Benchmark ({benchmark_ticker})', line=dict(color='black', dash='dash')))
    fig.update_layout(title=f"Cumulative Performance: '{metric}' Strategy (Top {rank_start}-{rank_end})", xaxis_title="Date", yaxis_title="Cumulative Growth")
    fig.show()

    # 6. Return the detailed results for forensic analysis
    final_backtest_results = {'cumulative_performance': cumulative_equity_df, 'period_by_period_debug': period_by_period_debug}
    print("\n✅ Full backtest complete. Results object is ready for forensic analysis.")
    return final_backtest_results

# --- E. VERIFICATION TOOLS (User Requested) ---

def verify_group_tickers_walk_forward_calculation(df_ohlcv, tickers_to_verify, benchmark_ticker,
                                                  start_date, calc_period, fwd_period,
                                                  master_calendar_ticker='VOO', export_csv=False):
    """Verifies portfolio and benchmark performance and optionally exports the data."""
    display(Markdown(f"## Verification Report for Portfolio vs. Benchmark"))
    display(Markdown(f"**Portfolio Tickers:** `{tickers_to_verify}`\n**Benchmark Ticker:** `{benchmark_ticker}`"))
    
    # 1. Setup trading day calendar and determine exact period dates
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    if start_idx >= len(master_trading_days):
        print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
    actual_start_date = master_trading_days[start_idx]
    
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    fwd_end_idx = min(calc_end_idx + fwd_period, len(master_trading_days) - 1)
    
    actual_calc_end_date = master_trading_days[calc_end_idx]
    actual_fwd_end_date = master_trading_days[fwd_end_idx]
    
    display(Markdown(f"**Analysis Start:** `{actual_start_date.date()}` (Selected: `{start_date_raw.date()}`)\n"
                    f"**Calc End:** `{actual_calc_end_date.date()}` ({calc_period} trading days)\n"
                    f"**Fwd End:** `{actual_fwd_end_date.date()}` ({fwd_period} trading days)"))

    # 2. Recreate the portfolio and benchmark series from scratch
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    portfolio_prices_raw_slice = df_close_full[tickers_to_verify].loc[actual_start_date:actual_fwd_end_date]
    portfolio_value_series = portfolio_prices_raw_slice.div(portfolio_prices_raw_slice.bfill().iloc[0]).mean(axis=1)
    benchmark_price_series = df_close_full.get(benchmark_ticker)
    
    # 3. Optionally export the underlying daily data to a CSV for external checking
    if export_csv:
        export_df = pd.DataFrame({
            'Portfolio_Normalized_Price': portfolio_value_series,
            'Portfolio_Daily_Return': portfolio_value_series.pct_change()
        })
        if benchmark_price_series is not None:
            norm_bm = benchmark_price_series.loc[actual_start_date:actual_fwd_end_date]
            norm_bm = norm_bm / norm_bm.bfill().iloc[0]
            export_df['Benchmark_Normalized_Price'] = norm_bm
            export_df['Benchmark_Daily_Return'] = norm_bm.pct_change()

        output_dir = 'export_csv'
        os.makedirs(output_dir, exist_ok=True)
        tickers_str = '_'.join(tickers_to_verify)
        filename = f"verify_group_{actual_start_date.date()}_{tickers_str}.csv"
        filepath = os.path.join(output_dir, filename)
        export_df.to_csv(filepath)
        print(f"\n✅ Data exported to: {filepath}")

    # 4. Define a helper to print detailed calculation steps
    def print_verification_steps(title, price_series):
        display(Markdown(f"#### Verification for: `{title}`"))
        if price_series.dropna().shape[0] < 2: print("  - Not enough data points."); return {'gain': np.nan, 'sharpe': np.nan}
        start_price, end_price = price_series.bfill().iloc[0], price_series.ffill().iloc[-1]
        gain = (end_price / start_price) - 1
        print(f"Start Value ({price_series.first_valid_index().date()}): {start_price:,.4f}\nEnd Value   ({price_series.last_valid_index().date()}): {end_price:,.4f}\nGain = {gain:.2%}")
        returns = price_series.pct_change()
        sharpe = calculate_sharpe(returns)
        print(f"Mean Daily Return: {returns.mean():.6f}\nStd Dev: {returns.std():.6f}\nSharpe = {sharpe:.2f}")
        return {'gain': gain, 'sharpe': sharpe}

    # 5. Run verification for each period
    display(Markdown("### A. Calculation Period"))
    perf_calc_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_start_date:actual_calc_end_date])
    if benchmark_price_series is not None:
        perf_calc_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_start_date:actual_calc_end_date])
    
    display(Markdown("### B. Forward Period"))
    perf_fwd_p = print_verification_steps("Group Portfolio", portfolio_value_series.loc[actual_calc_end_date:actual_fwd_end_date])
    if benchmark_price_series is not None:
        perf_fwd_b = print_verification_steps(f"Benchmark", benchmark_price_series.loc[actual_calc_end_date:actual_fwd_end_date])

def verify_ticker_ranking_metrics(df_ohlcv, ticker, start_date, calc_period,
                                  master_calendar_ticker='VOO', export_csv=False):
    """Verifies ranking metrics for a single ticker and optionally exports the data."""
    display(Markdown(f"## Verification Report for Ticker Ranking: `{ticker}`"))
    
    # 1. Setup trading day calendar and determine exact period dates
    if master_calendar_ticker not in df_ohlcv.index.get_level_values(0):
        raise ValueError(f"Master calendar ticker '{master_calendar_ticker}' not found in DataFrame.")
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

    start_date_raw = pd.to_datetime(start_date)
    start_idx = master_trading_days.searchsorted(start_date_raw)
    if start_idx >= len(master_trading_days):
        print(f"Error: Start date {start_date_raw.date()} is after the last available trading day."); return
    actual_start_date = master_trading_days[start_idx]
    
    calc_end_idx = min(start_idx + calc_period, len(master_trading_days) - 1)
    actual_calc_end_date = master_trading_days[calc_end_idx]

    # 2. Extract and prepare the raw data for the specific ticker and period
    df_ticker = df_ohlcv.loc[ticker].sort_index()
    calc_df = df_ticker.loc[actual_start_date:actual_calc_end_date].copy()
    if calc_df.empty or len(calc_df) < 2: 
        print("No data or not enough data in calc period."); return

    display(Markdown("### A. Calculation Period (for Ranking Metrics)"))
    display(Markdown(f"**Period Start:** `{actual_start_date.date()}`\n"
                    f"**Period End:** `{actual_calc_end_date.date()}`\n"
                    f"**Total Trading Days:** `{len(calc_df)}` (Requested: `{calc_period}`)"))
    
    display(Markdown("#### Detailed Metric Calculation Data"))
    
    # 3. Calculate all intermediate metrics as new columns for full transparency
    vdf = calc_df[['Adj High', 'Adj Low', 'Adj Close']].copy()
    vdf['Daily_Return'] = vdf['Adj Close'].pct_change()
    
    # Corrected True Range (TR) calculation for a single ticker (Series)
    tr_df = pd.DataFrame({
        'h_l': vdf['Adj High'] - vdf['Adj Low'],
        'h_cp': abs(vdf['Adj High'] - vdf['Adj Close'].shift(1)),
        'l_cp': abs(vdf['Adj Low'] - vdf['Adj Close'].shift(1))
    })
    vdf['TR'] = tr_df.max(axis=1)
    
    vdf['ATR_14'] = vdf['TR'].ewm(alpha=1/14, adjust=False).mean()
    vdf['ATRP'] = vdf['ATR_14'] / vdf['Adj Close']
    
    print("--- Start of Calculation Period ---")
    display(vdf.head())
    print("\n--- End of Calculation Period ---")
    display(vdf.tail())

    # 4. Optionally export this detailed breakdown to CSV
    if export_csv:
        output_dir = 'export_csv'
        os.makedirs(output_dir, exist_ok=True)
        filename = f"verify_ticker_{actual_start_date.date()}_{ticker}.csv"
        filepath = os.path.join(output_dir, filename)
        vdf.to_csv(filepath)
        print(f"\n✅ Data exported to: {filepath}")
    
    # 5. Print final metric calculations with formulas
    display(Markdown("#### `MetricValue` Verification Summary:"))
    
    calc_start_price = vdf['Adj Close'].bfill().iloc[0]
    calc_end_price = vdf['Adj Close'].ffill().iloc[-1]
    price_metric = (calc_end_price / calc_start_price)
    print(f"1. Price Metric: (Last Price / First Price) = ({calc_end_price:.2f} / {calc_start_price:.2f}) = {price_metric:.4f}")
    
    daily_returns = vdf['Daily_Return'].dropna()
    sharpe_ratio = calculate_sharpe(daily_returns)
    print(f"2. Sharpe Metric: (Mean Daily Return / Std Dev) * sqrt(252) = {sharpe_ratio:.4f}")

    atrp_mean = vdf['ATRP'].mean()
    mean_daily_return = vdf['Daily_Return'].mean()
    sharpe_atr = (mean_daily_return / atrp_mean) if atrp_mean > 0 else 0
    print(f"3. Sharpe (ATR) Metric: (Mean Daily Return / Mean ATRP) = ({mean_daily_return:.6f} / {atrp_mean:.6f}) = {sharpe_atr:.4f}")

# --- F. AUTOMATION SCRIPT - STRATEGY SEARCH ---

def run_strategy_search(df_ohlcv, config):
    """
    Runs the main backtesting loop with checkpointing to be resumable.
    """
    start_time = time.time() # <-- This now works because of 'import time'
    
    # --- 1. SETUP & LOAD PROGRESS ---
    print("--- Phase 1: Pre-processing and Loading Progress ---")
    quality_metrics_df = calculate_rolling_quality_metrics(df_ohlcv, window=252)
    print("Unstacking data for performance...")
    df_close_full = df_ohlcv['Adj Close'].unstack(level=0)
    df_high_full = df_ohlcv['Adj High'].unstack(level=0)
    df_low_full = df_ohlcv['Adj Low'].unstack(level=0)
    
    master_calendar_ticker = config['master_calendar_ticker']
    master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()
    print(f"Master trading day calendar created from '{master_calendar_ticker}' ({len(master_trading_days)} days).")

    results_path = config['results_output_path']
    completed_params = set()
    
    if os.path.exists(results_path): # <-- This now works because of 'import os'
        print(f"Found existing results file. Loading progress from: {results_path}")
        df_progress = pd.read_csv(results_path)
        for _, row in df_progress.iterrows():
            param_key = (
                row['calc_period'], row['fwd_period'], row['metric'],
                (row['rank_start'], row['rank_end'])
            )
            completed_params.add(param_key)
        print(f"Found {len(completed_params)} completed parameter sets to skip.")
    else:
        print("No existing results file found. Starting a new run.")

    print("✅ Pre-processing complete.\n")

    # --- 2. SETUP THE MAIN LOOP ---
    print("--- Phase 2: Setting up Simulation Loops ---")
    
    param_combinations = list(product(
        config['calc_periods'], config['fwd_periods'],
        config['metrics'], config['rank_slices']
    ))
    
    search_start_date = pd.to_datetime(config['search_start_date'])
    search_end_date = pd.to_datetime(config['search_end_date'])
    start_idx = master_trading_days.searchsorted(search_start_date, side='left')
    end_idx = master_trading_days.searchsorted(search_end_date, side='right')

    step_dates_map = {}
    print("Pre-calculating rebalancing schedules for each holding period...")
    for fwd_period in sorted(config['fwd_periods']):
        step_indices = range(start_idx, end_idx, fwd_period)
        step_dates_map[fwd_period] = master_trading_days[step_indices]
        print(f"  - Holding Period {fwd_period} days: {len(step_dates_map[fwd_period])} rebalances")
    
    print(f"Found {len(param_combinations)} total parameter sets to simulate.")
    print("✅ Setup complete. Starting main loop...\n")

    # --- 3. RUN THE MAIN LOOP ---
    print("--- Phase 3: Running Simulations ---")
    pbar = tqdm(param_combinations, desc="Parameter Sets")
    
    for params in pbar:
        calc_period, fwd_period, metric, rank_slice = params
        rank_start, rank_end = rank_slice
        
        param_key = (calc_period, fwd_period, metric, rank_slice)
        if param_key in completed_params:
            pbar.set_description(f"Skipping {param_key}")
            continue

        pbar.set_description(f"Running {param_key}")
        
        current_params_results = []
        
        # ==============================================================================
        # --- FIX: RESTORED THE MISSING INNER LOOP ---
        # ==============================================================================
        current_step_dates = step_dates_map[fwd_period]
        for step_date in current_step_dates:
            eligible_tickers = get_eligible_universe(
                quality_metrics_df, filter_date=step_date, thresholds=config['quality_thresholds']
            )
            if not eligible_tickers: continue
            
            df_close_step = df_close_full[eligible_tickers]
            df_high_step = df_high_full[eligible_tickers]
            df_low_step = df_low_full[eligible_tickers]

            step_result, _ = run_walk_forward_step(
                df_close_full=df_close_step, df_high_full=df_high_step, df_low_full=df_low_step,
                master_trading_days=master_trading_days, start_date=step_date,
                calc_period=calc_period, fwd_period=fwd_period,
                metric=metric, rank_start=rank_start, rank_end=rank_end,
                benchmark_ticker=config['benchmark_ticker'], debug=False
            )
            
            if step_result['error'] is None:
                p = step_result['performance_data']
                log_entry = {
                    'step_date': step_date.date(), 'calc_period': calc_period,
                    'fwd_period': fwd_period, 'metric': metric,
                    'rank_start': rank_start, 'rank_end': rank_end,
                    'num_universe': len(eligible_tickers),
                    'num_portfolio': len(step_result['tickers_to_display']),
                    'fwd_p_gain': p['fwd_p_gain'], 'fwd_b_gain': p['fwd_b_gain'],
                    'fwd_gain_delta': p['fwd_p_gain'] - p['fwd_b_gain'] if not np.isnan(p['fwd_b_gain']) else np.nan,
                    'fwd_p_sharpe': p['fwd_p_sharpe'],
                }
                current_params_results.append(log_entry)
        # ==============================================================================
        
        # --- CHECKPOINTING: INCREMENTAL SAVE ---
        if current_params_results:
            df_to_append = pd.DataFrame(current_params_results)
            df_to_append.to_csv(
                results_path,
                mode='a',
                header=not os.path.exists(results_path),
                index=False
            )
            completed_params.add(param_key)

    print("✅ Main loop finished.\n")
    
    # --- 4. RETURN FINAL DATAFRAME ---
    print("--- Phase 4: Loading Final Results ---")
    if os.path.exists(results_path):
        final_df = pd.read_csv(results_path)
        end_time = time.time()
        print(f"✅ Process complete. Total execution time: {time.time() - start_time:.2f} seconds.")
        return final_df
    else:
        print("Warning: No results were generated.")
        return None
    


#### **CELL 3: DATA LOADING**
*This cell loads your main dataset using the environment-aware path.*

In [ ]:
# ==============================================================================
# --- CELL 3: DATA LOADING ---
# ==============================================================================
import pandas as pd

data_file_path = env_config['data_path']
print(f"Attempting to load data from: {data_file_path}")

try:
    df_OHLCV = pd.read_parquet(data_file_path, engine='pyarrow')
    df_dev = df_OHLCV.copy() # Use df_dev for development as a good practice
    
    print("\n✅ Data loaded successfully.")
    print("\n--- DataFrame Info ---")
    df_dev.info()
except FileNotFoundError:
    print(f"\n❌ ERROR: FILE NOT FOUND at {data_file_path}. Please check paths in Cell 1.")

#### **CELL 4: BOT STRATEGY CONFIGURATION**
*This cell defines the strategy parameters you want to test.*


In [ ]:
# ==============================================================================
# --- CELL 4: BOT STRATEGY CONFIGURATION ---
# ==============================================================================
from itertools import product
import pandas as pd

# --- PRIMARY USER INPUTS FOR THE STRATEGY ---
# 5,  21, 42, 63, 126, 252
# 1W, 1M, 2M, 3M,  6M,  1Y
HOLDING_PERIODS_DAYS = [63]        # Test ~2, and 3 month holding periods
CALC_PERIODS_DAYS = [252]         # Use ~6 and 12 month lookbacks

bot_config = {
    # --- Time Parameters ---
    'search_start_date': '2014-01-01',
    'search_end_date': '2018-12-31',
    
    # --- Strategy Parameters (The Search Grid) ---
    'calc_periods': CALC_PERIODS_DAYS,
    'fwd_periods': HOLDING_PERIODS_DAYS,


    # 'metrics': ['Sharpe', 'Sharpe (ATR)'],
    'metrics': ['Price', 'Sharpe', 'Sharpe (ATR)'],    
        
    'rank_slices': [(1, 5)],

    # --- Data Quality ---
    'quality_thresholds': { 'min_median_dollar_volume': 10_000_000, 
                            'max_stale_pct': 0.05, 
                            'max_same_vol_count': 1 },

    # --- General Parameters ---
    'benchmark_ticker': 'VOO',
    'master_calendar_ticker': 'VOO',
    'results_output_path': env_config['results_path']
}

print("\n--- Bot Configuration Initialized ---")
print(f"Calculation Periods to Test: {bot_config['calc_periods']} trading days")
print(f"Forward and Holding Periods to Test (Forward and Holding Periods are the same): {bot_config['fwd_periods']} trading days")
print(f"Results will be saved to: {bot_config['results_output_path']}")


#### **CELL 5: EXECUTION**
*This is the final cell that runs the backtest and displays the results.*

In [ ]:
# ==============================================================================
# --- CELL 5: EXECUTION ---
# ==============================================================================

# --- Execute the Bot ---
dev_results_df = run_strategy_search(df_dev, bot_config)

# --- Display a sample of the results ---
if dev_results_df is not None:
    print("\n--- Sample of Generated Results ---")
    display(dev_results_df.head())
    print("\n--- Analysis of Best Performing Strategies ---")
    display(dev_results_df.groupby(['calc_period', 'fwd_period', 'metric', 'rank_start', 'rank_end'])['fwd_gain_delta'].mean().sort_values(ascending=False).to_frame())

### 4. Next Steps & Future Improvements

This system is a powerful foundation. Here are potential areas for future development:
1.  **Advanced Performance Analytics:** Create a new notebook or function to analyze the output CSV, calculating metrics like Max Drawdown, Calmar Ratio, and generating equity curves for the best strategies.
2.  **Visualization:** Build heatmaps and other plots to visualize how different parameters (e.g., `calc_period` vs. `fwd_period`) affect performance.
3.  **Realism:** Incorporate transaction costs and slippage into the performance calculations for a more realistic backtest.
4.  **Configuration Management:** For even more complex tests, move the `bot_config` dictionary into a separate `config.py` file to keep the notebook cleaner.

It has been a genuine pleasure working with you on this. You've built an impressive and professional-grade tool. I wish you the very best with your continued research and development

### 4. Plot an export_csv Row to Check


In [ ]:
row_to_check = 59
row_values =  dev_results_df.loc[row_to_check ].to_dict()
print(f'export_csv values for row {row_to_check}:\n')
for k, v in row_values.items():
    print(f'{k:<15}: {v}')


In [ ]:
results_container, debug_container = plot_walk_forward_analyzer(
    df_ohlcv=df_dev,
    # df_ohlcv=df_OHLCV,    
    default_start_date=row_values['step_date'],
    
    default_calc_period=row_values['calc_period'],
    default_fwd_period=row_values['fwd_period'],
    # default_calc_period=120,
    # default_fwd_period=30,

    default_metric=row_values['metric'],

    default_rank_start=row_values['rank_start'],
    default_rank_end=row_values['rank_end'],
    # default_rank_start=2,
    # default_rank_end=3,    

    default_benchmark_ticker='VOO',
    quality_thresholds=bot_config['quality_thresholds'],
    debug=True  # <-- Activate the new mode!
)

In [ ]:
print_nested(results_container)

# CHECK Metric_Sharpe (ATR) for Calc. Period has error.
- start date 2018-10-03
- Calc Period 252
- Fwd Period 63
- Metric Sharpe ATR
- Rank Start 1
- Rank End 5
-- Analysis Period 2018-10-03 to 2020-01-06 (this is full period, calc + fwd)
-- [BIL, MINT, SHV, BNDX, VCSH]
-- Metric_Sharpe (ATR) for BIL, MINT, SHV, BNDX matched my own calculation at bottom cell
-- Metric Sharpe (ATR) for VCSH is a bit off (code calc 0.194195, my calc 0.196112)

In [ ]:
print_nested(debug_container)

In [ ]:
results_dict = results_container[0]

print_nested(results_dict)


### Get Plot Parameters

In [ ]:
_tickers = results_dict['tickers_to_display']
_calc_start = results_dict['calc_period_start']
_calc_end = results_dict['calc_period_end']
_fwd_start = results_dict['forward_period_start']
_fwd_end = results_dict['forward_period_end']
_calc_period = results_dict['calc_period']
_fwd_period = results_dict['fwd_period']
_benchmark_ticker = results_dict['benchmark_ticker']

print(f'_tickers: {_tickers}')
print(f'_calc_start: {_calc_start}')
print(f'_calc_end: {_calc_end}')
print(f'_fwd_start: {_fwd_start}')
print(f'_fwd_end: {_fwd_end}')
print(f'_calc_period: {_calc_period}')
print(f'_fwd_period: {_fwd_period}')
print(f'_benchmark: {_benchmark_ticker}')

### Run verify_ticker_ranking_metrics with Plot Parameters to Check Calc. Period Calculation 

In [ ]:
for _ticker in _tickers:
    verify_ticker_ranking_metrics(df_OHLCV, 
                                  ticker=_ticker, 
                                  start_date=_calc_start, 
                                  calc_period=_calc_period,
                                  master_calendar_ticker=_benchmark_ticker, 
                                  export_csv=True)

### My Own Check on Calculation for One Ticker

In [ ]:
# -- Check Calculation for a Ticker -- #
_check_ticker = _tickers[4]
print(f'Check calculation for this ticker: {_check_ticker}')

# Get Ticker's OHLCV Data -- # 
_df = df_OHLCV.loc[_check_ticker][_calc_start:_fwd_end]

# -- Calculate Daily Return -- #
_df['Daily_Return'] = _df['Adj Close'].pct_change()

# -- Calculate True Range -- #
_df['TR'] = pd.concat([
    _df['Adj High'] - _df['Adj Low'],
    (_df['Adj High'] - _df['Adj Close'].shift(1)).abs(),
    (_df['Adj Low']  - _df['Adj Close'].shift(1)).abs()
], axis=1).max(axis=1)

# -- Calculate Average True Range (14 day period) -- #
window = 14
_df['ATR_14'] = pd.NA

# Seed the very first ATR value with the first non-NaN TR
first_idx = _df['TR'].first_valid_index()
_df.loc[first_idx, 'ATR_14'] = _df.loc[first_idx, 'TR']

# Iteratively apply the Wilder smoothing formula
for i in range(_df.index.get_loc(first_idx) + 1, len(_df)):
    prev_atr = _df.iloc[i-1]['ATR_14']
    curr_tr  = _df.iloc[i]['TR']
    _df.iloc[i, _df.columns.get_loc('ATR_14')] = (prev_atr * (window - 1) + curr_tr) / window

# -- Calculate ATRP -- #
_df['ATRP'] = _df['ATR_14'] / _df['Adj Close']

calc_pd_df = _df.loc[_calc_start:_calc_end]
fwd_pd_df = _df.loc[_fwd_start:_fwd_end]
print(f'Calc. Period:\n{calc_pd_df.head()}\n{calc_pd_df.tail()}')
print(f'\nFwd. Period:\n{fwd_pd_df.head()}\n{fwd_pd_df.tail()}')

In [ ]:
print(f'Metric Calculation for Ticker: {_check_ticker}')

# -- Calculation for Period Gain -- #
full_period_gain = _df['Adj Close'][_fwd_end] / _df['Adj Close'][_calc_start]
calc_period_gain = _df['Adj Close'][_calc_end] / _df['Adj Close'][_calc_start]
fwd_period_gain = _df['Adj Close'][_fwd_end] / _df['Adj Close'][_fwd_start]

print(f'\nfull_period_gain: {full_period_gain:.4f}')
print(f'calc_period_gain: {calc_period_gain:.4f}')
print(f'fwd_period_gain: {fwd_period_gain:.4f}')

# -- Calculation for Period Sharpe -- #
full_period_return = _df['Daily_Return'][_calc_start:_fwd_end]
calc_period_return = _df['Daily_Return'][_calc_start:_calc_end]
fwd_period_return = _df['Daily_Return'][_fwd_start:_fwd_end]

full_sharpe = full_period_return.mean() / full_period_return.std() * (252 ** 0.5)
calc_sharpe = calc_period_return.mean() / calc_period_return.std() * (252 ** 0.5)
fwd_sharpe = fwd_period_return.mean() / fwd_period_return.std() * (252 ** 0.5)

print(f'\nfull_sharpe: {full_sharpe:.4f}')
print(f'calc_sharpe: {calc_sharpe:.4f}')
print(f'fwd_sharpe: {fwd_sharpe:.4f}')

# -- Calculation for Period Sharpe ATR -- #
full_sharpe_ATR = full_period_return.mean() / _df['ATRP'][_calc_start:_fwd_end].mean()
calc_sharpe_ATR = calc_period_return.mean() / _df['ATRP'][_calc_start:_calc_end].mean()
fwd_sharpe_ATR = fwd_period_return.mean() / _df['ATRP'][_fwd_start:_fwd_end].mean()

print(f'\nfull_sharpe_ATR: {full_sharpe_ATR:.4f}')
print(f'calc_sharpe_ATR: {calc_sharpe_ATR:.4f}')
print(f'fwd_sharpe_ATR: {fwd_sharpe_ATR:.4f}')

print(f'\ncalc_period_return.mean(): {calc_period_return.mean()}')
print(f"_df['ATRP'][_calc_start:_calc_end].mean(): {_df['ATRP'][_calc_start:_calc_end].mean()}")
print(f'calc_sharpe_ATR: {calc_sharpe_ATR}')

In [ ]:
df = df_OHLCV.loc['VCSH'].copy()
df


In [ ]:
_df.info()

In [ ]:
# -- Check Calculation for a Ticker -- #
_check_ticker = _tickers[4]
print(f'Check calculation for this ticker: {_check_ticker}')

# Get Ticker's OHLCV Data -- # 
_df = df_OHLCV.loc[_check_ticker]['2018-09-30' : '2020-01-10']
# _df = df_OHLCV.loc[_check_ticker].copy()

# -- Calculate Daily Return -- #
_df['Daily_Return'] = _df['Adj Close'].pct_change()

# -- Calculate True Range -- #
_df['TR'] = pd.concat([
    _df['Adj High'] - _df['Adj Low'],
    (_df['Adj High'] - _df['Adj Close'].shift(1)).abs(),
    (_df['Adj Low']  - _df['Adj Close'].shift(1)).abs()
], axis=1).max(axis=1)

# -- Calculate Average True Range (14 day period) -- #
window = 14
_df['ATR_14'] = pd.NA

# Seed the very first ATR value with the first non-NaN TR
first_idx = _df['TR'].first_valid_index()
_df.loc[first_idx, 'ATR_14'] = _df.loc[first_idx, 'TR']

# Iteratively apply the Wilder smoothing formula
for i in range(_df.index.get_loc(first_idx) + 1, len(_df)):
    prev_atr = _df.iloc[i-1]['ATR_14']
    curr_tr  = _df.iloc[i]['TR']
    _df.iloc[i, _df.columns.get_loc('ATR_14')] = (prev_atr * (window - 1) + curr_tr) / window

# -- Calculate ATRP -- #
_df['ATRP'] = _df['ATR_14'] / _df['Adj Close']

# calc_pd_df = _df.loc[_calc_start:_calc_end]
# fwd_pd_df = _df.loc[_fwd_start:_fwd_end]
# print(f'Calc. Period:\n{calc_pd_df.head()}\n{calc_pd_df.tail()}')
# print(f'\nFwd. Period:\n{fwd_pd_df.head()}\n{fwd_pd_df.tail()}')

_df

In [ ]:
df_OHLCV.info()

### Below Cells Follows the Code to Calculate Sharpe (ATR)

In [ ]:
df_ohlcv = df_OHLCV.copy()

In [ ]:
ticker_to_check = 'VCSH' # <--- CHANGE THIS TO YOUR ACTUAL TICKER
start_date_raw = pd.to_datetime('2018-10-03')
calc_period_days = 252

# We need the master trading day calendar, just like the code uses.
# It's essential for getting the dates exactly right.
master_calendar_ticker = 'VOO' # Or another reliable ticker like 'SPY'
master_trading_days = df_ohlcv.loc[master_calendar_ticker].index.unique().sort_values()

# Isolate the full history for the ticker we're checking
df_ticker_full = df_ohlcv.loc[ticker_to_check]

In [ ]:
df_ticker_full

In [ ]:
# Find the index for our start date
start_idx = master_trading_days.searchsorted(start_date_raw)
actual_start_date = master_trading_days[start_idx]

# Find the index for the end of the calculation period
calc_end_idx = min(start_idx + calc_period_days, len(master_trading_days) - 1)
actual_calc_end_date = master_trading_days[calc_end_idx]

print(f"Raw Start Date: {start_date_raw.date()}")
print(f"Actual Start Date (Trading Day): {actual_start_date.date()}")
print(f"Actual Calc End Date (Trading Day): {actual_calc_end_date.date()}")

In [ ]:
# Slice the ticker's data to the exact date range
calc_df = df_ticker_full.loc[actual_start_date:actual_calc_end_date].copy()

print(f"Number of rows in calc_df: {len(calc_df)}")
print("--- First 3 rows of our calculation data ---")
display(calc_df.head())

In [ ]:
calc_df['Daily_Return'] = calc_df['Adj Close'].pct_change()
mean_daily_return = calc_df['Daily_Return'].mean()

print(f"Mean Daily Return: {mean_daily_return:.8f}")

In [ ]:
# 4a. Get the previous day's close for every day in our calc period.
# This is done by shifting the FULL history, then slicing.
prev_close_series = df_ticker_full['Adj Close'].shift(1).loc[calc_df.index]

# 4b. MY HYPOTHESIS: The first value of this series is NaN. Let's check.
print("--- Previous Day's Close (first 3 days) ---")
print(prev_close_series.head(3))
print("\n")

# 4c. Now calculate the three components of TR
component1 = calc_df['Adj High'] - calc_df['Adj Low']
component2 = abs(calc_df['Adj High'] - prev_close_series)
component3 = abs(calc_df['Adj Low'] - prev_close_series)

# 4d. Combine them to get the daily TR value
tr_df = pd.DataFrame({'c1': component1, 'c2': component2, 'c3': component3})
calc_df['TR'] = tr_df.max(axis=1)

# # Change TR to High - Low for the row 0
# calc_df.loc[calc_df.index[0], 'TR'] = (
#     calc_df.loc[calc_df.index[0], 'Adj High'] -
#     calc_df.loc[calc_df.index[0], 'Adj Low']
# )

print("--- Final TR values (first 3 days) ---")
print(calc_df[['Adj Close', 'TR']].head(3))

In [ ]:
# Calculate the ATR using the exact same parameters as the code
calc_df['ATR_14'] = calc_df['TR'].ewm(alpha=1/14, adjust=False).mean()

print("--- TR vs ATR_14 (first 5 days) ---")
display(calc_df[['TR', 'ATR_14']].head(5))

In [ ]:
# ATRP is the ATR divided by the close price
calc_df['ATRP'] = calc_df['ATR_14'] / calc_df['Adj Close']

# The final denominator is the mean of the daily ATRP values over the period
atrp_mean = calc_df['ATRP'].mean()

print(f"Mean ATRP (Denominator): {atrp_mean:.8f}")

In [ ]:
metric_sharpe_atr_manual = mean_daily_return / atrp_mean

print("--- FINAL VERIFICATION ---")
print(f"Numerator (MeanDailyReturn): {mean_daily_return:.8f}")
print(f"Denominator (ATRP_Mean):     {atrp_mean:.8f}")
print(f"Calculated Sharpe (ATR):     {metric_sharpe_atr_manual:.6f}")